In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas_market_calendars import get_calendar
from datetime import datetime, date, timedelta
import awswrangler as wr
import boto3
import pytz
import warnings
import pandas_ta as ta
import concurrent.futures

# Ignore warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Establish AWS session
session = boto3.session.Session(
# Uncomment and add your AWS credentials here if necessary
#     aws_access_key_id='your_access_key_id',
#     aws_secret_access_key='your_secret_access_key',
#     aws_session_token='your_session_token',
    region_name='ap-south-1'
)


In [2]:
s3_bucket = 'sg-ds-project'
s3_misc_prefix = 'alpha-generator/misc/'

In [3]:
nifty500_co_code = wr.s3.read_csv(f's3://{s3_bucket}/{s3_misc_prefix}Master_final_list.csv',boto3_session = session)
nifty500_co_code['co_code'] = nifty500_co_code['co_code'].astype('int64')

In [4]:
nifty500_co_code.head()

nse_symbol  co_code  is_nifty500
0  APOLLOHOSP      636            1
1  KALYANKJIL    42402            1
2       ARE&M     2110            1
3    MTARTECH    34242            1
4   PATANJALI      704            1

In [4]:
# print('Cleaning & parsing raw data started ... ')
# df_open = pd.read_excel(r"D:\alpha_generator\alpha_short_calls\data\ohlcv.xlsx", sheet_name='open', header=3, skiprows=[4, 5])
# df_open.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

# df_high = pd.read_excel(r"D:\alpha_generator\alpha_short_calls\data\ohlcv.xlsx", sheet_name='high', header=3, skiprows=[4, 5])
# df_high.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

# df_low = pd.read_excel(r"D:\alpha_generator\alpha_short_calls\data\ohlcv.xlsx", sheet_name='low', header=3, skiprows=[4, 5])
# df_low.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

# df_close = pd.read_excel(r"D:\alpha_generator\alpha_short_calls\data\ohlcv.xlsx", sheet_name='close', header=3, skiprows=[4, 5])
# df_close.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

# df_volume = pd.read_excel(r"D:\alpha_generator\alpha_short_calls\data\ohlcv.xlsx", sheet_name='volume', header=3, skiprows=[4, 5])
# df_volume.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

In [5]:
print('Cleaning & parsing raw data started ... ')
df_open = pd.read_excel(f's3://sg-ds-project/alpha-generator/quant-team-data/ohlcv.xlsx', sheet_name='open', header=3, skiprows=[4, 5])
df_open.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

df_high = pd.read_excel(f's3://sg-ds-project/alpha-generator/quant-team-data/ohlcv.xlsx', sheet_name='high', header=3, skiprows=[4, 5])
df_high.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

df_low = pd.read_excel(f's3://sg-ds-project/alpha-generator/quant-team-data/ohlcv.xlsx', sheet_name='low', header=3, skiprows=[4, 5])
df_low.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

df_close = pd.read_excel(f's3://sg-ds-project/alpha-generator/quant-team-data/ohlcv.xlsx', sheet_name='close', header=3, skiprows=[4, 5])
df_close.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

df_volume = pd.read_excel(f's3://sg-ds-project/alpha-generator/quant-team-data/ohlcv.xlsx', sheet_name='volume', header=3, skiprows=[4, 5])
df_volume.rename({'Unnamed: 0': 'Date'}, inplace=True, axis=1)

Cleaning & parsing raw data started ... 


In [6]:
df_open.shape,df_high.shape,df_low.shape,df_volume.shape

((6064, 503), (6064, 503), (6064, 503), (6064, 503))

In [7]:
df_open.Date.min(),df_open.Date.max()

(Timestamp('2000-01-03 00:00:00'), Timestamp('2024-11-04 00:00:00'))

In [8]:
ls_sym = df_open.columns.to_list()[1:]
len(ls_sym)

502

In [9]:
df_final = pd.DataFrame(columns=['date','symbol','open','high','low','close','volume'])

In [10]:
for syb in ls_sym:
    df_s = pd.DataFrame({'date': df_open['Date'],'open': df_open[syb],'high' :df_high[syb],'low': df_low[syb],'close': df_close[syb],'volume':df_volume[syb]})
    df_s['symbol']=syb
    df_final=pd.concat([df_final,df_s])

In [11]:
df_final.shape

(3044128, 7)

In [12]:
df_map = wr.s3.read_csv(f's3://{s3_bucket}/{s3_misc_prefix}bblg_codes_symbol.csv',boto3_session = session)

In [13]:
df_map.head()

co_code      symbol        upper
0    636.0  APOLLOHOSP      APHS IN
1  42402.0  KALYANKJIL  KALYANKJ IN
2   2110.0       ARE&M     ARENM IN
3  34242.0    MTARTECH  MTARTECH IN
4    704.0   PATANJALI  PATANJAL IN

In [14]:
df_map['upper'] = df_map['upper'].apply(lambda x: x.split()[0])
df_final['symbol'] = df_final['symbol'].apply(lambda x: x.split()[0])

In [15]:
raw_cleansed=df_final.merge(df_map,left_on='symbol',right_on='upper',how='left')

In [16]:
raw_cleansed=raw_cleansed[~raw_cleansed['symbol_x'].isin(['LXCHEM', 'AGIS', 'UNSP', 'NIFTY'])]

In [17]:
raw_cleansed['symbol_y']=np.where(raw_cleansed['symbol_x']=='LXCHEM','LXCHEM',raw_cleansed['symbol_y'])
raw_cleansed['symbol_y']=np.where(raw_cleansed['symbol_x']=='UNSP','UNSP',raw_cleansed['symbol_y'])
raw_cleansed['symbol_y']=np.where(raw_cleansed['symbol_x']=='AGIS','AEGISLOG',raw_cleansed['symbol_y'])

In [18]:
del raw_cleansed['upper']
del raw_cleansed['symbol_x']

In [19]:
raw_cleansed.rename(columns={'symbol_y':'nse_symbol'},inplace=True)
raw_cleansed['adj_close']=raw_cleansed['close']
raw_cleansed['is_nifty500']=1

In [20]:
raw_cleansed['date']=pd.to_datetime(raw_cleansed['date'],dayfirst=True)

In [21]:
raw_cleansed['date'].min(),raw_cleansed['date'].max()

(Timestamp('2000-01-03 00:00:00'), Timestamp('2024-11-04 00:00:00'))

In [22]:
raw_cleansed['nse_symbol'].nunique()

498

In [23]:
raw_cleansed['nse_symbol'].value_counts()

APOLLOHOSP    6064
IPCALAB       6064
MINDACORP     6064
UPL           6064
ULTRACEMCO    6064
TORNTPHARM    6064
PRAJIND       6064
DLF           6064
MEDPLUS       6064
ZYDUSLIFE     6064
PHOENIXLTD    6064
HONASA        6064
METROBRAND    6064
CELLO         6064
RBLBANK       6064
ANANDRATHI    6064
GNFC          6064
STARHEALTH    6064
HAVELLS       6064
MGL           6064
LTIM          6064
BATAINDIA     6064
TCS           6064
IDEA          6064
RECLTD        6064
BHARTIARTL    6064
COALINDIA     6064
CESC          6064
POWERGRID     6064
HCLTECH       6064
UTIAMC        6064
BAJFINANCE    6064
FEDERALBNK    6064
BAJAJ-AUTO    6064
HFCL          6064
INDHOTEL      6064
GLENMARK      6064
ASHOKLEY      6064
EIHOTEL       6064
TECHM         6064
GRASIM        6064
OFSS          6064
BALAMINES     6064
CGCL          6064
HUDCO         6064
DIVISLAB      6064
KPIL          6064
SHREECEM      6064
CHAMBLFERT    6064
BLS           6064
SAIL          6064
3MINDIA       6064
PETRONET    

In [24]:
raw_cleansed.head(3)

date    open    high     low   close    volume  co_code  \
6064 2000-01-03  103.92  104.97  103.92  104.97    5400.0    636.0   
6065 2000-01-04  113.37  113.39  113.37  113.39   24500.0    636.0   
6066 2000-01-05  122.41  122.49  113.42  121.69  120100.0    636.0   

      nse_symbol  adj_close  is_nifty500  
6064  APOLLOHOSP     104.97            1  
6065  APOLLOHOSP     113.39            1  
6066  APOLLOHOSP     121.69            1

In [25]:
raw_cleansed.drop(columns=['is_nifty500'],inplace=True)

raw_cleansed.drop(columns=['co_code'],inplace=True)

raw_cleansed.dropna(subset='open',inplace=True)

In [26]:
raw_cleansed.shape

(1986571, 8)

In [27]:
raw_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1986571 entries, 6064 to 3038063
Data columns (total 8 columns):
 #   Column      Dtype         
---  ------      -----         
 0   date        datetime64[ns]
 1   open        float64       
 2   high        float64       
 3   low         float64       
 4   close       float64       
 5   volume      float64       
 6   nse_symbol  object        
 7   adj_close   float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 136.4+ MB


In [28]:
raw_cleansed = raw_cleansed.sort_values(by=['nse_symbol','date'], ascending=[True, True])

In [29]:
raw_cleansed.head()

date    open    high     low   close     volume nse_symbol  \
683992 2019-09-19  302.50  317.63  302.50  317.63  7109860.0     360ONE   
683993 2019-09-20  332.25  333.50  332.25  333.50  2760648.0     360ONE   
683994 2019-09-23  350.18  350.18  350.18  350.18    34096.0     360ONE   
683995 2019-09-24  367.68  367.68  367.68  367.68  2496652.0     360ONE   
683996 2019-09-25  386.05  386.05  349.30  351.24   530580.0     360ONE   

        adj_close  
683992     317.63  
683993     333.50  
683994     350.18  
683995     367.68  
683996     351.24

In [30]:
raw_cleansed['nse_symbol'].unique().tolist()

['360ONE',
 '3MINDIA',
 'AARTIIND',
 'AAVAS',
 'ABB',
 'ABBOTINDIA',
 'ABCAPITAL',
 'ABFRL',
 'ACC',
 'ACE',
 'ACI',
 'ADANIENSOL',
 'ADANIENT',
 'ADANIGREEN',
 'ADANIPORTS',
 'ADANIPOWER',
 'AETHER',
 'AFFLE',
 'AIAENG',
 'AJANTPHARM',
 'ALKEM',
 'ALKYLAMINE',
 'ALLCARGO',
 'ALOKINDS',
 'AMBER',
 'AMBUJACEM',
 'ANANDRATHI',
 'ANGELONE',
 'ANURAS',
 'APARINDS',
 'APLAPOLLO',
 'APLLTD',
 'APOLLOHOSP',
 'APOLLOTYRE',
 'APTUS',
 'ARE&M',
 'ASAHIINDIA',
 'ASHOKLEY',
 'ASIANPAINT',
 'ASTERDM',
 'ASTRAL',
 'ASTRAZEN',
 'ATGL',
 'ATUL',
 'AUBANK',
 'AUROPHARMA',
 'AVANTIFEED',
 'AWL',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'BAJAJHLDNG',
 'BAJFINANCE',
 'BALAMINES',
 'BALKRISIND',
 'BALRAMCHIN',
 'BANDHANBNK',
 'BANKBARODA',
 'BANKINDIA',
 'BATAINDIA',
 'BAYERCROP',
 'BBTC',
 'BDL',
 'BEL',
 'BEML',
 'BERGEPAINT',
 'BHARATFORG',
 'BHARTIARTL',
 'BHEL',
 'BIKAJI',
 'BIOCON',
 'BIRLACORPN',
 'BLS',
 'BLUEDART',
 'BLUESTARCO',
 'BORORENEW',
 'BOSCHLTD',
 'BPCL',
 'BRIGADE',
 'BRITANNIA',
 'B

In [31]:
import pandas as pd
from pandas_market_calendars import get_calendar
from datetime import timedelta
import numpy as np

def cache_market_days(start_date: str, end_date: str):
    bse_calendar = get_calendar('XBOM')
    valid_days = bse_calendar.valid_days(start_date, end_date)
    return pd.to_datetime(valid_days).strftime('%Y-%m-%d').tolist()

# Map each date to the next market day using a precomputed list
def get_next_market_day_lookup(market_days):
    # Create a lookup dictionary for next market days
    market_days = pd.Series(market_days).sort_values()
    
    # Shift the series to get the next valid market day
    market_day_lookup = pd.DataFrame({
        'current_day': market_days,
        'next_market_day': market_days.shift(-1)
    }).dropna()
    
    return market_day_lookup.set_index('current_day').to_dict()['next_market_day']

def find_missing_dates(stock_df, market_day_lookup):
    stock_df['date'] = pd.to_datetime(stock_df['date'])
    stock_df['date_str'] = stock_df['date'].dt.strftime('%Y-%m-%d')

    stock_df['next_market_day'] = stock_df['date_str'].map(market_day_lookup)

    missing_dates = stock_df.loc[~stock_df['next_market_day'].isin(stock_df['date_str']), 'next_market_day']
    return missing_dates.dropna().tolist()

def get_missing_dates_for_all_stocks(df: pd.DataFrame):
    start_date = df['date'].min().date()
    end_date = df['date'].max().date()
    
    market_days = cache_market_days(str(start_date), str(end_date))
    
    market_day_lookup = get_next_market_day_lookup(market_days)
    
    missing_dates_dict = {}
    
    for stock_symbol, stock_df in df.groupby('nse_symbol'):
        print(f'Checking for {stock_symbol}')
        missing_dates = find_missing_dates(stock_df, market_day_lookup)
        print(f'Total dates missing for {stock_symbol}: {len(missing_dates)}')
        missing_dates_dict[stock_symbol] = missing_dates

    missing_dates_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in missing_dates_dict.items()]))
    
    return missing_dates_df


In [32]:
missing_dates_raw_cleansed = get_missing_dates_for_all_stocks(raw_cleansed)

Checking for 360ONE
Total dates missing for 360ONE: 26
Checking for 3MINDIA
Total dates missing for 3MINDIA: 82
Checking for AARTIIND
Total dates missing for AARTIIND: 82
Checking for AAVAS
Total dates missing for AAVAS: 29
Checking for ABB
Total dates missing for ABB: 82
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 82
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 34
Checking for ABFRL
Total dates missing for ABFRL: 56
Checking for ACC
Total dates missing for ACC: 82
Checking for ACE
Total dates missing for ACE: 82
Checking for ACI
Total dates missing for ACI: 12
Checking for ADANIENSOL
Total dates missing for ADANIENSOL: 45
Checking for ADANIENT
Total dates missing for ADANIENT: 82
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 30
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 82
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 80
Checking for AETHER
Total dates missing for AETHER: 13
Checking for AFFLE
Total dates missin

Total dates missing for FACT: 82
Checking for FDC
Total dates missing for FDC: 82
Checking for FEDERALBNK
Total dates missing for FEDERALBNK: 82
Checking for FINCABLES
Total dates missing for FINCABLES: 82
Checking for FINEORG
Total dates missing for FINEORG: 30
Checking for FINPIPE
Total dates missing for FINPIPE: 82
Checking for FIVESTAR
Total dates missing for FIVESTAR: 12
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 26
Checking for FORTIS
Total dates missing for FORTIS: 82
Checking for FSL
Total dates missing for FSL: 82
Checking for GAEL
Total dates missing for GAEL: 82
Checking for GAIL
Total dates missing for GAIL: 82
Checking for GESHIP
Total dates missing for GESHIP: 82
Checking for GICRE
Total dates missing for GICRE: 34
Checking for GILLETTE
Total dates missing for GILLETTE: 82
Checking for GLAND
Total dates missing for GLAND: 21
Checking for GLAXO
Total dates missing for GLAXO: 82
Checking for GLENMARK
Total dates missing for GLENMARK: 82
Checking for GLS
Tot

Total dates missing for MCX: 65
Checking for MEDANTA
Total dates missing for MEDANTA: 12
Checking for MEDPLUS
Total dates missing for MEDPLUS: 15
Checking for METROBRAND
Total dates missing for METROBRAND: 15
Checking for METROPOLIS
Total dates missing for METROPOLIS: 26
Checking for MFSL
Total dates missing for MFSL: 82
Checking for MGL
Total dates missing for MGL: 40
Checking for MHRIL
Total dates missing for MHRIL: 80
Checking for MINDACORP
Total dates missing for MINDACORP: 63
Checking for MMTC
Total dates missing for MMTC: 79
Checking for MOTHERSON
Total dates missing for MOTHERSON: 82
Checking for MOTILALOFS
Total dates missing for MOTILALOFS: 82
Checking for MPHASIS
Total dates missing for MPHASIS: 82
Checking for MRF
Total dates missing for MRF: 82
Checking for MRPL
Total dates missing for MRPL: 82
Checking for MSUMI
Total dates missing for MSUMI: 15
Checking for MTARTECH
Total dates missing for MTARTECH: 20
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 71
Checkin

Total dates missing for TRIDENT: 82
Checking for TRITURBINE
Total dates missing for TRITURBINE: 66
Checking for TRIVENI
Total dates missing for TRIVENI: 82
Checking for TTML
Total dates missing for TTML: 82
Checking for TV18BRDCST
Total dates missing for TV18BRDCST: 82
Checking for TVSMOTOR
Total dates missing for TVSMOTOR: 82
Checking for TVSSCS
Total dates missing for TVSSCS: 9
Checking for UBL
Total dates missing for UBL: 82
Checking for UCOBANK
Total dates missing for UCOBANK: 82
Checking for UJJIVANSFB
Total dates missing for UJJIVANSFB: 26
Checking for ULTRACEMCO
Total dates missing for ULTRACEMCO: 82
Checking for UNIONBANK
Total dates missing for UNIONBANK: 82
Checking for UNOMINDA
Total dates missing for UNOMINDA: 82
Checking for UPL
Total dates missing for UPL: 82
Checking for USHAMART
Total dates missing for USHAMART: 82
Checking for UTIAMC
Total dates missing for UTIAMC: 22
Checking for VAIBHAVGBL
Total dates missing for VAIBHAVGBL: 82
Checking for VARROC
Total dates missing

In [33]:
missing_dates_raw_cleansed.head()

360ONE     3MINDIA    AARTIIND       AAVAS         ABB  ABBOTINDIA  \
0  2020-02-19  2008-10-21  2008-10-21  2018-11-21  2008-10-21  2008-10-21   
1  2020-03-25  2008-10-28  2008-10-28  2019-02-19  2008-10-28  2008-10-28   
2  2020-04-01  2009-09-30  2009-09-30  2019-04-01  2009-09-30  2009-09-30   
3  2020-05-07  2010-03-16  2010-03-16  2020-02-19  2010-03-16  2010-03-16   
4  2020-10-30  2010-04-01  2010-04-01  2020-03-25  2010-04-01  2010-04-01   

    ABCAPITAL       ABFRL         ACC         ACE         ACI  ADANIENSOL  \
0  2017-12-01  2013-09-30  2008-10-21  2008-10-21  2023-03-22  2015-08-18   
1  2018-02-19  2013-11-15  2008-10-28  2008-10-28  2023-05-05  2015-12-24   
2  2018-04-02  2014-01-14  2009-09-30  2009-09-30  2023-08-16  2016-02-19   
3  2018-04-30  2014-02-19  2010-03-16  2010-03-16  2023-09-29  2016-04-01   
4  2018-08-17  2014-03-31  2010-04-01  2010-04-01  2024-01-22  2016-04-08   

     ADANIENT  ADANIGREEN  ADANIPORTS  ADANIPOWER      AETHER       AFFLE  \
0  2008-10-21  2018-08-17  2008-10-21  2009-09-30  2022-08-16  2020-02-19   
1  2008-10-28  2018-11-21  2008-10-28  2010-03-16  2023-03-22  2020-03-25   
2  2009-09-30  2019-02-19  2009-09-30  2010-04-01  2023-05-05  2020-04-01   
3  2010-03-16  2019-04-01  2010-03-16  2010-05-27  2023-08-16  2020-05-07   
4  2010-04-01  2020-02-19  2010-04-01  2010-07-01  2023-09-29  2020-10-30   

       AIAENG  AJANTPHARM       ALKEM  ALKYLAMINE    ALLCARGO    ALOKINDS  \
0  2008-10-21  2008-10-21  2015-12-24  2008-10-21  2008-10-21  2008-10-21   
1  2008-10-28  2008-10-28  2016-02-19  2008-10-28  2008-10-28  2008-10-28   
2  2009-09-30  2009-09-30  2016-04-01  2009-09-30  2009-09-30  2009-09-30   
3  2010-03-16  2010-03-16  2016-04-08  2010-03-16  2010-03-16  2010-03-16   
4  2010-04-01  2010-04-01  2016-08-17  2010-04-01  2010-04-01  2010-04-01   

        AMBER   AMBUJACEM  ANANDRATHI    ANGELONE      ANURAS    APARINDS  \
0  2018-02-19  2008-10-21  2022-04-01  2020-10-30  2021-04-01  2008-10-21   
1  2018-04-02  2008-10-28  2022-05-16  2021-02-19  2021-04-13  2008-10-28   
2  2018-04-30  2009-09-30  2022-08-16  2021-04-01  2021-05-26  2009-09-30   
3  2018-08-17  2010-03-16  2023-03-22  2021-04-13  2021-08-16  2010-03-16   
4  2018-11-21  2010-04-01  2023-05-05  2021-05-26  2021-10-19  2010-04-01   

    APLAPOLLO      APLLTD  APOLLOHOSP  APOLLOTYRE       APTUS       ARE&M  \
0  2012-02-16  2011-09-30  2008-10-21  2008-10-21  2021-10-19  2008-10-21   
1  2012-03-23  2011-10-26  2008-10-28  2008-10-28  2022-04-01  2008-10-28   
2  2012-04-02  2012-02-16  2009-09-30  2009-09-30  2022-05-16  2009-09-30   
3  2012-07-02  2012-03-23  2010-03-16  2010-03-16  2022-08-16  2010-03-16   
4  2012-11-13  2012-04-02  2010-04-01  2010-04-01  2023-03-22  2010-04-01   

   ASAHIINDIA    ASHOKLEY  ASIANPAINT     ASTERDM      ASTRAL    ASTRAZEN  \
0  2008-10-21  2008-10-21  2008-10-21  2018-04-02  2008-10-21  2008-10-21   
1  2008-10-28  2008-10-28  2008-10-28  2018-04-30  2008-10-28  2008-10-28   
2  2009-09-30  2009-09-30  2009-09-30  2018-08-17  2009-09-30  2009-09-30   
3  2010-03-16  2010-03-16  2010-03-16  2018-11-21  2010-03-16  2010-03-16   
4  2010-04-01  2010-04-01  2010-04-01  2019-02-19  2010-04-01  2010-04-01   

         ATGL        ATUL      AUBANK  AUROPHARMA  AVANTIFEED         AWL  \
0  2018-11-21  2008-10-21  2017-08-17  2008-10-21  2010-03-16  2022-04-01   
1  2019-02-19  2008-10-28  2017-12-01  2008-10-28  2010-04-01  2022-05-16   
2  2019-04-01  2009-09-30  2018-02-19  2009-09-30  2010-05-27  2022-08-16   
3  2020-02-19  2010-03-16  2018-04-02  2010-03-16  2010-07-01  2023-03-22   
4  2020-03-25  2010-04-01  2018-04-30  2010-04-01  2010-09-30  2023-05-05   

     AXISBANK  BAJAJ-AUTO  BAJAJFINSV  BAJAJHLDNG  BAJFINANCE   BALAMINES  \
0  2008-10-21  2008-10-21  2008-10-21  2008-10-21  2008-10-21  2008-10-21   
1  2008-10-28  2008-10-28  2008-10-28  2008-10-28  2008-10-28  2008-10-28   
2  2009-09-30  2009-09-30  2009-09-30  2009-09-30  2009-09-30  2009-09-3

In [34]:
import yfinance as yf
import pandas as pd
from pandas_market_calendars import get_calendar

def fetch_data_for_dates(symbol, dates):
    
    symbol = symbol + '.NS' 
    
    min_dt = pd.to_datetime(min(dates))
    max_dt = pd.to_datetime(max(dates)) + timedelta(days=1)
    
    stock_data = yf.download(symbol, start= min_dt, end= max_dt + timedelta(days=1))
    
    stock_data = stock_data[stock_data.index.isin(pd.to_datetime(dates))]
    
    return stock_data

def fill_missing_data(df, missing_dates_df):
    
    for symbol in missing_dates_df.columns:
        missing_dates = missing_dates_df[symbol].dropna().tolist()
        
        if not missing_dates:
            continue  
        
        print(f"Fetching missing data for {symbol}...")
        
        missing_data = fetch_data_for_dates(symbol, missing_dates)
        
        if not missing_data.empty:
            missing_data.reset_index(inplace=True)
            missing_data['nse_symbol'] = symbol
            
            missing_data.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close',
                                          'Adj Close':'adj_close','Volume':'volume'},inplace=True)
            
            df = pd.concat([df, missing_data], ignore_index=True)
    
    df = df.sort_values(by=['nse_symbol', 'date'])
    
    return df

def fill_missing_ohlcv_data(raw_cleansed):
    missing_dates_raw_cleansed = get_missing_dates_for_all_stocks(raw_cleansed)
    
    updated_df = fill_missing_data(raw_cleansed, missing_dates_raw_cleansed)
    
    return updated_df


In [35]:
updated_df = fill_missing_ohlcv_data(raw_cleansed)

Checking for 360ONE
Total dates missing for 360ONE: 26
Checking for 3MINDIA
Total dates missing for 3MINDIA: 82
Checking for AARTIIND
Total dates missing for AARTIIND: 82
Checking for AAVAS
Total dates missing for AAVAS: 29
Checking for ABB
Total dates missing for ABB: 82
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 82
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 34
Checking for ABFRL
Total dates missing for ABFRL: 56
Checking for ACC
Total dates missing for ACC: 82
Checking for ACE
Total dates missing for ACE: 82
Checking for ACI
Total dates missing for ACI: 12
Checking for ADANIENSOL
Total dates missing for ADANIENSOL: 45
Checking for ADANIENT
Total dates missing for ADANIENT: 82
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 30
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 82
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 80
Checking for AETHER
Total dates missing for AETHER: 13
Checking for AFFLE
Total dates missin

Total dates missing for FINPIPE: 82
Checking for FIVESTAR
Total dates missing for FIVESTAR: 12
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 26
Checking for FORTIS
Total dates missing for FORTIS: 82
Checking for FSL
Total dates missing for FSL: 82
Checking for GAEL
Total dates missing for GAEL: 82
Checking for GAIL
Total dates missing for GAIL: 82
Checking for GESHIP
Total dates missing for GESHIP: 82
Checking for GICRE
Total dates missing for GICRE: 34
Checking for GILLETTE
Total dates missing for GILLETTE: 82
Checking for GLAND
Total dates missing for GLAND: 21
Checking for GLAXO
Total dates missing for GLAXO: 82
Checking for GLENMARK
Total dates missing for GLENMARK: 82
Checking for GLS
Total dates missing for GLS: 17
Checking for GMDCLTD
Total dates missing for GMDCLTD: 82
Checking for GMMPFAUDLR
Total dates missing for GMMPFAUDLR: 32
Checking for GMRINFRA
Total dates missing for GMRINFRA: 82
Checking for GNFC
Total dates missing for GNFC: 82
Checking for GODFRYPHLP
T

Total dates missing for MPHASIS: 82
Checking for MRF
Total dates missing for MRF: 82
Checking for MRPL
Total dates missing for MRPL: 82
Checking for MSUMI
Total dates missing for MSUMI: 15
Checking for MTARTECH
Total dates missing for MTARTECH: 20
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 71
Checking for NAM-INDIA
Total dates missing for NAM-INDIA: 34
Checking for NATCOPHARM
Total dates missing for NATCOPHARM: 82
Checking for NATIONALUM
Total dates missing for NATIONALUM: 82
Checking for NAUKRI
Total dates missing for NAUKRI: 82
Checking for NAVINFLUOR
Total dates missing for NAVINFLUOR: 82
Checking for NBCC
Total dates missing for NBCC: 63
Checking for NCC
Total dates missing for NCC: 82
Checking for NESTLEIND
Total dates missing for NESTLEIND: 82
Checking for NETWORK18
Total dates missing for NETWORK18: 82
Checking for NH
Total dates missing for NH: 43
Checking for NHPC
Total dates missing for NHPC: 80
Checking for NIACL
Total dates missing for NIACL: 34
Checking fo

Total dates missing for ULTRACEMCO: 82
Checking for UNIONBANK
Total dates missing for UNIONBANK: 82
Checking for UNOMINDA
Total dates missing for UNOMINDA: 82
Checking for UPL
Total dates missing for UPL: 82
Checking for USHAMART
Total dates missing for USHAMART: 82
Checking for UTIAMC
Total dates missing for UTIAMC: 22
Checking for VAIBHAVGBL
Total dates missing for VAIBHAVGBL: 82
Checking for VARROC
Total dates missing for VARROC: 30
Checking for VBL
Total dates missing for VBL: 39
Checking for VEDL
Total dates missing for VEDL: 82
Checking for VGUARD
Total dates missing for VGUARD: 82
Checking for VIJAYA
Total dates missing for VIJAYA: 16
Checking for VIPIND
Total dates missing for VIPIND: 82
Checking for VOLTAS
Total dates missing for VOLTAS: 82
Checking for VTL
Total dates missing for VTL: 82
Checking for WELCORP
Total dates missing for WELCORP: 82
Checking for WELSPUNLIV
Total dates missing for WELSPUNLIV: 82
Checking for WESTLIFE
Total dates missing for WESTLIFE: 26
Checking for

[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


Fetching missing data for ZFCVINDIA...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching missing data for ZOMATO...
Fetching missing data for ZYDUSLIFE...


[*********************100%%**********************]  1 of 1 completed


In [36]:
updated_df.head()

date    open    high     low   close     volume nse_symbol  adj_close
0 2019-09-19  302.50  317.63  302.50  317.63  7109860.0     360ONE     317.63
1 2019-09-20  332.25  333.50  332.25  333.50  2760648.0     360ONE     333.50
2 2019-09-23  350.18  350.18  350.18  350.18    34096.0     360ONE     350.18
3 2019-09-24  367.68  367.68  367.68  367.68  2496652.0     360ONE     367.68
4 2019-09-25  386.05  386.05  349.30  351.24   530580.0     360ONE     351.24

In [37]:
updated_df.shape

(2014727, 8)

In [38]:
missing_dates_updated_df = get_missing_dates_for_all_stocks(updated_df)

Checking for 360ONE
Total dates missing for 360ONE: 2
Checking for 3MINDIA
Total dates missing for 3MINDIA: 9
Checking for AARTIIND
Total dates missing for AARTIIND: 9
Checking for AAVAS
Total dates missing for AAVAS: 2
Checking for ABB
Total dates missing for ABB: 9
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 9
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 2
Checking for ABFRL
Total dates missing for ABFRL: 4
Checking for ACC
Total dates missing for ACC: 9
Checking for ACE
Total dates missing for ACE: 9
Checking for ACI
Total dates missing for ACI: 2
Checking for ADANIENSOL
Total dates missing for ADANIENSOL: 37
Checking for ADANIENT
Total dates missing for ADANIENT: 9
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 2
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 9
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 8
Checking for AETHER
Total dates missing for AETHER: 2
Checking for AFFLE
Total dates missing for AFFLE: 2
C

Total dates missing for EXIDEIND: 9
Checking for FACT
Total dates missing for FACT: 9
Checking for FDC
Total dates missing for FDC: 9
Checking for FEDERALBNK
Total dates missing for FEDERALBNK: 9
Checking for FINCABLES
Total dates missing for FINCABLES: 9
Checking for FINEORG
Total dates missing for FINEORG: 2
Checking for FINPIPE
Total dates missing for FINPIPE: 9
Checking for FIVESTAR
Total dates missing for FIVESTAR: 2
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 2
Checking for FORTIS
Total dates missing for FORTIS: 9
Checking for FSL
Total dates missing for FSL: 9
Checking for GAEL
Total dates missing for GAEL: 9
Checking for GAIL
Total dates missing for GAIL: 9
Checking for GESHIP
Total dates missing for GESHIP: 9
Checking for GICRE
Total dates missing for GICRE: 2
Checking for GILLETTE
Total dates missing for GILLETTE: 9
Checking for GLAND
Total dates missing for GLAND: 2
Checking for GLAXO
Total dates missing for GLAXO: 9
Checking for GLENMARK
Total dates missing 

Total dates missing for MPHASIS: 9
Checking for MRF
Total dates missing for MRF: 9
Checking for MRPL
Total dates missing for MRPL: 9
Checking for MSUMI
Total dates missing for MSUMI: 2
Checking for MTARTECH
Total dates missing for MTARTECH: 2
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 6
Checking for NAM-INDIA
Total dates missing for NAM-INDIA: 2
Checking for NATCOPHARM
Total dates missing for NATCOPHARM: 9
Checking for NATIONALUM
Total dates missing for NATIONALUM: 9
Checking for NAUKRI
Total dates missing for NAUKRI: 9
Checking for NAVINFLUOR
Total dates missing for NAVINFLUOR: 9
Checking for NBCC
Total dates missing for NBCC: 5
Checking for NCC
Total dates missing for NCC: 9
Checking for NESTLEIND
Total dates missing for NESTLEIND: 9
Checking for NETWORK18
Total dates missing for NETWORK18: 9
Checking for NH
Total dates missing for NH: 2
Checking for NHPC
Total dates missing for NHPC: 8
Checking for NIACL
Total dates missing for NIACL: 2
Checking for NLCINDIA
Total d

Total dates missing for UCOBANK: 9
Checking for UJJIVANSFB
Total dates missing for UJJIVANSFB: 2
Checking for ULTRACEMCO
Total dates missing for ULTRACEMCO: 9
Checking for UNIONBANK
Total dates missing for UNIONBANK: 9
Checking for UNOMINDA
Total dates missing for UNOMINDA: 9
Checking for UPL
Total dates missing for UPL: 9
Checking for USHAMART
Total dates missing for USHAMART: 9
Checking for UTIAMC
Total dates missing for UTIAMC: 2
Checking for VAIBHAVGBL
Total dates missing for VAIBHAVGBL: 9
Checking for VARROC
Total dates missing for VARROC: 2
Checking for VBL
Total dates missing for VBL: 2
Checking for VEDL
Total dates missing for VEDL: 9
Checking for VGUARD
Total dates missing for VGUARD: 9
Checking for VIJAYA
Total dates missing for VIJAYA: 2
Checking for VIPIND
Total dates missing for VIPIND: 9
Checking for VOLTAS
Total dates missing for VOLTAS: 9
Checking for VTL
Total dates missing for VTL: 9
Checking for WELCORP
Total dates missing for WELCORP: 9
Checking for WELSPUNLIV
Total

In [39]:
missing_dates_updated_df.head()

360ONE     3MINDIA    AARTIIND       AAVAS         ABB  ABBOTINDIA  \
0  2024-01-22  2008-10-28  2008-10-28  2024-01-22  2008-10-28  2008-10-28   
1  2024-05-20  2010-11-05  2010-11-05  2024-05-20  2010-11-05  2010-11-05   
2         NaN  2011-04-04  2011-04-04         NaN  2011-04-04  2011-04-04   
3         NaN  2011-10-26  2011-10-26         NaN  2011-10-26  2011-10-26   
4         NaN  2012-11-13  2012-11-13         NaN  2012-11-13  2012-11-13   

    ABCAPITAL       ABFRL         ACC         ACE         ACI  ADANIENSOL  \
0  2024-01-22  2013-11-15  2008-10-28  2008-10-28  2024-01-22  2015-08-18   
1  2024-05-20  2014-04-01  2010-11-05  2010-11-05  2024-05-20  2015-12-24   
2         NaN  2024-01-22  2011-04-04  2011-04-04         NaN  2016-02-19   
3         NaN  2024-05-20  2011-10-26  2011-10-26         NaN  2016-04-01   
4         NaN         NaN  2012-11-13  2012-11-13         NaN  2016-04-08   

     ADANIENT  ADANIGREEN  ADANIPORTS  ADANIPOWER      AETHER       AFFLE  \
0  2008-10-28  2024-01-22  2008-10-28  2010-11-05  2024-01-22  2024-01-22   
1  2010-11-05  2024-05-20  2010-11-05  2011-04-04  2024-05-20  2024-05-20   
2  2011-04-04         NaN  2011-04-04  2011-10-26         NaN         NaN   
3  2011-10-26         NaN  2011-10-26  2012-11-13         NaN         NaN   
4  2012-11-13         NaN  2012-11-13  2013-11-15         NaN         NaN   

       AIAENG  AJANTPHARM       ALKEM  ALKYLAMINE    ALLCARGO    ALOKINDS  \
0  2008-10-28  2008-10-28  2024-01-22  2008-10-28  2008-10-28  2008-10-28   
1  2010-11-05  2010-11-05  2024-05-20  2010-11-05  2010-11-05  2010-11-05   
2  2011-04-04  2011-04-04         NaN  2011-04-04  2011-04-04  2011-04-04   
3  2011-10-26  2011-10-26         NaN  2011-10-26  2011-10-26  2011-10-26   
4  2012-11-13  2012-11-13         NaN  2012-11-13  2012-11-13  2012-11-13   

        AMBER   AMBUJACEM  ANANDRATHI    ANGELONE      ANURAS    APARINDS  \
0  2024-01-22  2008-10-28  2024-01-22  2024-01-22  2024-01-22  2008-10-28   
1  2024-05-20  2010-11-05  2024-05-20  2024-05-20  2024-05-20  2010-11-05   
2         NaN  2011-04-04         NaN         NaN         NaN  2011-04-04   
3         NaN  2011-10-26         NaN         NaN         NaN  2011-10-26   
4         NaN  2012-11-13         NaN         NaN         NaN  2012-11-13   

    APLAPOLLO      APLLTD  APOLLOHOSP  APOLLOTYRE       APTUS       ARE&M  \
0  2012-11-13  2011-10-26  2008-10-28  2008-10-28  2024-01-22  2008-10-21   
1  2013-11-15  2012-11-13  2010-11-05  2010-11-05  2024-05-20  2008-10-28   
2  2014-04-01  2013-11-15  2011-04-04  2011-04-04         NaN  2009-09-30   
3  2024-01-22  2014-04-01  2011-10-26  2011-10-26         NaN  2010-03-16   
4  2024-05-20  2024-01-22  2012-11-13  2012-11-13         NaN  2010-04-01   

   ASAHIINDIA    ASHOKLEY  ASIANPAINT     ASTERDM      ASTRAL    ASTRAZEN  \
0  2008-10-28  2008-10-28  2008-10-28  2024-01-22  2008-10-28  2008-10-28   
1  2010-11-05  2010-11-05  2010-11-05  2024-05-20  2010-11-05  2010-11-05   
2  2011-04-04  2011-04-04  2011-04-04         NaN  2011-04-04  2011-04-04   
3  2011-10-26  2011-10-26  2011-10-26         NaN  2011-10-26  2011-10-26   
4  2012-11-13  2012-11-13  2012-11-13         NaN  2012-11-13  2012-11-13   

         ATGL        ATUL      AUBANK  AUROPHARMA  AVANTIFEED         AWL  \
0  2024-01-22  2008-10-28  2024-01-22  2008-10-28  2010-11-05  2024-01-22   
1  2024-05-20  2010-11-05  2024-05-20  2010-11-05  2011-04-04  2024-05-20   
2         NaN  2011-04-04         NaN  2011-04-04  2011-10-26         NaN   
3         NaN  2011-10-26         NaN  2011-10-26  2012-11-13         NaN   
4         NaN  2012-11-13         NaN  2012-11-13  2013-11-15         NaN   

     AXISBANK  BAJAJ-AUTO  BAJAJFINSV  BAJAJHLDNG  BAJFINANCE   BALAMINES  \
0  2008-10-28  2008-10-28  2008-10-28  2008-10-28  2008-10-28  2008-10-28   
1  2010-11-05  2010-11-05  2010-11-05  2010-11-05  2010-11-05  2010-11-05   
2  2011-04-04  2011-04-04  2011-04-04  2011-04-04  2011-04-04  2011-04-0

In [40]:
import yfinance as yf
import pandas as pd

# Function to get OHLCV data and min/max dates for each stock
def get_ohlcv_and_dates(stock_list):
    date_info = []
    ohlcv_data = pd.DataFrame()

    for stock in stock_list:
        stock = stock + '.NS'
        df = yf.download(stock)
        symbol_cleaned = stock.replace('.NS', '') 

        if not df.empty:
            df['Stock'] = symbol_cleaned
            
            ohlcv_data = pd.concat([ohlcv_data, df])

            min_date = df.index.min()
            max_date = df.index.max()

            date_info.append({
                'Stock': symbol_cleaned,
                'Min Date': min_date,
                'Max Date': max_date
            })
        else:
            date_info.append({
                'Stock': symbol_cleaned,
                'Min Date': None,
                'Max Date': None
            })

    date_info_df = pd.DataFrame(date_info)
    
    return ohlcv_data, date_info_df


stocks = raw_cleansed['nse_symbol'].unique().tolist()

ohlcv_data_df, min_max_dates_df = get_ohlcv_and_dates(stocks)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [41]:
min_max_dates_df

Stock   Min Date   Max Date
0        360ONE 2019-09-19 2024-11-04
1       3MINDIA 2002-07-01 2024-11-04
2      AARTIIND 2002-07-01 2024-11-04
3         AAVAS 2018-10-08 2024-11-04
4           ABB 2002-07-01 2024-11-04
5    ABBOTINDIA 2002-08-12 2024-11-04
6     ABCAPITAL 2017-09-01 2024-11-04
7         ABFRL 2013-07-17 2024-11-04
8           ACC 2002-07-01 2024-11-04
9           ACE 2006-09-26 2024-11-04
10          ACI 2022-11-21 2024-11-04
11   ADANIENSOL 2023-08-16 2024-11-04
12     ADANIENT 2002-07-01 2024-11-04
13   ADANIGREEN 2018-06-18 2024-11-04
14   ADANIPORTS 2007-11-27 2024-11-04
15   ADANIPOWER 2009-08-20 2024-11-04
16       AETHER 2022-06-03 2024-11-04
17        AFFLE 2019-08-08 2024-11-04
18       AIAENG 2005-12-14 2024-11-04
19   AJANTPHARM 2002-07-01 2024-11-04
20        ALKEM 2015-12-23 2024-11-04
21   ALKYLAMINE 2007-11-26 2024-11-04
22     ALLCARGO 2006-06-23 2024-11-04
23     ALOKINDS 1996-01-02 2024-11-04
24        AMBER 2018-01-30 2024-11-04
25    AMBUJACEM 2002-07-01 2024-11-04
26   ANANDRATHI 2021-12-14 2024-11-04
27     ANGELONE 2020-10-05 2024-11-04
28       ANURAS 2021-03-24 2024-11-04
29     APARINDS 2004-07-12 2024-11-04
30    APLAPOLLO 2011-12-14 2024-11-04
31       APLLTD 2011-09-20 2024-11-04
32   APOLLOHOSP 2002-07-01 2024-11-04
33   APOLLOTYRE 2002-01-01 2024-11-04
34        APTUS 2021-08-24 2024-11-04
35        ARE&M 2023-10-18 2024-11-04
36   ASAHIINDIA 2002-07-01 2024-11-04
37     ASHOKLEY 2002-07-01 2024-11-04
38   ASIANPAINT 2002-07-01 2024-11-04
39      ASTERDM 2018-02-26 2024-11-04
40       ASTRAL 2007-03-20 2024-11-04
41     ASTRAZEN 2002-07-01 2024-11-04
42         ATGL 2018-11-05 2024-11-04
43         ATUL 2002-07-01 2024-11-04
44       AUBANK 2017-07-11 2024-11-04
45   AUROPHARMA 1996-01-15 2024-11-04
46   AVANTIFEED 2010-01-08 2024-11-04
47          AWL 2022-02-08 2024-11-04
48     AXISBANK 1998-11-27 2024-11-04
49   BAJAJ-AUTO 2002-07-01 2024-11-04
50   BAJAJFINSV 2002-08-12 2024-11-04
51   BAJAJHLDNG 2002-08-12 2024-11-04
52   BAJFINANCE 2002-07-01 2024-11-04
53    BALAMINES 2007-04-05 2024-11-04
54   BALKRISIND 2006-01-27 2024-11-04
55   BALRAMCHIN 2002-07-01 2024-11-04
56   BANDHANBNK 2018-03-27 2024-11-04
57   BANKBARODA 2002-07-01 2024-11-04
58    BANKINDIA 2002-07-01 2024-11-04
59    BATAINDIA 2002-07-01 2024-11-04
60    BAYERCROP 2010-01-08 2024-11-04
61         BBTC 2002-07-01 2024-11-04
62          BDL 2018-03-23 2024-11-04
63          BEL 2002-07-01 2024-11-04
64         BEML 2002-07-01 2024-11-04
65   BERGEPAINT 2002-07-01 2024-11-04
66   BHARATFORG 1996-01-01 2024-11-04
67   BHARTIARTL 2002-07-01 2024-11-04
68         BHEL 1996-01-01 2024-11-04
69       BIKAJI 2022-11-16 2024-11-04
70       BIOCON 2004-04-07 2024-11-04
71   BIRLACORPN 2002-07-01 2024-11-04
72          BLS 2016-06-15 2024-11-04
73     BLUEDART 2002-07-01 2024-11-04
74   BLUESTARCO 2002-07-01 2024-11-04
75    BORORENEW 2018-05-25 2024-11-04
76     BOSCHLTD 2002-07-01 2024-11-04
77         BPCL 1996-01-02 2024-11-04
78      BRIGADE 2007-12-31 2024-11-04
79    BRITANNIA 1996-01-02 2024-11-04
80          BSE 2017-02-03 2024-11-04
81        BSOFT 2002-07-01 2024-11-04
82       CAMPUS 2022-05-09 2024-11-04
83         CAMS 2020-10-05 2024-11-04
84        CANBK 2002-12-23 2024-11-04
85   CANFINHOME 2002-07-01 2024-11-04
86   CAPLIPOINT 2014-06-23 2024-11-04
87   CARBORUNIV 2002-07-01 2024-11-04
88   CASTROLIND 2002-08-12 2024-11-04
89          CCL 2004-06-07 2024-11-04
90         CDSL 2017-06-30 2024-11-04
91      CEATLTD 2008-02-01 2024-11-04
92        CELLO 2023-11-06 2024-11-04
93    CENTRALBK 2007-08-21 2024-11-04
94   CENTURYPLY 2006-03-23 2024-11-04
95         CERA 2007-11-02 2024-11-04
96         CESC 2002-07-01 2024-11-04
97         CGCL 2010-10-29 2024-11-04
98      CGPOWER 2002-07-01 2024-11-04
99       CHALET 2019-02-07 2024-11-04
100  CHAMBLFERT 2002-07-01 2024-11-04
101  CHEMPLASTS 2021-08-24 2024-11-04
102  CHENNPETRO 2002-07-01 2024-11-04
103    CHOLAFIN 2002-07-01 2024-11-04
104  CHOLAHLDNG 2002

In [42]:
for col in missing_dates_updated_df.columns:
    missing_dates_updated_df[col] = pd.to_datetime(missing_dates_updated_df[col])

min_max_new_df = pd.DataFrame({
    'Stock': missing_dates_updated_df.columns,
    'Min Date': missing_dates_updated_df.min().values,
    'Max Date': missing_dates_updated_df.max().values
})
    

# Merge the two DataFrames on the 'Stock' column to compare the dates
merged_df = pd.merge(min_max_new_df, min_max_dates_df, on='Stock', suffixes=('_New', '_Yahoo'))

# Compare if the new min/max dates fall within the Yahoo Finance min/max date ranges
merged_df['Min Date In Range'] = merged_df['Min Date_New'] >= merged_df['Min Date_Yahoo']
merged_df['Max Date In Range'] = merged_df['Max Date_New'] <= merged_df['Max Date_Yahoo']

In [43]:
merged_df

Stock Min Date_New Max Date_New Min Date_Yahoo Max Date_Yahoo  \
0        360ONE   2024-01-22   2024-05-20     2019-09-19     2024-11-04   
1       3MINDIA   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
2      AARTIIND   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
3         AAVAS   2024-01-22   2024-05-20     2018-10-08     2024-11-04   
4           ABB   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
5    ABBOTINDIA   2008-10-28   2024-05-20     2002-08-12     2024-11-04   
6     ABCAPITAL   2024-01-22   2024-05-20     2017-09-01     2024-11-04   
7         ABFRL   2013-11-15   2024-05-20     2013-07-17     2024-11-04   
8           ACC   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
9           ACE   2008-10-28   2024-05-20     2006-09-26     2024-11-04   
10          ACI   2024-01-22   2024-05-20     2022-11-21     2024-11-04   
11   ADANIENSOL   2015-08-18   2024-05-20     2023-08-16     2024-11-04   
12     ADANIENT   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
13   ADANIGREEN   2024-01-22   2024-05-20     2018-06-18     2024-11-04   
14   ADANIPORTS   2008-10-28   2024-05-20     2007-11-27     2024-11-04   
15   ADANIPOWER   2010-11-05   2024-05-20     2009-08-20     2024-11-04   
16       AETHER   2024-01-22   2024-05-20     2022-06-03     2024-11-04   
17        AFFLE   2024-01-22   2024-05-20     2019-08-08     2024-11-04   
18       AIAENG   2008-10-28   2024-05-20     2005-12-14     2024-11-04   
19   AJANTPHARM   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
20        ALKEM   2024-01-22   2024-05-20     2015-12-23     2024-11-04   
21   ALKYLAMINE   2008-10-28   2024-05-20     2007-11-26     2024-11-04   
22     ALLCARGO   2008-10-28   2024-05-20     2006-06-23     2024-11-04   
23     ALOKINDS   2008-10-28   2024-05-20     1996-01-02     2024-11-04   
24        AMBER   2024-01-22   2024-05-20     2018-01-30     2024-11-04   
25    AMBUJACEM   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
26   ANANDRATHI   2024-01-22   2024-05-20     2021-12-14     2024-11-04   
27     ANGELONE   2024-01-22   2024-05-20     2020-10-05     2024-11-04   
28       ANURAS   2024-01-22   2024-05-20     2021-03-24     2024-11-04   
29     APARINDS   2008-10-28   2024-05-20     2004-07-12     2024-11-04   
30    APLAPOLLO   2012-11-13   2024-05-20     2011-12-14     2024-11-04   
31       APLLTD   2011-10-26   2024-05-20     2011-09-20     2024-11-04   
32   APOLLOHOSP   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
33   APOLLOTYRE   2008-10-28   2024-05-20     2002-01-01     2024-11-04   
34        APTUS   2024-01-22   2024-05-20     2021-08-24     2024-11-04   
35        ARE&M   2008-10-21   2024-05-20     2023-10-18     2024-11-04   
36   ASAHIINDIA   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
37     ASHOKLEY   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
38   ASIANPAINT   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
39      ASTERDM   2024-01-22   2024-05-20     2018-02-26     2024-11-04   
40       ASTRAL   2008-10-28   2024-05-20     2007-03-20     2024-11-04   
41     ASTRAZEN   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
42         ATGL   2024-01-22   2024-05-20     2018-11-05     2024-11-04   
43         ATUL   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
44       AUBANK   2024-01-22   2024-05-20     2017-07-11     2024-11-04   
45   AUROPHARMA   2008-10-28   2024-05-20     1996-01-15     2024-11-04   
46   AVANTIFEED   2010-11-05   2024-05-20     2010-01-08     2024-11-04   
47          AWL   2024-01-22   2024-05-20     2022-02-08     2024-11-04   
48     AXISBANK   2008-10-28   2024-05-20     1998-11-27     2024-11-04   
49   BAJAJ-AUTO   2008-10-28   2024-05-20     2002-07-01     2024-11-04   
50   BAJAJFINSV   2008-10-28   2024-05-20     2002-08-12     2024-11-04   
51   BAJAJHLDNG   2008-10-28   2024-05-20     2002-08-12     2024-11-04   
52   BAJFINANCE   2008-10-28   2024

In [44]:
merged_df[merged_df['Min Date In Range']==False]

Stock Min Date_New Max Date_New Min Date_Yahoo Max Date_Yahoo  \
11   ADANIENSOL   2015-08-18   2024-05-20     2023-08-16     2024-11-04   
35        ARE&M   2008-10-21   2024-05-20     2023-10-18     2024-11-04   
60    BAYERCROP   2008-10-21   2024-05-20     2010-01-08     2024-11-04   
128   DEEPAKNTR   2008-10-21   2024-05-20     2010-09-29     2024-11-04   
216        IDFC   2008-10-21   2024-09-18     2024-10-09     2024-10-09   
274        KPIL   2008-10-21   2024-05-20     2023-05-30     2024-11-04   
286    LLOYDSME   2008-10-21   2024-05-20     2023-07-17     2024-11-04   
374     PVRINOX   2008-10-21   2024-05-20     2022-11-10     2024-11-04   
463  TV18BRDCST   2008-10-21   2024-09-18     2024-10-15     2024-10-15   

     Min Date In Range  Max Date In Range  
11               False               True  
35               False               True  
60               False               True  
128              False               True  
216              False               True  
274              False               True  
286              False               True  
374              False               True  
463              False               True

In [45]:
final = updated_df.copy()

In [46]:
final.shape

(2014727, 8)

In [47]:
stocks_remove = ['ADANIENSOL','ARE&M','KPIL','LLOYDSME','PVRINOX','TATAMTRDVR']

In [48]:
final = final[~final['nse_symbol'].isin(stocks_remove)]

In [49]:
final_ohlc = []

grouped = final.groupby('nse_symbol')

for symbol, group in grouped:
    print(f'Cleaning for {symbol}')
    if symbol == 'BAYERCROP':
        print(f'filtering for {symbol}')
        group = group[group['date'] >= '2010-01-08']
    if symbol == 'DEEPAKNTR':
        print(f'filtering for {symbol}')
        group = group[group['date'] >= '2010-09-29']

    final_ohlc.append(group)

cleaned_final = pd.concat(final_ohlc).reset_index(drop=True)

Cleaning for 360ONE
Cleaning for 3MINDIA
Cleaning for AARTIIND
Cleaning for AAVAS
Cleaning for ABB
Cleaning for ABBOTINDIA
Cleaning for ABCAPITAL
Cleaning for ABFRL
Cleaning for ACC
Cleaning for ACE
Cleaning for ACI
Cleaning for ADANIENT
Cleaning for ADANIGREEN
Cleaning for ADANIPORTS
Cleaning for ADANIPOWER
Cleaning for AETHER
Cleaning for AFFLE
Cleaning for AIAENG
Cleaning for AJANTPHARM
Cleaning for ALKEM
Cleaning for ALKYLAMINE
Cleaning for ALLCARGO
Cleaning for ALOKINDS
Cleaning for AMBER
Cleaning for AMBUJACEM
Cleaning for ANANDRATHI
Cleaning for ANGELONE
Cleaning for ANURAS
Cleaning for APARINDS
Cleaning for APLAPOLLO
Cleaning for APLLTD
Cleaning for APOLLOHOSP
Cleaning for APOLLOTYRE
Cleaning for APTUS
Cleaning for ASAHIINDIA
Cleaning for ASHOKLEY
Cleaning for ASIANPAINT
Cleaning for ASTERDM
Cleaning for ASTRAL
Cleaning for ASTRAZEN
Cleaning for ATGL
Cleaning for ATUL
Cleaning for AUBANK
Cleaning for AUROPHARMA
Cleaning for AVANTIFEED
Cleaning for AWL
Cleaning for AXISBANK
Clea

In [50]:
cleaned_final.shape

(1980613, 8)

In [51]:
missing_dates_cleaned_final = get_missing_dates_for_all_stocks(cleaned_final)

Checking for 360ONE
Total dates missing for 360ONE: 2
Checking for 3MINDIA
Total dates missing for 3MINDIA: 9
Checking for AARTIIND
Total dates missing for AARTIIND: 9
Checking for AAVAS
Total dates missing for AAVAS: 2
Checking for ABB
Total dates missing for ABB: 9
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 9
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 2
Checking for ABFRL
Total dates missing for ABFRL: 4
Checking for ACC
Total dates missing for ACC: 9
Checking for ACE
Total dates missing for ACE: 9
Checking for ACI
Total dates missing for ACI: 2
Checking for ADANIENT
Total dates missing for ADANIENT: 9
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 2
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 9
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 8
Checking for AETHER
Total dates missing for AETHER: 2
Checking for AFFLE
Total dates missing for AFFLE: 2
Checking for AIAENG
Total dates missing for AIAENG: 9
Checking f

Total dates missing for FINPIPE: 9
Checking for FIVESTAR
Total dates missing for FIVESTAR: 2
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 2
Checking for FORTIS
Total dates missing for FORTIS: 9
Checking for FSL
Total dates missing for FSL: 9
Checking for GAEL
Total dates missing for GAEL: 9
Checking for GAIL
Total dates missing for GAIL: 9
Checking for GESHIP
Total dates missing for GESHIP: 9
Checking for GICRE
Total dates missing for GICRE: 2
Checking for GILLETTE
Total dates missing for GILLETTE: 9
Checking for GLAND
Total dates missing for GLAND: 2
Checking for GLAXO
Total dates missing for GLAXO: 9
Checking for GLENMARK
Total dates missing for GLENMARK: 9
Checking for GLS
Total dates missing for GLS: 2
Checking for GMDCLTD
Total dates missing for GMDCLTD: 9
Checking for GMMPFAUDLR
Total dates missing for GMMPFAUDLR: 2
Checking for GMRINFRA
Total dates missing for GMRINFRA: 9
Checking for GNFC
Total dates missing for GNFC: 9
Checking for GODFRYPHLP
Total dates missing

Total dates missing for MFSL: 9
Checking for MGL
Total dates missing for MGL: 2
Checking for MHRIL
Total dates missing for MHRIL: 8
Checking for MINDACORP
Total dates missing for MINDACORP: 5
Checking for MMTC
Total dates missing for MMTC: 8
Checking for MOTHERSON
Total dates missing for MOTHERSON: 9
Checking for MOTILALOFS
Total dates missing for MOTILALOFS: 9
Checking for MPHASIS
Total dates missing for MPHASIS: 9
Checking for MRF
Total dates missing for MRF: 9
Checking for MRPL
Total dates missing for MRPL: 9
Checking for MSUMI
Total dates missing for MSUMI: 2
Checking for MTARTECH
Total dates missing for MTARTECH: 2
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 6
Checking for NAM-INDIA
Total dates missing for NAM-INDIA: 2
Checking for NATCOPHARM
Total dates missing for NATCOPHARM: 9
Checking for NATIONALUM
Total dates missing for NATIONALUM: 9
Checking for NAUKRI
Total dates missing for NAUKRI: 9
Checking for NAVINFLUOR
Total dates missing for NAVINFLUOR: 9
Checking f

Total dates missing for USHAMART: 9
Checking for UTIAMC
Total dates missing for UTIAMC: 2
Checking for VAIBHAVGBL
Total dates missing for VAIBHAVGBL: 9
Checking for VARROC
Total dates missing for VARROC: 2
Checking for VBL
Total dates missing for VBL: 2
Checking for VEDL
Total dates missing for VEDL: 9
Checking for VGUARD
Total dates missing for VGUARD: 9
Checking for VIJAYA
Total dates missing for VIJAYA: 2
Checking for VIPIND
Total dates missing for VIPIND: 9
Checking for VOLTAS
Total dates missing for VOLTAS: 9
Checking for VTL
Total dates missing for VTL: 9
Checking for WELCORP
Total dates missing for WELCORP: 9
Checking for WELSPUNLIV
Total dates missing for WELSPUNLIV: 9
Checking for WESTLIFE
Total dates missing for WESTLIFE: 2
Checking for WHIRLPOOL
Total dates missing for WHIRLPOOL: 9
Checking for WIPRO
Total dates missing for WIPRO: 9
Checking for YESBANK
Total dates missing for YESBANK: 9
Checking for ZEEL
Total dates missing for ZEEL: 9
Checking for ZENSARTECH
Total dates mi

In [52]:
ohlcv_data_df = ohlcv_data_df.reset_index()

In [53]:
ohlcv_data_df.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close',
                                          'Adj Close':'adj_close','Volume':'volume','Stock':'nse_symbol'},inplace=True)

In [54]:
ohlcv_data_df.head()

date        open        high         low       close   adj_close  \
0 2019-09-19  302.500000  317.625000  302.500000  317.625000  262.504486   
1 2019-09-20  332.250000  333.500000  332.250000  333.500000  275.624603   
2 2019-09-23  350.174988  350.174988  350.174988  350.174988  289.405853   
3 2019-09-24  367.674988  367.674988  367.674988  367.674988  303.868896   
4 2019-09-25  386.049988  386.049988  349.299988  351.237488  290.283936   

    volume nse_symbol  
0  7109860     360ONE  
1  2760648     360ONE  
2    34096     360ONE  
3  2496652     360ONE  
4   530580     360ONE

In [55]:
for symbol in missing_dates_cleaned_final.columns:
    print(f'Checking for {symbol}..')
    missing_dates = missing_dates_updated_df[symbol].dropna().tolist()
    
    for date in missing_dates:
        
        condition = (ohlcv_data_df['nse_symbol'] == symbol) & (ohlcv_data_df['date'] == date)
        available_data = ohlcv_data_df.loc[condition]
        print('Available..')
        
        if not available_data.empty:
            cleaned_final = pd.concat([cleaned_final, available_data], ignore_index=True)

Checking for 360ONE..
Available..
Available..
Checking for 3MINDIA..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for AARTIIND..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for AAVAS..
Available..
Available..
Checking for ABB..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for ABBOTINDIA..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for ABCAPITAL..
Available..
Available..
Checking for ABFRL..
Available..
Available..
Available..
Available..
Checking for ACC..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for ACE..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for 

Available..
Available..
Available..
Checking for BSE..
Available..
Available..
Checking for BSOFT..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for CAMPUS..
Available..
Available..
Checking for CAMS..
Available..
Available..
Checking for CANBK..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for CANFINHOME..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for CAPLIPOINT..
Available..
Available..
Checking for CARBORUNIV..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for CASTROLIND..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for CCL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Availa

Available..
Available..
Available..
Available..
Available..
Available..
Checking for FSL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GAEL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GAIL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GESHIP..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GICRE..
Available..
Available..
Checking for GILLETTE..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GLAND..
Available..
Available..
Checking for GLAXO..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for GLENMARK..
Available..
Available..
Available..
Avai

Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for IOC..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for IPCALAB..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for IRB..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for IRCON..
Available..
Available..
Checking for IRCTC..
Available..
Available..
Checking for IRFC..
Available..
Available..
Checking for ISEC..
Available..
Available..
Checking for ITC..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for ITI..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for J&KBANK..
Available..
Available..
Available..
Available..
Avai

Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for MRF..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for MRPL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for MSUMI..
Available..
Available..
Checking for MTARTECH..
Available..
Available..
Checking for MUTHOOTFIN..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for NAM-INDIA..
Available..
Available..
Checking for NATCOPHARM..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for NATIONALUM..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for NAUKRI..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking fo

Available..
Available..
Checking for SBIN..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SCHAEFFLER..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SCHNEIDER..
Available..
Available..
Available..
Available..
Available..
Checking for SHREECEM..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SHRIRAMFIN..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SHYAMMETL..
Available..
Available..
Checking for SIEMENS..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SIGNATURE..
Available..
Available..
Checking for SJVN..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for SKFIN

Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for USHAMART..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for UTIAMC..
Available..
Available..
Checking for VAIBHAVGBL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for VARROC..
Available..
Available..
Checking for VBL..
Available..
Available..
Checking for VEDL..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for VGUARD..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for VIJAYA..
Available..
Available..
Checking for VIPIND..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Available..
Checking for VOLTAS..
Available..
Available..
Available..
Available..
Available..

In [56]:
cleaned_final.head()

date    open    high     low   close     volume nse_symbol  adj_close
0 2019-09-19  302.50  317.63  302.50  317.63  7109860.0     360ONE     317.63
1 2019-09-20  332.25  333.50  332.25  333.50  2760648.0     360ONE     333.50
2 2019-09-23  350.18  350.18  350.18  350.18    34096.0     360ONE     350.18
3 2019-09-24  367.68  367.68  367.68  367.68  2496652.0     360ONE     367.68
4 2019-09-25  386.05  386.05  349.30  351.24   530580.0     360ONE     351.24

In [57]:
cleaned_final.shape

(1981258, 8)

In [58]:
missing_dates_all_data = get_missing_dates_for_all_stocks(cleaned_final)

Checking for 360ONE
Total dates missing for 360ONE: 2
Checking for 3MINDIA
Total dates missing for 3MINDIA: 7
Checking for AARTIIND
Total dates missing for AARTIIND: 7
Checking for AAVAS
Total dates missing for AAVAS: 2
Checking for ABB
Total dates missing for ABB: 7
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 7
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 2
Checking for ABFRL
Total dates missing for ABFRL: 3
Checking for ACC
Total dates missing for ACC: 7
Checking for ACE
Total dates missing for ACE: 7
Checking for ACI
Total dates missing for ACI: 2
Checking for ADANIENT
Total dates missing for ADANIENT: 7
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 2
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 7
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 6
Checking for AETHER
Total dates missing for AETHER: 2
Checking for AFFLE
Total dates missing for AFFLE: 2
Checking for AIAENG
Total dates missing for AIAENG: 7
Checking f

Total dates missing for FEDERALBNK: 7
Checking for FINCABLES
Total dates missing for FINCABLES: 7
Checking for FINEORG
Total dates missing for FINEORG: 2
Checking for FINPIPE
Total dates missing for FINPIPE: 7
Checking for FIVESTAR
Total dates missing for FIVESTAR: 2
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 2
Checking for FORTIS
Total dates missing for FORTIS: 7
Checking for FSL
Total dates missing for FSL: 7
Checking for GAEL
Total dates missing for GAEL: 7
Checking for GAIL
Total dates missing for GAIL: 7
Checking for GESHIP
Total dates missing for GESHIP: 7
Checking for GICRE
Total dates missing for GICRE: 2
Checking for GILLETTE
Total dates missing for GILLETTE: 7
Checking for GLAND
Total dates missing for GLAND: 2
Checking for GLAXO
Total dates missing for GLAXO: 7
Checking for GLENMARK
Total dates missing for GLENMARK: 7
Checking for GLS
Total dates missing for GLS: 2
Checking for GMDCLTD
Total dates missing for GMDCLTD: 7
Checking for GMMPFAUDLR
Total dates mi

Total dates missing for MRF: 7
Checking for MRPL
Total dates missing for MRPL: 7
Checking for MSUMI
Total dates missing for MSUMI: 2
Checking for MTARTECH
Total dates missing for MTARTECH: 2
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 5
Checking for NAM-INDIA
Total dates missing for NAM-INDIA: 2
Checking for NATCOPHARM
Total dates missing for NATCOPHARM: 7
Checking for NATIONALUM
Total dates missing for NATIONALUM: 7
Checking for NAUKRI
Total dates missing for NAUKRI: 7
Checking for NAVINFLUOR
Total dates missing for NAVINFLUOR: 7
Checking for NBCC
Total dates missing for NBCC: 4
Checking for NCC
Total dates missing for NCC: 7
Checking for NESTLEIND
Total dates missing for NESTLEIND: 7
Checking for NETWORK18
Total dates missing for NETWORK18: 7
Checking for NH
Total dates missing for NH: 2
Checking for NHPC
Total dates missing for NHPC: 6
Checking for NIACL
Total dates missing for NIACL: 2
Checking for NLCINDIA
Total dates missing for NLCINDIA: 7
Checking for NMDC
Total

Total dates missing for USHAMART: 7
Checking for UTIAMC
Total dates missing for UTIAMC: 2
Checking for VAIBHAVGBL
Total dates missing for VAIBHAVGBL: 7
Checking for VARROC
Total dates missing for VARROC: 2
Checking for VBL
Total dates missing for VBL: 2
Checking for VEDL
Total dates missing for VEDL: 7
Checking for VGUARD
Total dates missing for VGUARD: 7
Checking for VIJAYA
Total dates missing for VIJAYA: 2
Checking for VIPIND
Total dates missing for VIPIND: 7
Checking for VOLTAS
Total dates missing for VOLTAS: 7
Checking for VTL
Total dates missing for VTL: 7
Checking for WELCORP
Total dates missing for WELCORP: 7
Checking for WELSPUNLIV
Total dates missing for WELSPUNLIV: 7
Checking for WESTLIFE
Total dates missing for WESTLIFE: 2
Checking for WHIRLPOOL
Total dates missing for WHIRLPOOL: 7
Checking for WIPRO
Total dates missing for WIPRO: 7
Checking for YESBANK
Total dates missing for YESBANK: 7
Checking for ZEEL
Total dates missing for ZEEL: 7
Checking for ZENSARTECH
Total dates mi

In [59]:
cleaned_final.sort_values(by=['nse_symbol','date'],ascending=[True,True],inplace=True)

In [60]:
cleaned_final['date'].min(),cleaned_final['date'].max()

(Timestamp('2000-01-03 00:00:00'), Timestamp('2024-11-04 00:00:00'))

In [61]:
cleaned_final[cleaned_final['volume'].isna()].shape

(114, 8)

In [62]:
cleaned_final.loc[cleaned_final[['open', 'high', 'low', 'close']].notna().all(axis=1) & cleaned_final['volume'].isna(), 'volume'] = 0

In [63]:
cleaned_final[cleaned_final['volume'].isna()].shape

(0, 8)

In [64]:
cleaned_final.isna().sum()

date          0
open          0
high          0
low           0
close         0
volume        0
nse_symbol    0
adj_close     0
dtype: int64

In [65]:
import pandas as pd

def count_zero_volume_days(df):
    result = {}
    
    grouped = df.groupby('nse_symbol')

    for symbol, group in grouped:
        group = group.sort_values('date')
        
        first_valid_index = group['volume'].first_valid_index()
        if first_valid_index is not None:
            # Get the first date when the stock was listed
            first_valid_date = group.loc[first_valid_index, 'date']
            
            # Filter rows after the stock was listed
            listed_df = group[group['date'] >= first_valid_date]
            
            # Count consecutive days where volume is 0 since the stock was listed
            consecutive_zero_days = 0
            for vol in listed_df['volume']:
                if vol == 0:
                    consecutive_zero_days += 1
                else:
                    break
            
            # Store the result
            result[symbol] = consecutive_zero_days
    
    # Print the results
    for stock, days in result.items():
        print(f"Stock: {stock}, Continuous days with volume 0 since listing: {days}")
        
    return result


In [66]:
result = count_zero_volume_days(cleaned_final)

Stock: 360ONE, Continuous days with volume 0 since listing: 0
Stock: 3MINDIA, Continuous days with volume 0 since listing: 0
Stock: AARTIIND, Continuous days with volume 0 since listing: 0
Stock: AAVAS, Continuous days with volume 0 since listing: 0
Stock: ABB, Continuous days with volume 0 since listing: 0
Stock: ABBOTINDIA, Continuous days with volume 0 since listing: 0
Stock: ABCAPITAL, Continuous days with volume 0 since listing: 0
Stock: ABFRL, Continuous days with volume 0 since listing: 0
Stock: ACC, Continuous days with volume 0 since listing: 0
Stock: ACE, Continuous days with volume 0 since listing: 1
Stock: ACI, Continuous days with volume 0 since listing: 0
Stock: ADANIENT, Continuous days with volume 0 since listing: 0
Stock: ADANIGREEN, Continuous days with volume 0 since listing: 0
Stock: ADANIPORTS, Continuous days with volume 0 since listing: 1
Stock: ADANIPOWER, Continuous days with volume 0 since listing: 1
Stock: AETHER, Continuous days with volume 0 since listing: 

In [67]:
result

{'360ONE': 0,
 '3MINDIA': 0,
 'AARTIIND': 0,
 'AAVAS': 0,
 'ABB': 0,
 'ABBOTINDIA': 0,
 'ABCAPITAL': 0,
 'ABFRL': 0,
 'ACC': 0,
 'ACE': 1,
 'ACI': 0,
 'ADANIENT': 0,
 'ADANIGREEN': 0,
 'ADANIPORTS': 1,
 'ADANIPOWER': 1,
 'AETHER': 0,
 'AFFLE': 0,
 'AIAENG': 0,
 'AJANTPHARM': 0,
 'ALKEM': 0,
 'ALKYLAMINE': 0,
 'ALLCARGO': 1,
 'ALOKINDS': 0,
 'AMBER': 0,
 'AMBUJACEM': 0,
 'ANANDRATHI': 0,
 'ANGELONE': 0,
 'ANURAS': 0,
 'APARINDS': 0,
 'APLAPOLLO': 0,
 'APLLTD': 0,
 'APOLLOHOSP': 0,
 'APOLLOTYRE': 0,
 'APTUS': 0,
 'ASAHIINDIA': 0,
 'ASHOKLEY': 0,
 'ASIANPAINT': 0,
 'ASTERDM': 0,
 'ASTRAL': 1,
 'ASTRAZEN': 0,
 'ATGL': 0,
 'ATUL': 0,
 'AUBANK': 0,
 'AUROPHARMA': 0,
 'AVANTIFEED': 0,
 'AWL': 0,
 'AXISBANK': 0,
 'BAJAJ-AUTO': 0,
 'BAJAJFINSV': 0,
 'BAJAJHLDNG': 0,
 'BAJFINANCE': 0,
 'BALAMINES': 0,
 'BALKRISIND': 0,
 'BALRAMCHIN': 0,
 'BANDHANBNK': 0,
 'BANKBARODA': 0,
 'BANKINDIA': 0,
 'BATAINDIA': 0,
 'BAYERCROP': 0,
 'BBTC': 0,
 'BDL': 0,
 'BEL': 0,
 'BEML': 0,
 'BERGEPAINT': 0,
 'BHARATFO

In [68]:
symbols_with_value_gt_0 = [symbol for symbol, value in result.items() if value > 0]

In [69]:
len(symbols_with_value_gt_0),cleaned_final[cleaned_final['volume']==0]['nse_symbol'].nunique()

(61, 83)

In [70]:
def clean_data(df, row_count_dict):
    cleaned_data = []

    grouped = df.groupby('nse_symbol')

    for symbol, group in grouped:
        print(f'Cleaning for {symbol}')
        if symbol in row_count_dict:
            rows_to_remove = row_count_dict[symbol]
            if rows_to_remove > 0:
                print(f'Removed rows {rows_to_remove}')
                group = group.iloc[rows_to_remove:]

            cleaned_data.append(group)
    
        cleaned_df = pd.concat(cleaned_data).reset_index(drop=True)
    
    return cleaned_df

In [71]:
final_vol_cleaned = clean_data(cleaned_final, result)

Cleaning for 360ONE
Cleaning for 3MINDIA
Cleaning for AARTIIND
Cleaning for AAVAS
Cleaning for ABB
Cleaning for ABBOTINDIA
Cleaning for ABCAPITAL
Cleaning for ABFRL
Cleaning for ACC
Cleaning for ACE
Removed rows 1
Cleaning for ACI
Cleaning for ADANIENT
Cleaning for ADANIGREEN
Cleaning for ADANIPORTS
Removed rows 1
Cleaning for ADANIPOWER
Removed rows 1
Cleaning for AETHER
Cleaning for AFFLE
Cleaning for AIAENG
Cleaning for AJANTPHARM
Cleaning for ALKEM
Cleaning for ALKYLAMINE
Cleaning for ALLCARGO
Removed rows 1
Cleaning for ALOKINDS
Cleaning for AMBER
Cleaning for AMBUJACEM
Cleaning for ANANDRATHI
Cleaning for ANGELONE
Cleaning for ANURAS
Cleaning for APARINDS
Cleaning for APLAPOLLO
Cleaning for APLLTD
Cleaning for APOLLOHOSP
Cleaning for APOLLOTYRE
Cleaning for APTUS
Cleaning for ASAHIINDIA
Cleaning for ASHOKLEY
Cleaning for ASIANPAINT
Cleaning for ASTERDM
Cleaning for ASTRAL
Removed rows 1
Cleaning for ASTRAZEN
Cleaning for ATGL
Cleaning for ATUL
Cleaning for AUBANK
Cleaning for AUR

Cleaning for POLYCAB
Cleaning for POLYMED
Cleaning for POONAWALLA
Cleaning for POWERGRID
Removed rows 1
Cleaning for POWERINDIA
Cleaning for PPLPHARMA
Cleaning for PRAJIND
Cleaning for PRESTIGE
Removed rows 1
Cleaning for PRINCEPIPE
Cleaning for PRSMJOHNSN
Cleaning for QUESS
Cleaning for RADICO
Cleaning for RAILTEL
Cleaning for RAINBOW
Cleaning for RAJESHEXPO
Cleaning for RAMCOCEM
Cleaning for RATNAMANI
Cleaning for RAYMOND
Cleaning for RBA
Cleaning for RBLBANK
Cleaning for RCF
Cleaning for RECLTD
Removed rows 1
Cleaning for REDINGTON
Removed rows 1
Cleaning for RELIANCE
Cleaning for RENUKA
Removed rows 1
Cleaning for RHIM
Cleaning for RITES
Cleaning for RKFORGE
Cleaning for ROUTE
Cleaning for RRKABEL
Cleaning for RTNINDIA
Cleaning for RVNL
Cleaning for SAFARI
Cleaning for SAIL
Cleaning for SANOFI
Cleaning for SAPPHIRE
Cleaning for SAREGAMA
Cleaning for SBFC
Cleaning for SBICARD
Cleaning for SBILIFE
Removed rows 1
Cleaning for SBIN
Cleaning for SCHAEFFLER
Cleaning for SCHNEIDER
Cleanin

In [72]:
for i in (final_vol_cleaned[final_vol_cleaned['volume']==0]['nse_symbol'].unique().tolist()):
    print(i)

ABBOTINDIA
ALKYLAMINE
AVANTIFEED
BRITANNIA
CGCL
DEEPAKNTR
DLF
EMAMILTD
ENGINERSIN
FSL
GMRINFRA
GRINDWELL
HFCL
J&KBANK
JBMA
JSWSTEEL
MINDACORP
NESTLEIND
PATANJALI
PIIND
RKFORGE
RTNINDIA
UBL
WHIRLPOOL
ZYDUSLIFE


In [73]:
final_vol_cleaned.shape

(1981144, 8)

In [74]:
import pandas as pd

def filter_data_by_symbol(cleaned_df):
    cleaned_df.sort_values(by=['nse_symbol','date'],ascending=[True,True],inplace=True)
    
    grouped = cleaned_df.groupby('nse_symbol')

    filtered_data = []

    for symbol, group in grouped:
        print(f'Filtering for {symbol}')
        
        if symbol == 'PATANJALI':
            group = group[group['date'] >= '2003-03-01']
            
        elif symbol == 'ABBOTINDIA':
            group = group[group['date'] > '2009-09-30']
            
        elif symbol == 'ALKYLAMINE':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'AVANTIFEED':
            group = group[group['date'] > '2015-04-01']
        
        elif symbol == 'BRITANNIA':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'CGCL':
            group = group[group['date'] > '2013-09-30']
            
        elif symbol == 'DEEPAKNTR':
            group = group[group['date'] > '2013-04-11']
            
        elif symbol == 'EMAMILTD':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'ENGINERSIN':
            group = group[group['date'] > '2014-02-19']
        
        elif symbol == 'GRINDWELL':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'HFCL':
            group = group[group['date'] > '2011-02-16']
            
        elif symbol == 'J&KBANK':
            group = group[group['date'] > '2017-02-21']
            
        elif symbol == 'JBMA':
            group = group[group['date'] > '2013-07-01']
            
        elif symbol == 'JSWSTEEL':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'MINDACORP':
            group = group[group['date'] > '2014-04-01']
            
        elif symbol == 'NESTLEIND':
            group = group[group['date'] > '2009-09-30']
            
        elif symbol == 'PIIND':
            group = group[group['date'] > '2011-07-01']
            
        elif symbol == 'RKFORGE':
            group = group[group['date'] > '2010-05-27']
            
        elif symbol == 'RTNINDIA':
            group = group[group['date'] > '2020-02-19']
            
        elif symbol == 'UBL':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'WHIRLPOOL':
            group = group[group['date'] > '2009-09-30']
            
        elif symbol == 'ZYDUSLIFE':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'DLF':
            group = group[group['date'] > '2014-02-19']
            
        elif symbol == 'FSL':
            group = group[group['date'] > '2017-02-21']
            
        elif symbol == 'GMRINFRA':
            group = group[group['date'] > '2014-02-19']
        
        filtered_data.append(group)

    filtered_df = pd.concat(filtered_data).reset_index(drop=True)
    
    return filtered_df


In [75]:
final_data_to_save = filter_data_by_symbol(final_vol_cleaned)

Filtering for 360ONE
Filtering for 3MINDIA
Filtering for AARTIIND
Filtering for AAVAS
Filtering for ABB
Filtering for ABBOTINDIA
Filtering for ABCAPITAL
Filtering for ABFRL
Filtering for ACC
Filtering for ACE
Filtering for ACI
Filtering for ADANIENT
Filtering for ADANIGREEN
Filtering for ADANIPORTS
Filtering for ADANIPOWER
Filtering for AETHER
Filtering for AFFLE
Filtering for AIAENG
Filtering for AJANTPHARM
Filtering for ALKEM
Filtering for ALKYLAMINE
Filtering for ALLCARGO
Filtering for ALOKINDS
Filtering for AMBER
Filtering for AMBUJACEM
Filtering for ANANDRATHI
Filtering for ANGELONE
Filtering for ANURAS
Filtering for APARINDS
Filtering for APLAPOLLO
Filtering for APLLTD
Filtering for APOLLOHOSP
Filtering for APOLLOTYRE
Filtering for APTUS
Filtering for ASAHIINDIA
Filtering for ASHOKLEY
Filtering for ASIANPAINT
Filtering for ASTERDM
Filtering for ASTRAL
Filtering for ASTRAZEN
Filtering for ATGL
Filtering for ATUL
Filtering for AUBANK
Filtering for AUROPHARMA
Filtering for AVANTIFEE

In [76]:
final_data_to_save.shape

(1927322, 8)

In [77]:
final_data_to_save[final_data_to_save['volume']==0]['nse_symbol'].unique()

array([], dtype=object)

In [78]:
missing_dates_final_data_to_save = get_missing_dates_for_all_stocks(final_data_to_save)

Checking for 360ONE
Total dates missing for 360ONE: 2
Checking for 3MINDIA
Total dates missing for 3MINDIA: 7
Checking for AARTIIND
Total dates missing for AARTIIND: 7
Checking for AAVAS
Total dates missing for AAVAS: 2
Checking for ABB
Total dates missing for ABB: 7
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 6
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 2
Checking for ABFRL
Total dates missing for ABFRL: 3
Checking for ACC
Total dates missing for ACC: 7
Checking for ACE
Total dates missing for ACE: 7
Checking for ACI
Total dates missing for ACI: 2
Checking for ADANIENT
Total dates missing for ADANIENT: 7
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 2
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 7
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 6
Checking for AETHER
Total dates missing for AETHER: 2
Checking for AFFLE
Total dates missing for AFFLE: 2
Checking for AIAENG
Total dates missing for AIAENG: 7
Checking f

Total dates missing for ENGINERSIN: 2
Checking for EPL
Total dates missing for EPL: 7
Checking for EQUITASBNK
Total dates missing for EQUITASBNK: 2
Checking for ERIS
Total dates missing for ERIS: 2
Checking for ESCORTS
Total dates missing for ESCORTS: 7
Checking for EXIDEIND
Total dates missing for EXIDEIND: 7
Checking for FACT
Total dates missing for FACT: 7
Checking for FDC
Total dates missing for FDC: 7
Checking for FEDERALBNK
Total dates missing for FEDERALBNK: 7
Checking for FINCABLES
Total dates missing for FINCABLES: 7
Checking for FINEORG
Total dates missing for FINEORG: 2
Checking for FINPIPE
Total dates missing for FINPIPE: 7
Checking for FIVESTAR
Total dates missing for FIVESTAR: 2
Checking for FLUOROCHEM
Total dates missing for FLUOROCHEM: 2
Checking for FORTIS
Total dates missing for FORTIS: 7
Checking for FSL
Total dates missing for FSL: 2
Checking for GAEL
Total dates missing for GAEL: 7
Checking for GAIL
Total dates missing for GAIL: 7
Checking for GESHIP
Total dates mi

Total dates missing for MFSL: 7
Checking for MGL
Total dates missing for MGL: 2
Checking for MHRIL
Total dates missing for MHRIL: 6
Checking for MINDACORP
Total dates missing for MINDACORP: 2
Checking for MMTC
Total dates missing for MMTC: 6
Checking for MOTHERSON
Total dates missing for MOTHERSON: 7
Checking for MOTILALOFS
Total dates missing for MOTILALOFS: 7
Checking for MPHASIS
Total dates missing for MPHASIS: 7
Checking for MRF
Total dates missing for MRF: 7
Checking for MRPL
Total dates missing for MRPL: 7
Checking for MSUMI
Total dates missing for MSUMI: 2
Checking for MTARTECH
Total dates missing for MTARTECH: 2
Checking for MUTHOOTFIN
Total dates missing for MUTHOOTFIN: 5
Checking for NAM-INDIA
Total dates missing for NAM-INDIA: 2
Checking for NATCOPHARM
Total dates missing for NATCOPHARM: 7
Checking for NATIONALUM
Total dates missing for NATIONALUM: 7
Checking for NAUKRI
Total dates missing for NAUKRI: 7
Checking for NAVINFLUOR
Total dates missing for NAVINFLUOR: 7
Checking f

Total dates missing for VIPIND: 7
Checking for VOLTAS
Total dates missing for VOLTAS: 7
Checking for VTL
Total dates missing for VTL: 7
Checking for WELCORP
Total dates missing for WELCORP: 7
Checking for WELSPUNLIV
Total dates missing for WELSPUNLIV: 7
Checking for WESTLIFE
Total dates missing for WESTLIFE: 2
Checking for WHIRLPOOL
Total dates missing for WHIRLPOOL: 6
Checking for WIPRO
Total dates missing for WIPRO: 7
Checking for YESBANK
Total dates missing for YESBANK: 7
Checking for ZEEL
Total dates missing for ZEEL: 7
Checking for ZENSARTECH
Total dates missing for ZENSARTECH: 7
Checking for ZFCVINDIA
Total dates missing for ZFCVINDIA: 7
Checking for ZOMATO
Total dates missing for ZOMATO: 2
Checking for ZYDUSLIFE
Total dates missing for ZYDUSLIFE: 2


In [79]:
missing_dates_final_data_to_save

360ONE     3MINDIA    AARTIIND       AAVAS         ABB  ABBOTINDIA  \
0   2024-01-22  2008-10-28  2008-10-28  2024-01-22  2008-10-28  2010-11-05   
1   2024-05-20  2010-11-05  2010-11-05  2024-05-20  2010-11-05  2011-10-26   
2          NaN  2011-10-26  2011-10-26         NaN  2011-10-26  2012-11-13   
3          NaN  2012-11-13  2012-11-13         NaN  2012-11-13  2013-11-15   
4          NaN  2013-11-15  2013-11-15         NaN  2013-11-15  2024-01-22   
5          NaN  2024-01-22  2024-01-22         NaN  2024-01-22  2024-05-20   
6          NaN  2024-05-20  2024-05-20         NaN  2024-05-20         NaN   
7          NaN         NaN         NaN         NaN         NaN         NaN   
8          NaN         NaN         NaN         NaN         NaN         NaN   
9          NaN         NaN         NaN         NaN         NaN         NaN   
10         NaN         NaN         NaN         NaN         NaN         NaN   
11         NaN         NaN         NaN         NaN         NaN         NaN   
12         NaN         NaN         NaN         NaN         NaN         NaN   
13         NaN         NaN         NaN         NaN         NaN         NaN   
14         NaN         NaN         NaN         NaN         NaN         NaN   
15         NaN         NaN         NaN         NaN         NaN         NaN   
16         NaN         NaN         NaN         NaN         NaN         NaN   
17         NaN         NaN         NaN         NaN         NaN         NaN   
18         NaN         NaN         NaN         NaN         NaN         NaN   
19         NaN         NaN         NaN         NaN         NaN         NaN   
20         NaN         NaN         NaN         NaN         NaN         NaN   
21         NaN         NaN         NaN         NaN         NaN         NaN   
22         NaN         NaN         NaN         NaN         NaN         NaN   
23         NaN         NaN         NaN         NaN         NaN         NaN   
24         NaN         NaN         NaN         NaN         NaN         NaN   
25         NaN         NaN         NaN         NaN         NaN         NaN   
26         NaN         NaN         NaN         NaN         NaN         NaN   
27         NaN         NaN         NaN         NaN         NaN         NaN   
28         NaN         NaN         NaN         NaN         NaN         NaN   
29         NaN         NaN         NaN         NaN         NaN         NaN   
30         NaN         NaN         NaN         NaN         NaN         NaN   
31         NaN         NaN         NaN         NaN         NaN         NaN   
32         NaN         NaN         NaN         NaN         NaN         NaN   
33         NaN         NaN         NaN         NaN         NaN         NaN   
34         NaN         NaN         NaN         NaN         NaN         NaN   
35         NaN         NaN         NaN         NaN         NaN         NaN   
36         NaN         NaN         NaN         NaN         NaN         NaN   
37         NaN         NaN         NaN         NaN         NaN         NaN   
38         NaN         NaN         NaN         NaN         NaN         NaN   
39         NaN         NaN         NaN         NaN         NaN         NaN   
40         NaN         NaN         NaN         NaN         NaN         NaN   
41         NaN         NaN         NaN         NaN         NaN         NaN   
42         NaN         NaN         NaN         NaN         NaN         NaN   
43         NaN         NaN         NaN         NaN         NaN         NaN   
44         NaN         NaN         NaN         NaN         NaN         NaN   
45         NaN         NaN         NaN         NaN         NaN         NaN   
46         NaN         NaN         NaN         NaN         NaN         NaN   
47         NaN         NaN         NaN         NaN         NaN         NaN   
48         NaN         NaN         NaN         NaN         NaN         NaN   
49         NaN         NaN         NaN         NaN         NaN         NaN   
50         NaN         NaN    

In [80]:
missing_dates_final_data_to_save['3MINDIA']

0     2008-10-28
1     2010-11-05
2     2011-10-26
3     2012-11-13
4     2013-11-15
5     2024-01-22
6     2024-05-20
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
30           NaN
31           NaN
32           NaN
33           NaN
34           NaN
35           NaN
36           NaN
37           NaN
38           NaN
39           NaN
40           NaN
41           NaN
42           NaN
43           NaN
44           NaN
45           NaN
46           NaN
47           NaN
48           NaN
49           NaN
50           NaN
51           NaN
52           NaN
53           NaN
54           NaN
55           NaN
56           NaN
57           NaN
58           N

In [81]:
final_data_to_save.sort_values(by=['nse_symbol','date'],ascending=[True,True],inplace=True)

In [82]:
final_data_to_save.isna().sum()

date          0
open          0
high          0
low           0
close         0
volume        0
nse_symbol    0
adj_close     0
dtype: int64

In [83]:
# final_data_to_save.to_parquet(r"D:\alpha_generator\alpha_short_calls\data\all_raw_bloomberg_yahoo_vol_cleaned.parquet")

In [83]:
final_data_to_save['nse_symbol'].value_counts()

JINDALSAW     6141
SIEMENS       6141
FACT          6141
FDC           6141
FINCABLES     6141
FINPIPE       6141
SRF           6141
GAIL          6141
GESHIP        6141
GILLETTE      6141
GLAXO         6141
SKFINDIA      6141
GMDCLTD       6141
GNFC          6141
ESCORTS       6141
GODFRYPHLP    6141
GODREJIND     6141
GRAPHITE      6141
GRASIM        6141
SHRIRAMFIN    6141
SHREECEM      6141
GSFC          6141
SCHAEFFLER    6141
SBIN          6141
SANOFI        6141
HDFCBANK      6141
EXIDEIND      6141
SUNDARMFIN    6141
CONCOR        6141
DEEPAKFERT    6141
TATASTEEL     6141
CRISIL        6141
TATAPOWER     6141
TATAMOTORS    6141
CUB           6141
CUMMINSIND    6141
CYIENT        6141
DABUR         6141
TATAINVEST    6141
TATAELXSI     6141
DCMSHRIRAM    6141
TATACONSUM    6141
SUNDRMFAST    6141
TATACOMM      6141
TATACHEM      6141
DRREDDY       6141
SUPREMEIND    6141
EICHERMOT     6141
EIDPARRY      6141
EIHOTEL       6141
ELECON        6141
ELGIEQUIP     6141
SUNPHARMA   

In [84]:
final_data_to_save['date'] = pd.to_datetime(final_data_to_save['date'])
print(final_data_to_save['date'].min(),final_data_to_save['date'].max())
final_data_to_save['nse_symbol'].nunique()

2000-01-03 00:00:00 2024-11-04 00:00:00


489

In [85]:
max_dt = pd.to_datetime(final_data_to_save['date'].max())
max_dt = max_dt.strftime(('%Y-%m-%d'))

In [86]:
symbols_above_50_df = final_data_to_save[(final_data_to_save['date'] == max_dt) & (final_data_to_save['close'] >= 50)]

symbols_above_50 = symbols_above_50_df['nse_symbol'].unique().tolist()

filtered_stock_data = final_data_to_save[final_data_to_save['nse_symbol'].isin(symbols_above_50)].reset_index(drop=True)

In [87]:
filtered_stock_data.shape

(1889600, 8)

In [88]:
filtered_stock_data['nse_symbol'].nunique()

479

In [89]:
filtered_stock_data.sort_values(by=['nse_symbol', 'date'], ascending=[True, True], inplace=True)

filtered_stock_data['next_day_return_close'] = ((filtered_stock_data.groupby('nse_symbol')['close'].shift(-1) - filtered_stock_data.groupby('nse_symbol')['open'].shift(-1)) / filtered_stock_data.groupby('nse_symbol')['open'].shift(-1) * 100).round(2)

filtered_stock_data['target_close'] = (filtered_stock_data['next_day_return_close'] >= 2).astype(int)

filtered_stock_data['next_day_return_high'] = ((filtered_stock_data.groupby('nse_symbol')['high'].shift(-1) - filtered_stock_data.groupby('nse_symbol')['open'].shift(-1)) / filtered_stock_data.groupby('nse_symbol')['open'].shift(-1) * 100).round(2)

filtered_stock_data['target_high'] = (filtered_stock_data['next_day_return_high'] >= 2).astype(int)

filtered_stock_data['next_day_return_low'] = ((filtered_stock_data.groupby('nse_symbol')['low'].shift(-1) - filtered_stock_data.groupby('nse_symbol')['open'].shift(-1)) / filtered_stock_data.groupby('nse_symbol')['open'].shift(-1) * 100).round(2)

filtered_stock_data['target_low'] = (filtered_stock_data['next_day_return_low'] <= -2).astype(int)

In [90]:
filtered_stock_data.head(2)

date    open    high     low   close     volume nse_symbol  adj_close  \
0 2019-09-19  302.50  317.63  302.50  317.63  7109860.0     360ONE     317.63   
1 2019-09-20  332.25  333.50  332.25  333.50  2760648.0     360ONE     333.50   

   next_day_return_close  target_close  next_day_return_high  target_high  \
0                   0.38             0                  0.38            0   
1                   0.00             0                  0.00            0   

   next_day_return_low  target_low  
0                  0.0           0  
1                  0.0           0

In [91]:
filtered_stock_data['target_close'].value_counts(), filtered_stock_data['target_close'].value_counts(normalize=True)*100

(0    1603980
 1     285620
 Name: target_close, dtype: int64,
 0    84.884632
 1    15.115368
 Name: target_close, dtype: float64)

In [92]:
filtered_stock_data['target_high'].value_counts(), filtered_stock_data['target_high'].value_counts(normalize=True)*100

(0    1227029
 1     662571
 Name: target_high, dtype: int64,
 0    64.935912
 1    35.064088
 Name: target_high, dtype: float64)

In [93]:
filtered_stock_data['target_low'].value_counts(), filtered_stock_data['target_low'].value_counts(normalize=True)*100

(0    1180205
 1     709395
 Name: target_low, dtype: int64,
 0    62.457928
 1    37.542072
 Name: target_low, dtype: float64)

In [94]:
filtered_stock_data.isna().sum()

date                       0
open                       0
high                       0
low                        0
close                      0
volume                     0
nse_symbol                 0
adj_close                  0
next_day_return_close    479
target_close               0
next_day_return_high     479
target_high                0
next_day_return_low      479
target_low                 0
dtype: int64

In [95]:
filtered_stock_data['date'].min(),filtered_stock_data['date'].max()

(Timestamp('2000-01-03 00:00:00'), Timestamp('2024-11-04 00:00:00'))

In [96]:
filtered_stock_data.to_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_raw_bloomberg_yahoo_vol_cleaned_with_target_intraday.parquet")

In [2]:
filtered_stock_data = pd.read_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_raw_bloomberg_yahoo_vol_cleaned_with_target_intraday.parquet")

In [3]:
filtered_stock_data['date'] = pd.to_datetime(filtered_stock_data['date'])
print(filtered_stock_data['date'].min(),filtered_stock_data['date'].max())
filtered_stock_data['nse_symbol'].nunique()

2000-01-03 00:00:00 2024-11-07 00:00:00


480

In [4]:
filtered_stock_data = filtered_stock_data[filtered_stock_data['date']>='2022-01-01']

In [5]:
def add_technical_indicators(df):
    """
    Add all specified technical indicators to the DataFrame with parameters suitable for short-term analysis.
    
    Parameters:
    - df: The DataFrame to which indicators will be added.
    """
    
    # Add Aberration Indicator
    df.ta.aberration(length=29, append=True)  #done

    # Add Accumulation Bands Indicator
    df.ta.accbands(length=5, c=4, mamode='dema', append=True)  #done

    # Add Accumulation Distribution Indicator
    df.ta.ad(append=True) #done

    # Add Accumulation Distribution Oscillator Indicator
    df.ta.adosc(high='high', low='low', close='close', volume='volume', fast=29, slow=28, append=True)  #done

    # Add Arnaud Legoux Moving Average (ALMA) Indicator
    df.ta.alma(close='close', length=22, sigma=6.0, distribution_offset=0.85, append=True)  #done

    # Add Adaptive Moving Average (AMA) Indicator
    df.ta.amat(close='close', fast=24, slow=25, lookback=28, mamode='rma', append=True)  #done

    # Add Awesome Oscillator (AO) Indicator
    df.ta.ao(high='high', low='low', fast=8, slow=28, append=True)  #done

    # Add Accumulation On-Balance Volume Indicator
    df.ta.aobv(close='close', volume='volume', fast=18, slow=29, max_lookback=22, min_lookback=22, mamode="sinwma",append=True) #done

    # Add Absolute Price Oscillator (APO) Indicator
    df.ta.apo(close='close', fast=18, slow=10, mamode="rma", append=True)  #done

    # Add Average True Range (ATR) Indicator
    df.ta.atr(high='high', low='low', close='close', length=28, mamode="ema", append=True) #done

    # Add Bollinger Bands Indicator
    df.ta.bbands(close='close', length=21, std=2, mamode="sma", append=True) #done

    # Add Bias Indicator
    df.ta.bias(close='close', length=27, mamode="sma", append=True) #done

    # Add Bollinger Bands Percentage (BRAR) Indicator
    df.ta.brar(open='open', high='high', low='low', close='close', length=25, scalar=100, append=True)  #done

    # Add Candle Pattern Recognition Indicator
    df.ta.cdl_pattern(open='open', high='high', low='low', close='close', append=True) #done

    # Add Candle Pattern Recognition with Z Indicator
    df.ta.cdl_z(open='open', high='high', low='low', close='close', length=27, ddof=5 , append=True) #done

    # Add Chaikin's Money Flow (CMF) Indicator
    df.ta.cmf(high='high', low='low', close='close',volume='volume', open='open', length=22, append=True)  #done

    # Add Center of Gravity (CG) Indicator
    df.ta.cg(close ='close', length=8, append=True) #done

    # Add Choppiness Index Indicator
    df.ta.chop(high='high', low='low', close='close', length=11, atr_length=26, append=True)  #done

    # Add Cumulative Keltner Channel with Standard Deviation (CKSP) Indicator
    df.ta.cksp(high='high', low='low', close='close', p=7, x=1, q=6, tvmode=True, append=True) #done

    # Add Chaikin Oscillator Indicator
    df.ta.cmo(close='close', length=22, scalar=100, drift=1, append=True) #done

    # Add Coppock Curve Indicator
    df.ta.coppock(close='close', length=4, fast=10, slow=26, append=True)   #done

    # Add Cumulative Trend Indicator
    df.ta.cti(close='close', length=25, append=True)   #done

    # Add Decreasing Trend Indicator
    df.ta.decreasing(close='close', length=25, strict=False, asint=True, append=True) #done

    # Add Double Exponential Moving Average (DEMA) Indicator
    df.ta.dema(close='close', length=16, append=True)  #done

    # Add Directional Movement Indicator (DM) Indicator
    df.ta.dm(high='high', low='low', length=28, mamode="ema", append=True)  #done

    # Add Donchian Channel Indicator
    df.ta.donchian(high='high', low='low', lower_length=27, upper_length=22, append=True) #done

    # Add Detrended Price Oscillator (DPO) Indicator
#     df.ta.dpo(close='close', length=20, centered=False, append=True) 

    # Add Exponential Bollinger Bands (EBSW) Indicator
    df.ta.ebsw(length=23, append=True)  # Short-term length   #done

    # Add Elder Force Index (EFI) Indicator
    df.ta.efi(close='close', length=7, mamode='rma', append=True)  #done

    # Add Exponential Moving Average (EMA) Indicator
    df.ta.ema(close='close', length=28, adjust=False, sma=True, append=True)  #done

    # Add Entropy Indicator
    df.ta.entropy(close='close', length=27, base=2, append=True)  #done

    # Add Ease of Movement (EOM) Indicator
    df.ta.eom(high='high', low='low', close='close', volume='volume', length=26, append=True)  #done

    # Add Efficiency Ratio (ER) Indicator
    df.ta.er(close='close', length=21, append=True)  #done

    # Add Elder-Ray Index (ERI) Indicator
    df.ta.eri(high='high', low='low', close='close', length=25, append=True)  #done

    # Add Fisher Transform Indicator
    df.ta.fisher(high='high', low='low', length=27, signal=5, append=True) #done

    # Add Forward Weighted Moving Average (FWMA) Indicator
    df.ta.fwma(close='close', length=24, append=True)  # Short-term length #done

    # Add Heikin-Ashi Candlestick Indicator
    df.ta.ha(append=True) #done

    # Add High-Low Oscillator Indicator
    df.ta.hilo(high='high',low='low',close='close',high_length=16,low_length=28, mamode='swma', append=True)  #done

    # Add High-Low Average Indicator
    df.ta.hl2(high='high', low='low', append=True) #done

    # Add High-Low-Close Average Indicator
    df.ta.hlc3(high='high', low='low', close='close', append=True) #done

    # Add Hull Moving Average (HMA) Indicator
    df.ta.hma(close='close', length=27, append=True)  #done

    # Add Hull Weighted Moving Average (HWMA) Indicator
#     df.ta.hwma(close='close', na=0.2, nb=0.1, nc=0.1, append=True)  #pending

    # Add Ichimoku Cloud Indicator
    df.ta.ichimoku(high='high', low='low', close='close', tenkan=11, kijun=24, senkou=11, include_chikou=True, append=True)  #done

    # Add Increasing Trend Indicator
    df.ta.increasing(close='close', length=25, strict=False, asint=True, append=True)  #done

    # Add Inertia Indicator
    df.ta.inertia(close='close', length=5, rvi_length=4, mamode='ema', append=True)  #done

    # Add Keltner Channel Indicator
    df.ta.kc(high='high', low='low', close='close', length=10, scalar=2, mamode='tema', append=True)  #done

    # Add KDJ Indicator
    df.ta.kdj(high='high', low='low', close='close', length=27, signal=27, append=True)  #done

    # Add Know Sure Thing (KST) Indicator
    df.ta.kst(close='close', roc1=11, roc2=17, roc3=9, roc4=17, sma1=7, sma2=6, sma3=6, sma4=26, signal=6, append=True) #done 

    # Add Kurtosis Indicator
    df.ta.kurtosis(close='close', length=25, append=True)  #done

    # Add Klinger Volume Oscillator (KVO) Indicator
#     df.ta.kvo(high='high', low='low', close='close', volume='volume', fast=34, slow=55, signal=13, append=True)  #pending

    # Add Linear Regression Indicator
    df.ta.linreg(close='close', length=25, offset=0, append=True)  #done

    # Add Log Return Indicator
    df.ta.log_return(close='close', length=27, cumulative=False, offset=0, append=True)  #done

    # Add Mean Absolute Deviation (MAD) Indicator
    df.ta.mad(close='close', length=22, offset=0, append=True)  #done

    # Add Mass Index Indicator
    df.ta.massi(high='high', low='low', fast=5, slow=28, offset=0, append=True)  #done

    # Add Median Indicator
    df.ta.median(close='close', length=28, offset=0, append=True)  #done

    # Add Momentum Indicator
    df.ta.mom(close='close', length=25, append=True)  # Short-term length #done

    # Add Normalized Average True Range (NATR) Indicator
    df.ta.natr(high='high', low='low', close='close', length=5, scalar=100, mamode='trima', append=True)  #done

    # Add Negative Volume Index (NVI) Indicator
#     df.ta.nvi(close='close', volume='volume', length=1, initial=100, append=True)  #pending

    # Add OHLC4 Indicator (Open, High, Low, Close Average)
    df.ta.ohlc4(open='open', high='high', low='low', close='close', offset=0, append=True) #done

    # Add Price Distance Indicator
    df.ta.pdist(open='open', high='high', low='low', close='close', drift=3, offset=0, append=True) #done

    # Add Percent Return Indicator
    df.ta.percent_return(close='close', length=27, append=True)  #done

    # Add Percentage Growth Oscillator (PGO) Indicator
    df.ta.pgo(high='high', low='low', close='close', length=26, append=True)  #done

    # Add Percentage Price Oscillator (PPO) Indicator
    df.ta.ppo(close='close', fast=9, slow=28, signal=24, mamode='ema', append=True) #done

    # Add Parabolic SAR Indicator
    df[['PSARaf_0.02_0.2', 'PSARr_0.02_0.2']] = df.ta.psar(af0=0.02, af=0.02, max_af=0.2)[['PSARaf_0.02_0.2', 'PSARr_0.02_0.2']]  #done

    # Add Price Strength Line (PSL) Indicator
    df.ta.psl(close='close', open='open', length=6, scalar=100, append=True)  #done

    # Add Price Volume Indicator (PVI) Indicator
#     df.ta.pvi(close='close', volume='volume', length=1, initial=1000, append=True)  #pending

    # Add Price Volume Oscillator (PVO) Indicator
    df.ta.pvo(volume='volume', fast=3, slow=29, signal=26, append=True)  #done

    # Add Price Volume On-Balance (PVOL) Indicator
    df.ta.pvol(close='close', volume='volume', append=True)  

    # Add Price Volume Ratio (PVR) Indicator
    df.ta.pvr(close='close', volume='volume', append=True) 

    # Add Price Volume Trend (PVT) Indicator
    df.ta.pvt(close='close', volume='volume', append=True) 

    # Add Price Weighted Moving Average (PWMA) Indicator
    df.ta.pwma(close='close', length=28, append=True)  # Short-term length #done

    # Add Quantitative Qualitative Estimation (QQE) Indicator
    df[['QQE_18_26_4.236', 'QQE_18_26_4.236_RSIMA']] = df.ta.qqe(length=18, smooth=26)[['QQE_18_26_4.236', 'QQE_18_26_4.236_RSIMA']]  #done

    # Add Qstick Indicator
    df.ta.qstick(open='open', close='close', length=25, append=True)  #done
    
    # Add Relative Strength Index (RSX) Indicator
    df.ta.rsx(close='close', length=25, append=True)  #done

    # Add Relative Vigor Index (RVGI) Indicator
    df.ta.rvgi(open='open', high='high', low='low', close='close', length=20, swma_length=11, append=True)  #done

    # Add Relative Vigor Index (RVI) Indicator
    df.ta.rvi(close='close', high='high', low='low', length=15, scalar=100, append=True) #done

    # Add Sine Weighted Moving Average (SINWMA) Indicator
    df.ta.sinwma(close='close', length=28, append=True)  #done

    # Add Skewness Indicator
    df.ta.skew(close='close', length=28, append=True)  #done

    # Add Slope Indicator
    df.ta.slope(close='close', length=25, append=True)  #done

    # Add Squeeze Indicator
#     df.ta.squeeze(high='high', low='low', close='close', bb_length=20, bb_std=2, kc_length=20, kc_scalar=1.5, mom_length=12, mom_smooth=12, tr=True, append=True)  #pending

    # Add Squeeze Pro Indicator
    df.ta.squeeze_pro(high='high', low='low', close='close', bb_length=9, kc_length=15, mom_length=30, mom_smooth=29, tr=True, mamode='swma', append=True)  #done

    # Add Smoothed Stochastic Forecast (SSF) Indicator
    df.ta.ssf(close='close', length=22, poles=2, append=True)  #done

    # Add Standard Deviation Indicator
    df.ta.stdev(close='close', length=27, ddof=5, talib=True, offset=0, append=True)  #done

    # Add Stochastic RSI Indicator
    df.ta.stochrsi(close='close', length=24, rsi_length=25, k=15, d=7, mamode='midpoint', append=True)  #done

    # Add Supertrend Indicator
    df[['SUPERT_13_3.0', 'SUPERTd_13_3.0']] = df.ta.supertrend(high='high', low='low', close='close', length=13, multiplier=3.0)[['SUPERT_13_3.0', 'SUPERTd_13_3.0']] #done

    # Add Smoothed Weighted Moving Average (SWMA) Indicator
    df.ta.swma(close='close', length=28, asc=True, append=True)  #done

    # Add T3 Indicator
    df.ta.t3(close='close', length=26, append=True)  #done

    # Add Triple Exponential Moving Average (TEMA) Indicator
    df.ta.tema(close='close', length=7, offset=0, append=True)  # Short-term length #done

    # Add Thermo Indicator
    df.ta.thermo(high='high',low= 'low', length=22, long=14, short=8, mamode='sinwma', drift=1, append=True)  #done

    # Add Triangular Moving Average (TRIMA) Indicator
    df.ta.trima(close='close', length=28, append=True) 

    # Add Triple Exponential Average (TRIX) Indicator
    df.ta.trix(close='close', length=27, signal=5, scalar=100, append=True)  #done

    # Add True Range Indicator
    df.ta.true_range(high='high', low='low', close='close', append=True)  #done

    # Add True Strength Index (TSI) Indicator
    df.ta.tsi(close='close', fast=27, slow=30, signal=27, mamode='rma', scalar=100, append=True)  #done

    # Add TT-Momentum Trend Indicator
    df.ta.ttm_trend(high='high', low='low', close='close', length=22, append=True)  #done

    # Add Ultimate Oscillator Indicator
    df.ta.uo(high='high', low='low', close='close', fast=19, medium=15, slow=15, append=True) #done

    # Add Vertical Horizontal Filter (VHF) Indicator
    df.ta.vhf(close='close', length=14, append=True)  #done

    # Add Variable Index Dynamic Average (VIDYA) Indicator
    df.ta.vidya(close='close', length=4, adjust=True, sma=False, append=True)  #done

    # Add Vortex Indicator
    df.ta.vortex(high='high', low='low', close='close', length=21, append=True)  # Short-term length #done

    # Add Volume Weighted Moving Average (VWMA) Indicator
    df.ta.vwma(close='close', volume='volume', length=22, append=True)  # Short-term length   #done

    # Add Weighted Close Price Indicator
    df.ta.wcp(high='high', low='low', close='close', append=True)  # Short-term length   #done

    # Add Williams %R Indicator
    df.ta.willr(high='high', low='low', close='close', length=27, append=True)  # Short-term length   #done

    # Add Weighted Moving Average (WMA) Indicator
#     df.ta.wma(close='close', length=10, append=True)  # Short-term length   #pending

    # Add Zero Lag Moving Average (ZLMA) Indicator
    df.ta.zlma(close='close', length=19, mamode='ema', append=True)  # Short-term length   #done

    # Add Z-Score Indicator
    df.ta.zscore(close='close', length=25, std=1, append=True)  # Short-term length   #done

    return df


In [6]:
grouped = filtered_stock_data.groupby('nse_symbol')

df_list = []

for symbol, group in grouped:
    print(f"Processing stock symbol: {symbol}")

    # Reset index and sort by date
    group = group.reset_index(drop=True)
    group = group.sort_values(by='date')
    
    # Add technical indicators
    group_with_indicators = add_technical_indicators(group)
    
    # Reset index and sort by date
    group_with_indicators = group_with_indicators.reset_index()
    
    # Save to parquet file
#     group_with_indicators.to_parquet(rf"D:\alpha_generator\alpha_short_calls\all_stocks_features\{symbol}_technical_indicators.parquet")
    
    # Append to list
    df_list.append(group_with_indicators)

# Concatenate all DataFrames and reset index
final_df = pd.concat(df_list, ignore_index=True)

Processing stock symbol: 360ONE
Processing stock symbol: 3MINDIA
Processing stock symbol: AARTIIND
Processing stock symbol: AAVAS
Processing stock symbol: ABB
Processing stock symbol: ABBOTINDIA
Processing stock symbol: ABCAPITAL
Processing stock symbol: ABFRL
Processing stock symbol: ACC
Processing stock symbol: ACE
Processing stock symbol: ACI
Processing stock symbol: ADANIENT
Processing stock symbol: ADANIGREEN
Processing stock symbol: ADANIPORTS
Processing stock symbol: ADANIPOWER
Processing stock symbol: AETHER
Processing stock symbol: AFFLE
Processing stock symbol: AIAENG
Processing stock symbol: AJANTPHARM
Processing stock symbol: ALKEM
Processing stock symbol: ALKYLAMINE
Processing stock symbol: ALLCARGO
Processing stock symbol: AMBER
Processing stock symbol: AMBUJACEM
Processing stock symbol: ANANDRATHI
Processing stock symbol: ANGELONE
Processing stock symbol: ANURAS
Processing stock symbol: APARINDS
Processing stock symbol: APLAPOLLO
Processing stock symbol: APLLTD
Processin

Processing stock symbol: JSL
Processing stock symbol: JSWENERGY
Processing stock symbol: JSWINFRA
Processing stock symbol: JSWSTEEL
Processing stock symbol: JUBLFOOD
Processing stock symbol: JUBLINGREA
Processing stock symbol: JUBLPHARMA
Processing stock symbol: JUSTDIAL
Processing stock symbol: JWL
Processing stock symbol: JYOTHYLAB
Processing stock symbol: KAJARIACER
Processing stock symbol: KALYANKJIL
Processing stock symbol: KANSAINER
Processing stock symbol: KARURVYSYA
Processing stock symbol: KAYNES
Processing stock symbol: KEC
Processing stock symbol: KEI
Processing stock symbol: KFINTECH
Processing stock symbol: KIMS
Processing stock symbol: KNRCON
Processing stock symbol: KOTAKBANK
Processing stock symbol: KPITTECH
Processing stock symbol: KPRMILL
Processing stock symbol: KRBL
Processing stock symbol: KSB
Processing stock symbol: LALPATHLAB
Processing stock symbol: LATENTVIEW
Processing stock symbol: LAURUSLABS
Processing stock symbol: LEMONTREE
Processing stock symbol: LICHSG

In [7]:
# final_df = final_df.reset_index(drop=True)

# final_df.to_parquet(r"D:\alpha_generator\alpha_short_calls\all_stocks_features\all_stocks_with_technical_indicators.parquet")

In [8]:
def getRSI(df: pd.DataFrame, timeperiod: int = 22) -> pd.DataFrame:
    """
    Calculates RSI (Relative Strength Index) features.

    Params:
    df : pd.DataFrame
        Input DataFrame containing stock data with 'close' column.
    timeperiod : int, default = 14
        Time periods to consider for RSI calculation.
      
    Returns:
    rsi_df : pd.DataFrame
        DataFrame containing RSI values.
    """
    res_df = df.copy()
    # Calculate RSI using 'adj_close' column
    res_df[f'RSI_{timeperiod}'] = ta.RSI(res_df["adj_close"], timeperiod)
    res_df[f'1w_avg_RSI_{timeperiod}'] = res_df[f'RSI_{timeperiod}'].rolling(window=5).mean()
    res_df[f'1m_avg_RSI_{timeperiod}'] = res_df[f'RSI_{timeperiod}'].rolling(window=21).mean()
    res_df[f'1w_slope_RSI_{timeperiod}'] = res_df[f'RSI_{timeperiod}'].diff(periods=5)
    res_df[f'1m_slope_RSI_{timeperiod}'] = res_df[f'RSI_{timeperiod}'].diff(periods=21)

    overbought_threshold = 70
    oversold_threshold = 30

    res_df['RSI_Overbought'] = (res_df[f'RSI_{timeperiod}'] > overbought_threshold).astype(int)
    res_df['RSI_Oversold'] = (res_df[f'RSI_{timeperiod}'] < oversold_threshold).astype(int)

    consecutive_days_threshold = 5
    res_df['RSI_Above_70_Consecutive'] = res_df['RSI_Overbought'].rolling(window=consecutive_days_threshold).sum()
    res_df['RSI_Below_30_Consecutive'] = res_df['RSI_Oversold'].rolling(window=consecutive_days_threshold).sum()
    res_df['RSI_ROC'] = res_df[f'RSI_{timeperiod}'].diff()
    rsi_band_width = 10  # Adjust the band width as needed
    res_df['RSI_Upper_Band'] = res_df[f'RSI_{timeperiod}'] + (rsi_band_width / 2)
    res_df['RSI_Lower_Band'] = res_df[f'RSI_{timeperiod}'] - (rsi_band_width / 2)

    # Create DataFrame with RSI values and divergence features
    rsi_df = pd.DataFrame({
            f"RSI_{timeperiod}": res_df[f'RSI_{timeperiod}'].values,
            f'1w_avg_RSI_{timeperiod}': res_df[f'1w_avg_RSI_{timeperiod}'].values,
            f'1m_avg_RSI_{timeperiod}': res_df[f'1m_avg_RSI_{timeperiod}'].values,
            f'1w_slope_RSI_{timeperiod}': res_df[f'1w_slope_RSI_{timeperiod}'].values,
            f'1m_slope_RSI_{timeperiod}': res_df[f'1m_slope_RSI_{timeperiod}'].values,

            'RSI_Overbought': res_df['RSI_Overbought'].values,
            'RSI_Oversold': res_df['RSI_Oversold'].values,
            f'RSI_Above_70_{consecutive_days_threshold}D_Consecutive': res_df['RSI_Above_70_Consecutive'].values,
            f'RSI_Below_30_{consecutive_days_threshold}D_Consecutive': res_df['RSI_Below_30_Consecutive'].values,
            'RSI_ROC': res_df['RSI_ROC'].values,
            'RSI_Upper_Band': res_df['RSI_Upper_Band'].values,
            'RSI_Lower_Band': res_df['RSI_Lower_Band'].values,
        }, index=res_df.index)

    return rsi_df


def getSMA(df: pd.DataFrame, window1: int = 13, window2: int = 42) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates simple moving average indicator with features

    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    window1 : int, default = 7
        Time periods (in rows) to consider while calculating the first SMA values
    window2 : int, default = 21
        Time periods (in rows) to consider while calculating the second SMA values
    
    Returns:
    sma_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate SMAs
    sma_short = ta.SMA(df["adj_close"], timeperiod=window1)
    sma_long = ta.SMA(df["adj_close"], timeperiod=window2)
    
    # Calculate divergence and convergence
    div_sma = ((sma_short < sma_long) & (sma_short.shift(1) > sma_long.shift(1))).astype(int)
    conv_sma = ((sma_short > sma_long) & (sma_short.shift(1) < sma_long.shift(1))).astype(int)
    
    # Difference between sma_short and sma_long
    diff_sma_short_long = sma_short - sma_long
    
    # Calculate ratios
    ratio_short = df["adj_close"] / sma_short
    ratio_long = df["adj_close"] / sma_long
    
    # Calculate standard deviations
    std_close7 = df["adj_close"].rolling(window=window1).std()
    std_close21 = df["adj_close"].rolling(window=window2).std()


    # Calculate the coefficient of variance
    ratio_cv_sma_std_close7 = sma_short / std_close7
    ratio_cv_sma_std_close21 = sma_long / std_close21
    
    
    # Calculate days since last crossover
    days_since_div = div_sma.groupby((div_sma != div_sma.shift()).cumsum()).cumcount() + 1
    days_since_conv = conv_sma.groupby((conv_sma != conv_sma.shift()).cumsum()).cumcount() + 1

    # Reset count to 0 when divergence or convergence occurs
    days_since_div = days_since_div.where(div_sma == 0, 0)
    days_since_conv = days_since_conv.where(conv_sma == 0, 0)
    
    # Create DataFrame with features
    sma_df = pd.DataFrame({
        f"divergence_SMA_{window1}_{window2}": div_sma,
        f"convergence_SMA_{window1}_{window2}": conv_sma,
        f"ratio_adj_close_SMA{window1}": ratio_short,
        f"cv_SMA_{window1}": ratio_cv_sma_std_close7,
        f"diff_SMA_{window1}_{window2}": diff_sma_short_long,
        f"days_since_divergence_SMA_{window1}_{window2}": days_since_div,
        f"days_since_convergence_SMA_{window1}_{window2}": days_since_conv,
        f"ratio_adj_close_SMA_{window2}": ratio_long,
        f"cv_SMA_{window2}": ratio_cv_sma_std_close21
    }, index=df.index)
    
    return sma_df


def getEMA(df: pd.DataFrame, window1: int = 8, window2: int = 42) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates exponential moving average indicator with features

    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    window1 : int, default = 7
        Time periods (in rows) to consider while calculating the first EMA values
    window2 : int, default = 21
        Time periods (in rows) to consider while calculating the second EMA values
    
    Returns:
    ema_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate EMAs
    ema_short = ta.EMA(df["adj_close"], timeperiod=window1)
    ema_long = ta.EMA(df["adj_close"], timeperiod=window2)
    
    # Calculate divergence and convergence
    div_ema = ((ema_short < ema_long) & (ema_short.shift(1) > ema_long.shift(1))).astype(int)
    conv_ema = ((ema_short > ema_long) & (ema_short.shift(1) < ema_long.shift(1))).astype(int)
    
    # Difference between ema_short and ema_long
    diff_ema_short_long = ema_short - ema_long
    diff_ema_short_long_pct = ((ema_short - ema_long)/ema_long) * 100
    
    # Calculate ratios
    ratio_short = df["adj_close"] / ema_short
    ratio_long = df["adj_close"] / ema_long
    
    # Calculate standard deviations
    std_close7 = df["adj_close"].ewm(span=window1).std()
    std_close21 = df["adj_close"].ewm(span=window2).std()

    # Calculate the ratios EMA7/STD(Close7)
    ratio_cv_ema_std_close7 =  (std_close7/ema_short)*100
    ratio_cv_ema_std_close21 = (std_close21/ema_long)*100 
    
    # Calculate days since last crossover
    days_since_div = div_ema.groupby((div_ema != div_ema.shift()).cumsum()).cumcount() + 1
    days_since_conv = conv_ema.groupby((conv_ema != conv_ema.shift()).cumsum()).cumcount() + 1

    # Reset count to 0 when divergence or convergence occurs
    days_since_div = days_since_div.where(div_ema == 0, 0)
    days_since_conv = days_since_conv.where(conv_ema == 0, 0)
    
    # Create DataFrame with features
    ema_df = pd.DataFrame({
        f"divergence_EMA_{window1}_{window2}": div_ema,
        f"convergence_EMA_{window1}_{window2}": conv_ema,
        f"ratio_EMA_{window1}": ratio_short,
        f"cv_EMA_{window1}": ratio_cv_ema_std_close7,
        f"diff_ema_{window1}_{window2}": diff_ema_short_long,
        f"diff_ema_{window1}_{window2}_pct": diff_ema_short_long_pct,
        f"days_since_divergence_EMA_{window1}_{window2}": days_since_div,
        f"days_since_convergence_EMA_{window1}_{window2}": days_since_conv,
        f"ratio_EMA_{window2}": ratio_long,
        f"cv_EMA_{window2}": ratio_cv_ema_std_close21
    }, index=df.index)
    
    return ema_df


def getDEMA(df: pd.DataFrame, window1: int = 18, window2: int = 38) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates double exponential moving average indicator with features

    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    window1 : int, default = 7
        Time periods (in rows) to consider while calculating the first DEMA values
    window2 : int, default = 21
        Time periods (in rows) to consider while calculating the second DEMA values
    
    Returns:
    dema_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate DEMAs
    dema_short = ta.DEMA(df["adj_close"], timeperiod=window1)
    dema_long = ta.DEMA(df["adj_close"], timeperiod=window2)
    
    # Calculate divergence and convergence
    div_dema = ((dema_short < dema_long) & (dema_short.shift(1) > dema_long.shift(1))).astype(int)
    conv_dema = ((dema_short > dema_long) & (dema_short.shift(1) < dema_long.shift(1))).astype(int)
    
    # Difference between dema_short and dema_long
    diff_dema_short_long = dema_short - dema_long
    
    # Calculate ratios
    ratio_short = df["adj_close"] / dema_short
    ratio_long = df["adj_close"] / dema_long
    
    # Calculate standard deviations
    std_close7 = df["adj_close"].ewm(span=window1).std()
    std_close21 = df["adj_close"].ewm(span=window2).std()

    # Calculate the coefficient of variance
    ratio_cv_dema_std_close7 =  (std_close7/dema_short)*100
    ratio_cv_dema_std_close21 = (std_close21/dema_long)*100 
    
    # Calculate days since last crossover
    days_since_div = div_dema.groupby((div_dema != div_dema.shift()).cumsum()).cumcount() + 1
    days_since_conv = conv_dema.groupby((conv_dema != conv_dema.shift()).cumsum()).cumcount() + 1

    # Reset count to 0 when divergence or convergence occurs
    days_since_div = days_since_div.where(div_dema == 0, 0)
    days_since_conv = days_since_conv.where(conv_dema == 0, 0)
    
    # Create DataFrame with features
    dema_df = pd.DataFrame({
        f"divergence_DEMA_{window1}_{window2}": div_dema,
        f"convergence_DEMA_{window1}_{window2}": conv_dema,
        f"ratio_DEMA_{window1}": ratio_short,
        f"cv_DEMA_{window1}": ratio_cv_dema_std_close7,
        f"diff_DEMA_{window1}_{window2}": diff_dema_short_long,
        f"days_since_divergence_DEMA_{window1}_{window2}": days_since_div,
        f"days_since_convergence_DEMA_{window1}_{window2}": days_since_conv,
        f"ratio_DEMA_{window2}": ratio_long,
        f"cv_DEMA_{window2}": ratio_cv_dema_std_close21
    }, index=df.index)
    
    return dema_df


def getBBANDS(df: pd.DataFrame, bbands_window: int = 19) -> pd.DataFrame:
    """
    MOMENTUM INDICATOR
    Calculates Bollinger Bands ranging bands indicator with features
    
    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    bbands_window : int, default = 20
        Time periods (in rows) to consider while calculating Bollinger Bands
        
    Returns:
    bbands_df : pd.DataFrame
        DataFrame with calculated features
    """
    res_df = df.copy()
    # Calculate Bollinger Bands
    res_df['upper_band'], res_df['mid_band'], res_df['low_band'] = ta.BBANDS(res_df["adj_close"], timeperiod=bbands_window)
    
    # Calculate deviation from upper band as a percentage
    bb_deviation = ((res_df["adj_close"] - res_df['upper_band']) / res_df['upper_band']) * 100

    res_df['price_within_BBANDS'] = ((res_df['adj_close'] >= res_df['low_band']) & (res_df['adj_close'] <= res_df['upper_band'])).astype(int)
    res_df['pct_width_of_BBANDS'] = ((res_df['upper_band'] - res_df['low_band']) / res_df['mid_band'] * 100)
    BBANDS_SQUEEZE_THRESH = 2.0
    res_df['squeeze_BBANDS'] = (res_df['pct_width_of_BBANDS'] < BBANDS_SQUEEZE_THRESH).astype(int)
    res_df['expansion_BBANDS'] = (res_df['pct_width_of_BBANDS'] >= BBANDS_SQUEEZE_THRESH).astype(int)
    res_df['price_rel_BBANDS'] = (res_df['adj_close'] - res_df['low_band']) / (res_df['upper_band'] - res_df['low_band'])
    res_df['above_ub_BBANDS'] = (res_df['adj_close'] > res_df['upper_band']).astype(int)
    res_df['below_lb_BBANDS'] = (res_df['adj_close'] < res_df['low_band']).astype(int)
    res_df['dist_frm_ub_BBANDS'] = res_df['adj_close'] - res_df['upper_band']
    res_df['dist_frm_ub_BBANDS_pct'] = (res_df['adj_close'] - res_df['upper_band'])/res_df['upper_band'] * 100
    res_df['dist_frm_lb_BBANDS'] = res_df['adj_close'] - res_df['low_band']
    res_df['dist_frm_lb_BBANDS_pct'] = (res_df['adj_close'] - res_df['low_band'])/res_df['low_band'] * 100
    res_df['1w_slope_ub_BBANDS'] = res_df['upper_band'].diff(periods=5)
    res_df['1m_slope_ub_BBANDS'] = res_df['upper_band'].diff(periods=21)

    # Create DataFrame with deviation feature
    bbands_df = pd.DataFrame({
        f"BB_deviation_upper_band_{bbands_window}": bb_deviation,
        'price_within_BBANDS': res_df['price_within_BBANDS'].values,
        'squeeze_BBANDS': res_df['squeeze_BBANDS'].values,
        'expansion_BBANDS': res_df['expansion_BBANDS'].values,
        'price_rel_BBANDS': res_df['price_rel_BBANDS'].values,
        'above_ub_BBANDS': res_df['above_ub_BBANDS'].values,
        'below_lb_BBANDS': res_df['below_lb_BBANDS'].values,
        'dist_frm_ub_BBANDS': res_df['dist_frm_ub_BBANDS'].values,
        'dist_frm_ub_BBANDS_pct':res_df['dist_frm_ub_BBANDS_pct'].values,
        'dist_frm_lb_BBANDS': res_df['dist_frm_lb_BBANDS'].values,
        'dist_frm_lb_BBANDS_pct':res_df['dist_frm_lb_BBANDS_pct'].values,
        '1w_slope_ub_BBANDS': res_df['1w_slope_ub_BBANDS'].values,
        '1m_slope_ub_BBANDS': res_df['1m_slope_ub_BBANDS'].values,
    }, index=res_df.index)
    
    return bbands_df

def getCCI(df: pd.DataFrame, cci_window: int = 25) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates commodity channel index indicator with features
    
    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    cci_window : int, default = 14
        Time periods (in rows) to consider while calculating CCI values
        
    Returns:
    cci_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate CCI
    res_df = df.copy()
    res_df['cci'] = ta.CCI(res_df["high"], res_df["low"], res_df["adj_close"], timeperiod=cci_window)
    overbought_threshold = 100
    oversold_threshold = -100
    res_df['CCI_Overbought'] = (res_df['cci'] > overbought_threshold).astype(int)
    res_df['CCI_Oversold'] = (res_df['cci'] < oversold_threshold).astype(int)
    res_df['CCI_ROC'] = res_df['cci'].diff()
    cci_slope_window = 5
    res_df['CCI_1w_slope'] = res_df['cci'].diff(periods=cci_slope_window)
    res_df['Bullish_Trend_CCI'] = (res_df['cci'] > 0).astype(int)
    res_df['Bearish_Trend_CCI'] = (res_df['cci'] < 0).astype(int)
    cci_range_window = 14
    res_df['CCI_Range'] = res_df['cci'].rolling(window=cci_range_window).max() - res_df['cci'].rolling(window=cci_range_window).min()

    # Create DataFrame with features
    cci_df = pd.DataFrame({
        f"{cci_window}_CCI": res_df['cci'].values,
        'CCI_Overbought': res_df['CCI_Overbought'].values,
        'CCI_Oversold': res_df['CCI_Oversold'].values,
        'CCI_ROC': res_df['CCI_ROC'].values,
        'CCI_1w_slope': res_df['CCI_1w_slope'].values,
        'Bullish_Trend_CCI': res_df['Bullish_Trend_CCI'].values,
        'Bearish_Trend_CCI': res_df['Bearish_Trend_CCI'].values,
        'CCI_Range': res_df['CCI_Range'].values,
    }, index=res_df.index)
    
    return cci_df


def getMACD(df: pd.DataFrame, ema_fast: int = 21, ema_slow: int = 28, signal_period: int = 8) -> pd.DataFrame:
    """
    MOMENTUM INDICATOR
    Calculates moving average convergence and divergence with features
    
    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    ema_fast : int, default = 12
        Fast EMA lookback period to calculate
    ema_slow : int, default = 26
        Slow EMA lookback period to calculate
    signal_period : int, default = 9
        EMA lookback period of the MACD line
    
    Returns:
    macd_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate MACD, Signal, and Histogram
    res_df = df.copy()
    res_df['macd_line'], res_df['signal_line'], res_df['histogram'] = ta.MACD(res_df["adj_close"], fastperiod=ema_fast, slowperiod=ema_slow, signalperiod=signal_period)
    
    # Calculate MACD convergence and divergence
    convergence_macd = ((res_df['macd_line'] > res_df['signal_line']) & (res_df['macd_line'].shift(1) < res_df['signal_line'].shift(1))).astype(int)
    divergence_macd = ((res_df['macd_line'] < res_df['signal_line']) & (res_df['macd_line'].shift(1) > res_df['signal_line'].shift(1))).astype(int)
    
    res_df['MACD_Above_Signal'] = (res_df['macd_line'] > res_df['signal_line']).astype(int)
    res_df['MACD_Below_Signal'] = (res_df['macd_line'] < res_df['signal_line']).astype(int)
    macd_histogram_window = 14  # Adjust the window size as needed
    res_df['MACD_Histogram_Volatility'] = res_df['histogram'].rolling(window=macd_histogram_window).std()
    bullish_threshold = 0.5  # Define your bullish threshold
    bearish_threshold = -0.5  # Define your bearish threshold
    res_df['MACD_Bullish_Zone'] = (res_df['histogram'] > bullish_threshold).astype(int)
    res_df['MACD_Bearish_Zone'] = (res_df['histogram'] < bearish_threshold).astype(int)
    res_df['MACD_ROC'] = res_df['macd_line'].diff()
    macd_slope_window = 5  # Adjust the window size as needed
    res_df['MACD_Slope'] = res_df['macd_line'].diff(periods=macd_slope_window)
    res_df['MACD_Cross_Above_0'] = (res_df['macd_line'] > 0).astype(int)
    res_df['MACD_Cross_Below_0'] = (res_df['macd_line'] < 0).astype(int)
    res_df['MACD_Absolute'] = res_df['macd_line'].abs()

    # Create DataFrame with calculated features
    macd_df = pd.DataFrame({
        f"MACD_histogram_{ema_fast}_{ema_slow}": res_df['histogram'].values,
        f"convergence_MACD_{ema_fast}_{ema_slow}": convergence_macd,
        f"divergence_MACD_{ema_fast}_{ema_slow}": divergence_macd,
        'MACD_Above_Signal': res_df['MACD_Above_Signal'].values,
        'MACD_Below_Signal': res_df['MACD_Below_Signal'].values,
        'MACD_Histogram_Volatility': res_df['MACD_Histogram_Volatility'].values,
        'MACD_Bullish_Zone': res_df['MACD_Bullish_Zone'].values,
        'MACD_Bearish_Zone': res_df['MACD_Bearish_Zone'].values,
        'MACD_ROC': res_df['MACD_ROC'].values,
        'MACD_Slope': res_df['MACD_Slope'].values,
        'MACD_Cross_Above_0': res_df['MACD_Cross_Above_0'].values,
        'MACD_Cross_Below_0': res_df['MACD_Cross_Below_0'].values,
        'MACD_Absolute': res_df['MACD_Absolute'].values,
    }, index=res_df.index)
    
    return macd_df


def getMFI(df: pd.DataFrame, timeperiod: int = 25, pct_change_period: int = 5) -> pd.DataFrame:
    """
    MOMENTUM INDICATOR
    Adds MFI indicator based on index
    
    Params:
    df : pd.DataFrame
        Dataset in form of dataframe
    timeperiod : int, default = 14 rows
        Lookback rows to calculate MFI values
    pct_change_period : int, default = 5 rows
        Rows to consider when calculating percentage change in MFI values
        
    Returns:
    mfi_df : pd.DataFrame
        DataFrame with calculated features
    """
    res_df = df.copy()
    # Calculate Money Flow Index (MFI)
    res_df['mfi'] = ta.MFI(res_df['high'], res_df['low'], res_df['adj_close'], res_df['volume'], timeperiod=timeperiod)
    overbought_threshold = 70
    oversold_threshold = 30
    res_df['MFI_Overbought'] = (res_df['mfi'] > overbought_threshold).astype(int)
    res_df['MFI_Oversold'] = (res_df['mfi'] < oversold_threshold).astype(int)

    res_df['1w_slope_MFI'] = res_df['mfi'].diff(5)
    res_df['1m_slope_MFI'] = res_df['mfi'].diff(21)

    res_df['divergence_MFI'] = np.where((res_df['adj_close'].diff() > 0) & (res_df['mfi'].diff() < 0), 1, 0)        
    
    # Create DataFrame with features
    mfi_df = pd.DataFrame({
        f"mfi{timeperiod}_MFI": res_df['mfi'].values,
        'MFI_Overbought': res_df['MFI_Overbought'].values,
        'MFI_Oversold': res_df['MFI_Oversold'].values,
        '1w_slope_MFI': res_df['1w_slope_MFI'].values,
        '1m_slope_MFI': res_df['1m_slope_MFI'].values,
        'divergence_MFI': res_df['divergence_MFI'].values
    }, index=res_df.index)

    return mfi_df


def getROC(df: pd.DataFrame, timeperiod: int = 27) -> pd.DataFrame:
    """
    MOMENTUM INDICATOR
    Calculates Rate of Change (ROC) indicator with features including divergence and percentage change.

    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe.
    timeperiod : int, default = 10
        Lookback period to calculate ROC values.
    pct_change_period : int, default = 5
        Lookback period for calculating percentage change.

    Returns:
    roc_df : pd.DataFrame
        DataFrame with calculated features.
    """

    # Initialize DataFrame
    roc_df = pd.DataFrame(index=df.index)

    # Calculate Rate of Change (ROC)
    roc = ta.ROC(df["adj_close"], timeperiod=timeperiod)
    # Create DataFrame with features
    roc_df[f"{timeperiod}_ROC"] = roc
    
    return roc_df


def getSTOCH(df, period_k=14, period_d=3):
    """
    Calculate the stochastic oscillator for a given DataFrame.

    Parameters:
    - df (DataFrame): Input DataFrame containing OHLC (Open, High, low, adj_close) data.
    - period_k (int): Number of periods to consider for %K calculation (default is 14).
    - period_d (int): Number of periods to consider for %D calculation (default is 3).

    Returns:
    - DataFrame: DataFrame containing the calculated stochastic oscillator values (%K, %D, %k_%d_divergence).
    """
    stoch_df = df.copy()
    # Adds a "n_high" column with max value of previous k periods
    stoch_df['n_high'] = stoch_df['high'].rolling(period_k).max()
    
    # Adds an "n_low" column with min value of previous K periods
    stoch_df['n_low'] = stoch_df['low'].rolling(period_k).min()
    
    # Uses the min/max values to calculate the %k (as a percentage)
    stoch_df['%K_STOCH'] = (stoch_df['adj_close'] - stoch_df['n_low']) * 100 / (stoch_df['n_high'] - stoch_df['n_low'])
    
    # Uses the %k to calculates a SMA over the past period_d values of %k
    stoch_df['%D_STOCH'] = stoch_df['%K_STOCH'].rolling(period_d).mean()
    
    stoch_df['%K_%D_diff_STOCH'] = stoch_df['%K_STOCH'] - stoch_df['%D_STOCH']

    overbought_threshold = 80  # Define your overbought threshold
    oversold_threshold = 20  # Define your oversold threshold
    stoch_df['STOCH_Overbought_Cross'] = (stoch_df['%K_STOCH'] > overbought_threshold).astype(int)
    stoch_df['STOCH_Oversold_Cross'] = (stoch_df['%K_STOCH'] < oversold_threshold).astype(int)

    # Calculate Stochastic Oscillator Rate of Change
    stoch_df['Stoch_ROC'] = stoch_df['%K_STOCH'].diff()

    # Calculate Stochastic Oscillator Slope over a specified period
    stoch_slope_window = 5  # Adjust the window size as needed
    stoch_df['Stoch_Slope'] = stoch_df['%K_STOCH'].diff(periods=stoch_slope_window)

    # Create binary features for Stochastic Oscillator trend
    stoch_df['STOCH_Bullish_Trend'] = (stoch_df['%K_STOCH'] > stoch_df['%D_STOCH']).astype(int)
    stoch_df['STOCH_Bearish_Trend'] = (stoch_df['%K_STOCH'] < stoch_df['%D_STOCH']).astype(int)

    # Calculate Stochastic Oscillator Range
    stoch_osc_range_window = 14  # Define your range window
    stoch_df['Stoch_OSC_Range'] = stoch_df['%K_STOCH'].rolling(window=stoch_osc_range_window).max() - stoch_df['%K_STOCH'].rolling(window=stoch_osc_range_window).min()

    stoch_df = stoch_df[['%K_STOCH','%D_STOCH','%K_%D_diff_STOCH', 'STOCH_Overbought_Cross', 'STOCH_Oversold_Cross', 'Stoch_ROC', 'Stoch_Slope',
                         'STOCH_Bullish_Trend', 'STOCH_Bearish_Trend', 'Stoch_OSC_Range']]
    
    return stoch_df


def getVWAP(df: pd.DataFrame, window: int = 7) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates Volume Weighted Average Price (VWAP) with features
    
    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    window : int, default = 14
        Time period in rows to consider while calculating VWAP
        
    Returns:
    vwap_df : pd.DataFrame
        DataFrame with calculated features
    """
    vwap_df = pd.DataFrame(index=df.index)
    
    # Calculate typical price
    typical_price = (df['high'] + df['low'] + df['adj_close']) / 3
    
    # Calculate VWAP using typical price
    vwap = (typical_price * df['volume']).rolling(window=window).sum() / df['volume'].rolling(window=window).sum()
    
    # Calculate deviation from VWAP
    vwap_deviation = df['adj_close'] - vwap

    
    # Create DataFrame with features
    vwap_df[f'VWAP_Deviation_{window}'] = vwap_deviation
    
    return vwap_df


def getOBV(df, window_OBV=5):
    """
    Create features related to the On-Balance Volume (OBV) indicator.

    Parameters:
    df (pd.DataFrame): DataFrame containing OHLCV data.
    window_OBV (int): Rolling window period for calculating the standard deviation of OBV. Default is 5.

    Returns:
    pd.DataFrame: DataFrame with OBV standard deviation, slope, volume ratio, breakout, and momentum features added.
    """
    obv_df = df.copy()

    # Calculate OBV
    obv_df[f'OBV_{window_OBV}'] = ta.OBV(obv_df['adj_close'], obv_df['volume'])

    # Log transformation of OBV handling positive, negative, and zero values
    obv_df[f'log_OBV_{window_OBV}'] = np.sign(obv_df[f'OBV_{window_OBV}']) * np.log(abs(obv_df[f'OBV_{window_OBV}']) + 1)
    obv_df['1w_slope_OBV'] = obv_df[f'OBV_{window_OBV}'].diff(5) / obv_df[f'OBV_{window_OBV}'].shift(5)
    obv_df['1m_slope_OBV'] = obv_df[f'OBV_{window_OBV}'].diff(21) / obv_df[f'OBV_{window_OBV}'].shift(21)
    obv_df['divergence_OBV'] = np.where((obv_df['adj_close'].diff() > 0) & (obv_df[f'OBV_{window_OBV}'].diff() < 0), 1, 0)
    # obv_df['cumsum_OBV'] = obv_df[f'OBV_{window_OBV}'].cumsum()
    obv_df['1w_sma_OBV'] = obv_df[f'OBV_{window_OBV}'].rolling(window=5).mean()
    obv_df['1w_ema_OBV'] = obv_df[f'OBV_{window_OBV}'].ewm(span=5, adjust=False).mean()

    obv_df = obv_df[[f'log_OBV_{window_OBV}', '1w_slope_OBV', '1m_slope_OBV', 'divergence_OBV', '1w_sma_OBV', '1w_ema_OBV']]

    return obv_df


def getADX_DMI(df, n_ADX=22):
    """
    Calculate ADX, DMI, and DMI Crossover.

    Parameters:
    df (pandas.DataFrame): DataFrame containing OHLCV data.
    n_ADX (int): Period of ADX. Default is 14.

    Returns:
    pandas.DataFrame: DataFrame with ADX, DMI, and DMI Crossover features.
    """
    adx_dmi_df = df.copy()
    # Calculate ADX
    adx_dmi_df[f'ADX_{n_ADX}'] = ta.ADX(adx_dmi_df['high'], adx_dmi_df['low'], adx_dmi_df['adj_close'], timeperiod=n_ADX)

    # Calculate DMI
    adx_dmi_df[f'ADX_plusDI_{n_ADX}'] = ta.PLUS_DI(adx_dmi_df['high'], adx_dmi_df['low'], adx_dmi_df['adj_close'], timeperiod=n_ADX)
    adx_dmi_df[f'ADX_minusDI_{n_ADX}'] = ta.MINUS_DI(adx_dmi_df['high'], adx_dmi_df['low'], adx_dmi_df['adj_close'], timeperiod=n_ADX)
    
    #Calculate difference in plusDI and minusDI
    adx_dmi_df[f'ADX_diff_plusDI_minusDI_{n_ADX}'] = adx_dmi_df[f'ADX_plusDI_{n_ADX}']-adx_dmi_df[f'ADX_minusDI_{n_ADX}']
    
    strong_trend_threshold = 25  # Define your strong trend threshold
    weak_trend_threshold = 15  # Define your weak trend threshold
    adx_dmi_df['ADX_Strong_Trend'] = (adx_dmi_df[f'ADX_{n_ADX}'] > strong_trend_threshold).astype(int)
    adx_dmi_df['ADX_Weak_Trend'] = (adx_dmi_df[f'ADX_{n_ADX}'] < weak_trend_threshold).astype(int)
    adx_dmi_df['ADX_ROC'] = adx_dmi_df[f'ADX_{n_ADX}'].diff()
    # adx_slope_window = 5  # Adjust the window size as needed
    # adx_dmi_df['ADX_Slope'] = adx_dmi_df[f'ADX_{n_ADX}'].diff(periods=adx_slope_window)
    adx_crossover_threshold = 20  # Define your ADX crossover threshold
    adx_dmi_df['ADX_Above_Threshold'] = (adx_dmi_df[f'ADX_{n_ADX}'] > adx_crossover_threshold).astype(int)
    adx_dmi_df['ADX_Below_Threshold'] = (adx_dmi_df[f'ADX_{n_ADX}'] < adx_crossover_threshold).astype(int)
    # adx_dmi_df['Price_vs_ADX_Divergence'] = adx_dmi_df['adj_close'] - adx_dmi_df[f'ADX_{n_ADX}']
    adx_upper_channel = strong_trend_threshold * 2  # Define your upper channel multiplier
    adx_lower_channel = strong_trend_threshold * 0.5  # Define your lower channel multiplier
    # adx_dmi_df['ADX_Upper_Channel'] = adx_dmi_df[f'ADX_{n_ADX}'] * adx_upper_channel
    # adx_dmi_df['ADX_Lower_Channel'] = adx_dmi_df[f'ADX_{n_ADX}'] * adx_lower_channel
    adx_dmi_df[f'1w_slope_ADX_{n_ADX}'] = adx_dmi_df[f'ADX_{n_ADX}'].diff(periods=5)
    adx_dmi_df[f'1m_slope_ADX_{n_ADX}'] = adx_dmi_df[f'ADX_{n_ADX}'].diff(periods=21)

    adx_dmi_df = adx_dmi_df[[f'ADX_{n_ADX}', f'ADX_plusDI_{n_ADX}', f'ADX_minusDI_{n_ADX}',f'ADX_diff_plusDI_minusDI_{n_ADX}',
                             'ADX_Strong_Trend', 'ADX_Weak_Trend', 'ADX_ROC', 'ADX_Above_Threshold', 'ADX_Below_Threshold', 
                             f'1w_slope_ADX_{n_ADX}', f'1m_slope_ADX_{n_ADX}']]
    return adx_dmi_df

def getBOP(df):
    """
    Calculate Balance of Power (BOP) indicator and create features.
    
    Parameters:
    bop_df (pandas.DataFrame): DataFrame containing OHLCV data.

    Returns:
    pandas.DataFrame: DataFrame with BOP features added.
    """
    bop_df = df.copy()
    # Calculate BOP value using talib
    bop_df['BOP'] = ta.BOP(bop_df['open'].values, bop_df['high'].values, bop_df['low'].values, bop_df['adj_close'].values)
    
    bop_df = bop_df[['BOP']]
    return bop_df


def getKAMA(df: pd.DataFrame, fast_period: int = 7, slow_period: int = 27) -> pd.DataFrame:
    """
    TREND INDICATOR
    Calculates Kaufman's Adaptive Moving Average (KAMA) indicator with features

    Params:
    df : pd.DataFrame
        Dataset in the form of a dataframe
    fast_period : int, default = 7
        Time periods (in rows) to consider while calculating the fast KAMA values
    slow_period : int, default = 21
        Time periods (in rows) to consider while calculating the slow KAMA values
    
    Returns:
    kama_df : pd.DataFrame
        DataFrame with calculated features
    """
    
    # Calculate KAMA
    kama_fast = ta.KAMA(df["adj_close"], timeperiod=fast_period)
    kama_slow = ta.KAMA(df["adj_close"], timeperiod=slow_period)
    
    # Calculate divergence and convergence
    div_kama = ((kama_fast > kama_slow) & (kama_fast.shift(1) < kama_slow.shift(1))).astype(int)
    conv_kama = ((kama_fast < kama_slow) & (kama_fast.shift(1) > kama_slow.shift(1))).astype(int)
    
    # Calculate days since last crossover
    days_since_div = div_kama.groupby((div_kama != div_kama.shift()).cumsum()).cumcount()+1
    days_since_conv = conv_kama.groupby((conv_kama != conv_kama.shift()).cumsum()).cumcount()+1

    # Reset count to 0 when divergence or convergence occurs
    days_since_div = days_since_div.where(div_kama == 0, 0)
    days_since_conv = days_since_conv.where(conv_kama == 0, 0)

    # Calculate the difference between slow and fast KAMA
    kama_diff = kama_fast - kama_slow

    # Calculate the difference between slow and fast KAMA in percentage
    kama_diff_pct = ((kama_fast - kama_slow)/kama_fast)*100
    
    # Create DataFrame with features
    kama_df = pd.DataFrame({
        f"KAMA_difference_{fast_period}_{slow_period}": kama_diff, 
        f"KAMA_difference_{fast_period}_{slow_period}_pct": kama_diff_pct,
        f"days_since_divergence_KAMA_{fast_period}_{slow_period}": days_since_div,
        f"days_since_convergence_KAMA_{fast_period}_{slow_period}": days_since_conv,
    }, index=df.index)
    
    return kama_df


def getPAR_SAR(df):
    """
    A function to create features of Parabolic SAR (Stop and Reverse) indicator.

    Input:
    par_sar_df (pandas.DataFrame): DataFrame containing OHLCV (Open, High, low, adj_close, Volume) data with symbols.

    Output:
    pandas.DataFrame: DataFrame with Parabolic SAR features added.
    """
    par_sar_df = df.copy()
    # Calculate Parabolic SAR values
    par_sar_df['Parabolic_SAR'] = ta.SAR(par_sar_df['high'], par_sar_df['low'])

    # Calculate Parabolic SAR values
    par_sar_df['diff_adj_close_SAR'] = par_sar_df['adj_close'] - par_sar_df['Parabolic_SAR']
    
    # Calculate percentage difference between Close and Parabolic SAR
    par_sar_df['diff_adj_close_SAR_pct'] = ((par_sar_df['adj_close'] - par_sar_df['Parabolic_SAR']) / par_sar_df['adj_close']) * 100

    par_sar_df = par_sar_df[['diff_adj_close_SAR','diff_adj_close_SAR_pct']]
    return par_sar_df

def getAroon(df, n_AROON= 27):
    """
    Calculate Aroon features including Indicator_Aroon, slope of Aroon-Up and Aroon-Down lines,
    and Aroon Oscillator.

    Parameters:
    aroon_df : pd.DataFrame
        DataFrame containing OHLCV data.
    n_AROON : int, optional
        Period of Aroon. Default is 14.

    Returns:
    pd.DataFrame
        DataFrame with Aroon features added.
    """
    aroon_df = df.copy()
    # Calculate Aroon-Up and Aroon-Down values
    aroon_up, aroon_down = ta.AROON(aroon_df['high'], aroon_df['low'], timeperiod=n_AROON)

    # Calculate Aroon Oscillator
    aroon_oscillator = aroon_up - aroon_down

    # Add features to DataFrame
    aroon_df[f'AROON_Oscillator{n_AROON}'] = aroon_oscillator
    aroon_df = aroon_df[[f'AROON_Oscillator{n_AROON}']]
    aroon_df[f'AROON_Oscillator{n_AROON}_5d_uptrend_count']= aroon_df[f'AROON_Oscillator{n_AROON}'].rolling(5).apply(lambda x:sum(x>0))
    aroon_df[f'AROON_Oscillator{n_AROON}_5d_downtrend_count']= aroon_df[f'AROON_Oscillator{n_AROON}'].rolling(5).apply(lambda x:sum(x<0))
    return aroon_df


#Candlestick patterns
def extract_candlestick_patterns(df):
    """
    Extract candlestick patterns from OHLC data.

    Parameters:
    data (pandas.DataFrame): DataFrame containing OHLC (open, High, low, Close) data.

    Returns:
    pandas.DataFrame: DataFrame with candlestick patterns as features.
    """
    # Calculate candlestick patterns
    candle_df = df.copy()
    candle_df['Doji'] = ta.CDLDOJI(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Engulfing'] = ta.CDLENGULFING(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Harami'] = ta.CDLHARAMI(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Hammer'] = ta.CDLHAMMER(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Inverted_Hammer'] = ta.CDLINVERTEDHAMMER(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Morning_Star'] = ta.CDLMORNINGSTAR(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'], penetration=0)
    candle_df['Evening_Star'] = ta.CDLEVENINGSTAR(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'], penetration=0)
    candle_df['Marubozu'] = ta.CDLMARUBOZU(candle_df['open'], candle_df['high'], candle_df['low'], candle_df['close'])
    candle_df['Inside_Bar_Pattern'] = (((candle_df['close'] > candle_df['high'].shift(1)) | (candle_df['high'] > candle_df['high'].shift(2))) & ((candle_df['high'].shift(1) < candle_df['high'].shift(2))
                & (candle_df['low'].shift(1) > candle_df['low'].shift(2)) & (candle_df['open'].shift(2) != candle_df['close'].shift(2)))).astype(int)

    candle_df[['Doji','Engulfing','Harami','Inverted_Hammer','Morning_Star','Evening_Star','Marubozu']] = candle_df[['Doji','Engulfing','Harami','Inverted_Hammer','Morning_Star','Evening_Star','Marubozu']].replace({0:0,100:1,-100:-1})
    candle_df = candle_df[['Doji','Engulfing','Harami','Inverted_Hammer','Morning_Star','Evening_Star','Marubozu','Inside_Bar_Pattern']]
    
    return candle_df

def eight_trigram(df):
    """
    Detect eight trigram patterns from OHLC data.

    Parameters:
    trigram_df (pandas.DataFrame): DataFrame containing OHLC (Open, High, low, Close) data.

    Returns:
    pandas.DataFrame: DataFrame with eight trigram patterns as features.
    """
    trigram_df = df.copy()

    # Calculate trigram patterns
    trigram_df['BullishHorn'] = ((trigram_df['high'] > trigram_df['high'].shift(1)) & 
                                 (trigram_df['low'] < trigram_df['low'].shift(1)) & 
                                 (trigram_df['close'] > trigram_df['close'].shift(1))).astype(int)

    trigram_df['BearHorn'] = ((trigram_df['high'] > trigram_df['high'].shift(1)) & 
                              (trigram_df['low'] < trigram_df['low'].shift(1)) & 
                              (trigram_df['close'] < trigram_df['close'].shift(1))).astype(int)

    trigram_df['BullishHigh'] = ((trigram_df['high'] > trigram_df['high'].shift(1)) & 
                                 (trigram_df['low'] > trigram_df['low'].shift(1)) & 
                                 (trigram_df['close'] > trigram_df['close'].shift(1))).astype(int)

    trigram_df['BearHigh'] = ((trigram_df['high'] > trigram_df['high'].shift(1)) & 
                              (trigram_df['low'] > trigram_df['low'].shift(1)) & 
                              (trigram_df['close'] < trigram_df['close'].shift(1))).astype(int)

    trigram_df['BullishLow'] = ((trigram_df['high'] < trigram_df['high'].shift(1)) & 
                                (trigram_df['low'] < trigram_df['low'].shift(1)) & 
                                (trigram_df['close'] > trigram_df['close'].shift(1))).astype(int)

    trigram_df['BearLow'] = ((trigram_df['high'] < trigram_df['high'].shift(1)) & 
                             (trigram_df['low'] < trigram_df['low'].shift(1)) & 
                             (trigram_df['close'] < trigram_df['close'].shift(1))).astype(int)
    
    trigram_df = trigram_df[['BullishHorn', 'BearHorn', 'BullishHigh', 'BearHigh', 'BullishLow', 'BearLow']]

    return trigram_df


def fibonacci_pivot_points(df, window=21):
    # DONE: Add % away from key-levels
    df = df.sort_index(ascending=True)
    df[f'PP_{window}'] = (df['high'].rolling(window=window).max() + 
                          df['low'].rolling(window=window).min() + 
                          df['adj_close'].rolling(window=window).mean()) / 3

    df[f'R1_{window}'] = df[f'PP_{window}'] + 0.382 * (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())
    df[f'R2_{window}'] = df[f'PP_{window}'] + 0.618 * (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())
    df[f'R3_{window}'] = df[f'PP_{window}'] + (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())
    df[f'S1_{window}'] = df[f'PP_{window}'] - 0.382 * (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())
    df[f'S2_{window}'] = df[f'PP_{window}'] - 0.618 * (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())
    df[f'S3_{window}'] = df[f'PP_{window}'] - (df['high'].rolling(window=window).max() - df['low'].rolling(window=window).min())

    df[f'pct_awy_PP_{window}'] = ((df['adj_close'] - df[f"PP_{window}"]) / df[f"PP_{window}"]) * 100
    df[f'pct_awy_R1_{window}'] = ((df['adj_close'] - df[f"R1_{window}"]) / df[f"R1_{window}"]) * 100
    df[f'pct_awy_R2_{window}'] = ((df['adj_close'] - df[f"R2_{window}"]) / df[f"R2_{window}"]) * 100
    df[f'pct_awy_R3_{window}'] = ((df['adj_close'] - df[f"R3_{window}"]) / df[f"R3_{window}"]) * 100
    df[f'pct_awy_S1_{window}'] = ((df['adj_close'] - df[f"S1_{window}"]) / df[f"S1_{window}"]) * 100
    df[f'pct_awy_S2_{window}'] = ((df['adj_close'] - df[f"S2_{window}"]) / df[f"S2_{window}"]) * 100
    df[f'pct_awy_S3_{window}'] = ((df['adj_close'] - df[f"S3_{window}"]) / df[f"S3_{window}"]) * 100

    df.drop([f'PP_{window}', f'R1_{window}', f'R2_{window}', f'R3_{window}', f'S1_{window}', f'S2_{window}', f'S3_{window}'], axis=1, inplace=True)

    return df


def get_simple_ratios(df):
    simple_ratios_df = df.copy()

    simple_ratios_df['date'] = pd.to_datetime(simple_ratios_df['date'])
    simple_ratios_df = simple_ratios_df.set_index('date')

    simple_ratios_df['52w_high'] = simple_ratios_df['high'].rolling(window='365D', min_periods=1).max()
    simple_ratios_df['6m_high'] = simple_ratios_df['high'].rolling(window='180D', min_periods=1).max()
    simple_ratios_df['3m_high'] = simple_ratios_df['high'].rolling(window='90D', min_periods=1).max()
    simple_ratios_df['1m_high'] = simple_ratios_df['high'].rolling(window='30D', min_periods=1).max()
    simple_ratios_df['1w_high'] = simple_ratios_df['high'].rolling(window='7D', min_periods=1).max()
    
    simple_ratios_df['52w_low'] = simple_ratios_df['low'].rolling(window='365D', min_periods=1).min()
    simple_ratios_df['6m_low'] = simple_ratios_df['low'].rolling(window='180D', min_periods=1).min()
    simple_ratios_df['3m_low'] = simple_ratios_df['low'].rolling(window='90D', min_periods=1).min()
    simple_ratios_df['1m_low'] = simple_ratios_df['low'].rolling(window='30D', min_periods=1).min()
    simple_ratios_df['1w_low'] = simple_ratios_df['low'].rolling(window='7D', min_periods=1).min()

    simple_ratios_df['pct_awy_52w_high'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['52w_high']) / simple_ratios_df['52w_high']) * 100)
    simple_ratios_df['pct_awy_6m_high'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['6m_high']) / simple_ratios_df['6m_high']) * 100)
    simple_ratios_df['pct_awy_3m_high'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['3m_high']) / simple_ratios_df['3m_high']) * 100)
    simple_ratios_df['pct_awy_1m_high'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['1m_high']) / simple_ratios_df['1m_high']) * 100)
    simple_ratios_df['pct_awy_1w_high'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['1w_high']) / simple_ratios_df['1w_high']) * 100)
    
    simple_ratios_df['pct_awy_52w_low'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['52w_low']) / simple_ratios_df['52w_low']) * 100)
    simple_ratios_df['pct_awy_6m_low'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['6m_low']) / simple_ratios_df['6m_low']) * 100)
    simple_ratios_df['pct_awy_3m_low'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['3m_low']) / simple_ratios_df['3m_low']) * 100)
    simple_ratios_df['pct_awy_1m_low'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['1m_low']) / simple_ratios_df['1m_low']) * 100)
    simple_ratios_df['pct_awy_1w_low'] = np.abs(((simple_ratios_df['adj_close'] - simple_ratios_df['1w_low']) / simple_ratios_df['1w_low']) * 100)

    simple_ratios_df['pct_gap_prc_diff'] = ((simple_ratios_df['open'] - simple_ratios_df['adj_close'].shift(1)) / simple_ratios_df['adj_close'].shift(1)) * 100

    simple_ratios_df['1d_gap_up'] = np.where(simple_ratios_df['pct_gap_prc_diff'] > 0, 1, 0)
    simple_ratios_df['1d_gap_down'] = np.where(simple_ratios_df['pct_gap_prc_diff'] < 0, 1, 0)
    simple_ratios_df['1m_gap_up_cnts'] = simple_ratios_df['1d_gap_up'].rolling(window='30D', min_periods=1).sum()
    simple_ratios_df['1m_gap_down_cnts'] = simple_ratios_df['1d_gap_down'].rolling(window='30D', min_periods=1).sum()
    simple_ratios_df['1w_gap_up_cnts'] = simple_ratios_df['1d_gap_up'].rolling(window='7D', min_periods=1).sum()
    simple_ratios_df['1w_gap_down_cnts'] = simple_ratios_df['1d_gap_down'].rolling(window='7D', min_periods=1).sum()

    simple_ratios_df['52w_vol'] = simple_ratios_df['adj_close'].rolling(window='365D', min_periods=1).std()
    simple_ratios_df['6m_vol'] = simple_ratios_df['adj_close'].rolling(window='180D', min_periods=1).std()
    simple_ratios_df['3m_vol'] = simple_ratios_df['adj_close'].rolling(window='90D', min_periods=1).std()
    simple_ratios_df['1m_vol'] = simple_ratios_df['adj_close'].rolling(window='30D', min_periods=1).std()
    simple_ratios_df['1w_vol'] = simple_ratios_df['adj_close'].rolling(window='7D', min_periods=1).std()

    simple_ratios_df['1w_vol_EMA_14'] = simple_ratios_df['1w_vol'].ewm(span=14, adjust=False).mean()
    simple_ratios_df['1m_vol_EMA_14'] = simple_ratios_df['1m_vol'].ewm(span=14, adjust=False).mean()
    simple_ratios_df['3m_vol_EMA_14'] = simple_ratios_df['3m_vol'].ewm(span=14, adjust=False).mean()
    simple_ratios_df['6m_vol_EMA_14'] = simple_ratios_df['6m_vol'].ewm(span=14, adjust=False).mean()
    simple_ratios_df['1y_vol_EMA_14'] = simple_ratios_df['52w_vol'].ewm(span=14, adjust=False).mean()
    
    simple_ratios_df['momentum_1w'] = simple_ratios_df['adj_close'] - simple_ratios_df['adj_close'].shift(5)
    simple_ratios_df['momentum_1m'] = simple_ratios_df['adj_close'] - simple_ratios_df['adj_close'].shift(21)
    # simple_ratios_df['momentum_3m'] = simple_ratios_df['adj_close'] - simple_ratios_df['adj_close'].shift(62)
    # simple_ratios_df['momentum_6m'] = simple_ratios_df['adj_close'] - simple_ratios_df['adj_close'].shift(124)
    # simple_ratios_df['momentum_1y'] = simple_ratios_df['adj_close'] - simple_ratios_df['adj_close'].shift(252)

    simple_ratios_df['daily_vol_chng_pct'] = (simple_ratios_df['volume'].pct_change(periods=1)) * 100
    simple_ratios_df['weekly_vol_chng_pct'] = (simple_ratios_df['volume'].pct_change(periods=5)) * 100
    simple_ratios_df['monthly_vol_chng_pct'] = (simple_ratios_df['volume'].pct_change(periods=21)) * 100

    simple_ratios_df['mf_volume'] = (((2 * simple_ratios_df['adj_close'] - simple_ratios_df['low'] - simple_ratios_df['high']) / (simple_ratios_df['high'] - simple_ratios_df['low'])).fillna(0)) * simple_ratios_df['volume']
    simple_ratios_df['CMF_1w'] = (simple_ratios_df['mf_volume'].rolling(window='7D', min_periods=1).sum() / simple_ratios_df['volume'].rolling(window='7D', min_periods=1).sum()).fillna(0)
    simple_ratios_df['CMF_1m'] = (simple_ratios_df['mf_volume'].rolling(window='30D', min_periods=1).sum() / simple_ratios_df['volume'].rolling(window='30D', min_periods=1).sum()).fillna(0)
    vol_1w_MA = simple_ratios_df['volume'].rolling(window='7D', min_periods=1).mean()
    vol_1m_MA = simple_ratios_df['volume'].rolling(window='30D', min_periods=1).mean()
    simple_ratios_df['volume_oscillator'] = vol_1w_MA - vol_1m_MA
    vol_1d_chng = simple_ratios_df['volume'].diff()
    simple_ratios_df['up_vol'] = np.where(vol_1d_chng > 0, vol_1d_chng, 0)
    simple_ratios_df['down_vol'] = np.where(vol_1d_chng < 0, -vol_1d_chng, 0)
    simple_ratios_df['avg_up_vol_1w'] = simple_ratios_df['up_vol'].rolling(window='7D', min_periods=1).mean()
    simple_ratios_df['avg_down_vol_1w'] = simple_ratios_df['down_vol'].rolling(window='7D', min_periods=1).mean()
    simple_ratios_df['vrs_1w'] = simple_ratios_df['avg_up_vol_1w'] / simple_ratios_df['avg_down_vol_1w']
    simple_ratios_df['VRSI_1w'] = 100 - (100 / (1 + simple_ratios_df['vrs_1w']))
    simple_ratios_df['avg_up_vol_1m'] = simple_ratios_df['up_vol'].rolling(window='30D', min_periods=1).mean()
    simple_ratios_df['avg_down_vol_1m'] = simple_ratios_df['down_vol'].rolling(window='30D', min_periods=1).mean()
    simple_ratios_df['vrs_1m'] = simple_ratios_df['avg_up_vol_1m'] / simple_ratios_df['avg_down_vol_1m']
    simple_ratios_df['VRSI_1m'] = 100 - (100 / (1 + simple_ratios_df['vrs_1m']))
    simple_ratios_df['VPT_1w'] = (simple_ratios_df['adj_close'] * simple_ratios_df['volume']).rolling(window='7D', min_periods=1).sum() / simple_ratios_df['volume'].rolling(window='7D', min_periods=1).sum()
    simple_ratios_df['VPT_1m'] = (simple_ratios_df['adj_close'] * simple_ratios_df['volume']).rolling(window='30D', min_periods=1).sum() / simple_ratios_df['volume'].rolling(window='30D', min_periods=1).sum()

    simple_ratios_df['hl_diff'] = simple_ratios_df['high'] - simple_ratios_df['low']
    simple_ratios_df['hprevc_diff'] = abs(simple_ratios_df['high'] - simple_ratios_df['adj_close'].shift(1))
    simple_ratios_df['lprevc_diff'] = abs(simple_ratios_df['low'] - simple_ratios_df['adj_close'].shift(1))
    simple_ratios_df['TR'] = simple_ratios_df[['hl_diff', 'hprevc_diff', 'lprevc_diff']].max(axis=1)

    simple_ratios_df['atr_1w'] = simple_ratios_df['high'].rolling(window='7D', min_periods=1).max() - simple_ratios_df['low'].rolling(window='7D', min_periods=1).min()
    simple_ratios_df['atr_1m'] = simple_ratios_df['high'].rolling(window='30D', min_periods=1).max() - simple_ratios_df['low'].rolling(window='30D', min_periods=1).min()
    simple_ratios_df['atr_ratio'] = simple_ratios_df['atr_1w'] / simple_ratios_df['atr_1m']
    simple_ratios_df['atr_oscillator'] = simple_ratios_df['atr_1w'] - simple_ratios_df['atr_1m']

    simple_ratios_df.drop(['52w_high', '6m_high', '3m_high', '1m_high', '1w_high', '52w_low', '6m_low', '3m_low', '1m_low', '1w_low', 
                           'hl_diff', 'hprevc_diff', 'lprevc_diff', 'up_vol', 'down_vol', 'avg_up_vol_1w', 'avg_down_vol_1w', 'vrs_1w',
                           'avg_up_vol_1m', 'avg_down_vol_1m', 'vrs_1m'], axis=1, inplace=True)

    simple_ratios_df = simple_ratios_df.reset_index()
    simple_ratios_df = simple_ratios_df[['pct_awy_52w_high', 'pct_awy_6m_high', 'pct_awy_3m_high', 'pct_awy_1m_high', 'pct_awy_1w_high', 'pct_awy_52w_low', 'pct_awy_6m_low', 
                                         'pct_awy_3m_low', 'pct_awy_1m_low', 'pct_awy_1w_low', '1d_gap_up', '1d_gap_down',
                                         '1m_gap_up_cnts', '1m_gap_down_cnts', '1w_gap_up_cnts', '1w_gap_down_cnts', '52w_vol', '6m_vol', '3m_vol',
                                         '1m_vol', '1w_vol','momentum_1w', 'momentum_1m',
                                         '1w_vol_EMA_14', '1m_vol_EMA_14', '3m_vol_EMA_14', '6m_vol_EMA_14', '1y_vol_EMA_14', 'daily_vol_chng_pct', 'weekly_vol_chng_pct', 
                                         'monthly_vol_chng_pct', 'CMF_1w', 'CMF_1m', 'volume_oscillator', 'VRSI_1w', 'VRSI_1m', 'VPT_1w', 'VPT_1m', 'TR', ]]
    
    return simple_ratios_df

In [9]:

def add_transform_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds all indicator features to the input DataFrame.
    
    Params:
    df : pd.DataFrame
        Input DataFrame containing stock data.
        
    Returns:
    df_with_indicators : pd.DataFrame
        DataFrame with added indicator features.
    """
    # Add RSI features
    rsi_df = getRSI(df)
    
    # Add SMA features
    sma_df = getSMA(df)
    sma_monthly_df = getSMA(df, window1=50, window2=100)
    
    # Add EMA features
    ema_df = getEMA(df)
    ema_monthly_df = getEMA(df, window1=50, window2=100)
    
    # Add DEMA features
    dema_df = getDEMA(df)
    dema_monthly_df = getDEMA(df, window1=50, window2=100)
    
    # Add BBANDS features
    bbands_df = getBBANDS(df)
    
    # Add CCI features
    cci_df = getCCI(df)
    
    # Add MACD features
    macd_df = getMACD(df)
    
    # Add MFI features
    mfi_df = getMFI(df)
    
    # Add ROC features
    roc_df = getROC(df)

    # Add Stoch features
    stoch_df = getSTOCH(df)

    # Add VWAP features
    vwap_df = getVWAP(df)

    # Add OBV features
    obv_df = getOBV(df)

    #Add ADX_DMI features
    adx_dmi_df = getADX_DMI(df)

    #Add BOP features
    bop_df = getBOP(df)

    #Add KAMA features
    kama_df = getKAMA(df)
    kama_monthly_df = getKAMA(df, fast_period=50, slow_period=100)

    #Add PAR SAR features
    par_sar_df = getPAR_SAR(df)

    #Add ARoon features
    aroon_df = getAroon(df)
    
    #Candlestick patterns
    candle_df = extract_candlestick_patterns(df)

    #Trigram patterns
    trigram_df = eight_trigram(df)

    # add simpe features
    simple_features_df = get_simple_ratios(df)
    
    ############################## R360 Feature Transformation ##############################
    
    # Concatenate all indicator features    
    df_with_indicators = pd.concat([df, rsi_df, sma_df, sma_monthly_df, ema_df, ema_monthly_df, dema_df, dema_monthly_df, bbands_df, cci_df,macd_df, mfi_df, roc_df,stoch_df,vwap_df,
                                    obv_df,adx_dmi_df,bop_df,kama_df, kama_monthly_df,par_sar_df,aroon_df,candle_df,trigram_df, simple_features_df], axis=1)
    
    df_with_indicators.set_index('date', inplace=True)

    df_with_indicators = fibonacci_pivot_points(df_with_indicators, window=5)
    
    df_with_indicators = fibonacci_pivot_points(df_with_indicators, window=20)

    return df_with_indicators


In [10]:
import talib as ta
grouped1 = final_df.groupby('nse_symbol')

df_list1 = []

for symbol, group1 in grouped1:
    print(f"Processing stock symbol: {symbol}")

    # Reset index and sort by dat
    group1 = group1.reset_index(drop=True)
    group1 = group1.sort_values(by='date')
    
    # Add technical indicators
    group_with_indicators1 = add_transform_indicators(group1)
    
    # Reset index and sort by date
    group_with_indicators1 = group_with_indicators1.reset_index()
    
    # Append to list
    df_list1.append(group_with_indicators1)

# Concatenate all DataFrames and reset index
final_df1 = pd.concat(df_list1, ignore_index=True)
# final_df = final_df.reset_index(drop=True)

Processing stock symbol: 360ONE
Processing stock symbol: 3MINDIA
Processing stock symbol: AARTIIND
Processing stock symbol: AAVAS
Processing stock symbol: ABB
Processing stock symbol: ABBOTINDIA
Processing stock symbol: ABCAPITAL
Processing stock symbol: ABFRL
Processing stock symbol: ACC
Processing stock symbol: ACE
Processing stock symbol: ACI
Processing stock symbol: ADANIENT
Processing stock symbol: ADANIGREEN
Processing stock symbol: ADANIPORTS
Processing stock symbol: ADANIPOWER
Processing stock symbol: AETHER
Processing stock symbol: AFFLE
Processing stock symbol: AIAENG
Processing stock symbol: AJANTPHARM
Processing stock symbol: ALKEM
Processing stock symbol: ALKYLAMINE
Processing stock symbol: ALLCARGO
Processing stock symbol: AMBER
Processing stock symbol: AMBUJACEM
Processing stock symbol: ANANDRATHI
Processing stock symbol: ANGELONE
Processing stock symbol: ANURAS
Processing stock symbol: APARINDS
Processing stock symbol: APLAPOLLO
Processing stock symbol: APLLTD
Processin

Processing stock symbol: JSL
Processing stock symbol: JSWENERGY
Processing stock symbol: JSWINFRA
Processing stock symbol: JSWSTEEL
Processing stock symbol: JUBLFOOD
Processing stock symbol: JUBLINGREA
Processing stock symbol: JUBLPHARMA
Processing stock symbol: JUSTDIAL
Processing stock symbol: JWL
Processing stock symbol: JYOTHYLAB
Processing stock symbol: KAJARIACER
Processing stock symbol: KALYANKJIL
Processing stock symbol: KANSAINER
Processing stock symbol: KARURVYSYA
Processing stock symbol: KAYNES
Processing stock symbol: KEC
Processing stock symbol: KEI
Processing stock symbol: KFINTECH
Processing stock symbol: KIMS
Processing stock symbol: KNRCON
Processing stock symbol: KOTAKBANK
Processing stock symbol: KPITTECH
Processing stock symbol: KPRMILL
Processing stock symbol: KRBL
Processing stock symbol: KSB
Processing stock symbol: LALPATHLAB
Processing stock symbol: LATENTVIEW
Processing stock symbol: LAURUSLABS
Processing stock symbol: LEMONTREE
Processing stock symbol: LICHSG

In [11]:
final_df1.head(1)

date  index    open    high     low   close    volume nse_symbol  \
0 2022-01-03      0  325.45  339.26  322.93  334.51  561384.0     360ONE   

   adj_close  next_day_return_close  target_close  next_day_return_high  \
0     334.51                   1.05             0                  2.39   

   target_high  next_day_return_low  target_low  ABER_ZG_29_15  ABER_SG_29_15  \
0            1                 -0.6           0            NaN            NaN   

   ABER_XG_29_15  ABER_ATR_29_15  ACCBL_5  ACCBM_5  ACCBU_5             AD  \
0            NaN             NaN      NaN      NaN      NaN  234798.084507   

   ADOSC_29_28  ALMA_22_6.0_0.85  AMATr_LR_24_25_28  AMATr_SR_24_25_28  \
0          NaN               NaN                  0                  0   

   AO_8_28       OBV  OBV_min_22  OBV_max_22  OBVs_18  OBVs_29  AOBV_LR_2  \
0      NaN  561384.0         NaN         NaN      NaN      NaN          0   

   AOBV_SR_2  APO_10_18  ATRe_28  BBL_21_2.0  BBM_21_2.0  BBU_21_2.0  \
0          0        NaN      NaN         NaN         NaN         NaN   

   BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27  AR_25  BR_25  CDL_2CROWS  \
0         NaN         NaN          NaN    NaN    NaN         0.0   

   CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  CDL_3OUTSIDE  \
0              0.0          0.0              0.0           0.0   

   CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  CDL_ADVANCEBLOCK  \
0                0.0                 0.0                0.0               0.0   

   CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  CDL_CONCEALBABYSWALL  \
0           0.0            0.0                  0.0                   0.0   

   CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  CDL_DOJI_10_0.1  CDL_DOJISTAR  \
0                0.0                 0.0              0.0           0.0   

   CDL_DRAGONFLYDOJI  CDL_ENGULFING  CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  \
0                0.0            0.0                  0.0              0.0   

   CDL_GAPSIDESIDEWHITE  CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  \
0                   0.0                 0.0         0.0             0.0   

   CDL_HARAMI  CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0         0.0              0.0           0.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
0                 0.0          0.0                  0.0               0.0   

   CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
0                 0.0           0.0           0.0              0.0   

   CDL_MATHOLD  CDL_MORNINGDOJISTAR  CDL_MORNINGSTAR  CDL_ONNECK  \
0          0.0                  0.0              0.0         0.0   

   CDL_PIERCING  CDL_RICKSHAWMAN  CDL_RISEFALL3METHODS  CDL_SEPARATINGLINES  \
0           0.0              0.0                   0.0                  0.0   

   CDL_SHOOTINGSTAR  CDL_SHORTLINE  CDL_SPINNINGTOP  CDL_STALLEDPATTERN  \
0               0.0            0.0              0.0                 0.0   

   CDL_STICKSANDWICH  CDL_TAKURI  CDL_TASUKIGAP  CDL_THRUSTING  CDL_TRISTAR  \
0                0.0         0.0            0.0            0.0          0.0   

   CDL_UNIQUE3RIVER  CDL_UPSIDEGAP2CROWS  CDL_XSIDEGAP3METHODS  open_Z_27_5  \
0               0.0                  0.0                   0.0          NaN   

   high_Z_27_5  low_Z_27_5  close_Z_27_5  CMF_22  CG_8  CHOP_11_26_100  \
0          NaN         NaN           NaN     NaN   NaN             NaN   

   CKSPl_7_1.0_6  CKSPs_7_1.0_6  CMO_22  COPC_10_26_4  CTI_25  DEC_25  \
0            NaN            NaN     NaN           NaN     NaN       0   

   DEMA_16  DMP_28  DMN_28  DCL_27_22  DCM_27_22  DCU_27_22  EBSW_40_10  \
0      NaN     NaN     NaN        NaN        NaN        NaN         NaN   

   EFI_7  EMA_28  ENTP_27  EOM_26_100000000  ER_21  BULLP_25  BEARP_25  \
0    NaN     NaN

In [12]:
# final_df1 = final_df1.reset_index(drop=True)
final_df1.to_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_stocks_with_technical_transform_indicators.parquet")

In [2]:
final_df1 = pd.read_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_stocks_with_technical_transform_indicators.parquet")

In [9]:
final_df1.drop(columns=['HILOl_16_28','HILOs_16_28','VIDYA_4','ICS_24','TRIXs_27_5','T3_26_0.7',
                 'ISB_24',
                  'ALMA_22_6.0_0.85','high_Z_27_5','close_Z_27_5','CHOP_11_26_100','CTI_25','ER_21','SUPERT_13_3.0','VTXP_21','VTXM_21','ZS_25','price_rel_BBANDS',
'%K_STOCH','%D_STOCH','%K_%D_diff_STOCH','Stoch_ROC','Stoch_Slope','Stoch_OSC_Range','1w_slope_OBV','VRSI_1w','VRSI_1m',
                  'BR_25','open_Z_27_5','low_Z_27_5','EBSW_40_10','VHF_14','cv_SMA_13','cv_SMA_42','cv_SMA_50','cv_SMA_100','1m_slope_OBV'],inplace=True)

In [10]:
final_df1.isna().sum()

date                                         0
index                                        0
open                                         0
high                                         0
low                                          0
close                                        0
volume                                       0
nse_symbol                                   0
adj_close                                    0
next_day_return_close                      480
target_close                                 0
next_day_return_high                       480
target_high                                  0
next_day_return_low                        480
target_low                                   0
ABER_ZG_29_15                            13440
ABER_SG_29_15                            13440
ABER_XG_29_15                            13440
ABER_ATR_29_15                            7200
ACCBL_5                                   3840
ACCBM_5                                   3840
ACCBU_5      

In [11]:
grouped2 = final_df1.groupby('nse_symbol')

df_list2 = []

for symbol, group2 in grouped2:
    print(f"Cleaning stock symbol: {symbol}")

    # Reset index and sort by date
    group2 = group2.reset_index(drop=True)
    group2 = group2.sort_values(by='date')

    # Store the last row to reattach later
    last_row = group2.iloc[-1]

    # Add technical indicators
    print('Dates before cleaning: ', group2['date'].min(), group2['date'].max())
    print('Shape before cleaning: ', group2.shape)

    # Clean the group but keep the last row
    group_cleaned2 = group2.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

    # Reattach the last row, ensuring no duplicates if it was removed
    if last_row.name not in group_cleaned2.index:
        group_cleaned2 = group_cleaned2.append(last_row)
        print('append  done')

    print('Dates after cleaning: ', group_cleaned2['date'].min(), group_cleaned2['date'].max())
    print('Shape after cleaning: ', group_cleaned2.shape)
    print('--' * 60)

    # Reset index and sort by date
    group_cleaned2 = group_cleaned2.reset_index()

    # Append to list
    df_list2.append(group_cleaned2)

# Concatenate all DataFrames and reset index
final_df2 = pd.concat(df_list2, ignore_index=True)

Cleaning stock symbol: 360ONE
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: 3MINDIA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: AARTIIND
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
--------------------------------------

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: APLAPOLLO
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: APLLTD
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: APOLLOHOSP
Dates before

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: BANDHANBNK
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: BANKBARODA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: BANKINDIA
Dates be

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: BSOFT
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: CAMPUS
Dates before cleaning:  2022-05-09 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (618, 414)
append  done
Dates after cleaning:  2023-02-21 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (420, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: CAMS
Dates before cleaning:

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: CIEINDIA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: CIPLA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: CLEAN
Dates before cleani

...

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: RECLTD
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: REDINGTON
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: RELIANCE
Dates before c

Cleaning stock symbol: SJVN
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: SKFINDIA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: SOBHA
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------

Cleaning stock symbol: TATACOMM
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: TATACONSUM
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: TATAELXSI
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
--------------------------------

Cleaning stock symbol: UBL
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: ULTRACEMCO
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: UNIONBANK
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
-------------------------------------

append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------
Cleaning stock symbol: ZYDUSLIFE
Dates before cleaning:  2022-01-03 00:00:00 2024-11-07 00:00:00
Shape before cleaning:  (702, 414)
append  done
Dates after cleaning:  2022-10-20 00:00:00 2024-11-07 00:00:00
Shape after cleaning:  (504, 414)
------------------------------------------------------------------------------------------------------------------------


In [12]:
final_df2.head(1)

level_0       date  index    open    high     low   close    volume  \
0      198 2022-10-20    198  448.25  448.25  422.25  426.67  304160.0   

  nse_symbol  adj_close  next_day_return_close  target_close  \
0     360ONE     426.67                  -3.77             0   

   next_day_return_high  target_high  next_day_return_low  target_low  \
0                  1.49            0                -4.82           1   

   ABER_ZG_29_15  ABER_SG_29_15  ABER_XG_29_15  ABER_ATR_29_15     ACCBL_5  \
0     425.516207     441.410878     409.621536       15.894671  393.001927   

      ACCBM_5    ACCBU_5            AD  ADOSC_29_28  AMATr_LR_24_25_28  \
0  436.425503  488.05582 -2.488510e+06 -6381.464564                  1   

   AMATr_SR_24_25_28   AO_8_28         OBV  OBV_min_22  OBV_max_22  \
0                  0  5.327411  13481036.0  12598512.0  13785196.0   

        OBVs_18       OBVs_29  AOBV_LR_2  AOBV_SR_2  APO_10_18    ATRe_28  \
0  1.296939e+07  1.275755e+07          1          0   2.599889  15.519676   

   BBL_21_2.0  BBM_21_2.0  BBU_21_2.0  BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27  \
0  405.929639  428.685238  451.440837    10.61646     0.45572      0.00035   

       AR_25  CDL_2CROWS  CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  \
0  93.911911         0.0              0.0          0.0            100.0   

   CDL_3OUTSIDE  CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  \
0           0.0                0.0                 0.0                0.0   

   CDL_ADVANCEBLOCK  CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  \
0               0.0        -100.0            0.0                  0.0   

   CDL_CONCEALBABYSWALL  CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  \
0                   0.0                0.0                 0.0   

   CDL_DOJI_10_0.1  CDL_DOJISTAR  CDL_DRAGONFLYDOJI  CDL_ENGULFING  \
0              0.0           0.0                0.0         -100.0   

   CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  CDL_GAPSIDESIDEWHITE  \
0                  0.0              0.0                   0.0   

   CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  CDL_HARAMI  \
0                 0.0         0.0             0.0         0.0   

   CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0              0.0           0.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
0                 0.0          0.0                  0.0               0.0   

   CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
0                 0.0           0.0           0.0              0.0   

   CDL_MATHOLD  CDL_MORNINGDOJISTAR  CDL_MORNINGSTAR  CDL_ONNECK  \
0          0.0                  0.0              0.0         0.0   

   CDL_PIERCING  CDL_RICKSHAWMAN  CDL_RISEFALL3METHODS  CDL_SEPARATINGLINES  \
0           0.0              0.0                   0.0                  0.0   

   CDL_SHOOTINGSTAR  CDL_SHORTLINE  CDL_SPINNINGTOP  CDL_STALLEDPATTERN  \
0               0.0            0.0              0.0                 0.0   

   CDL_STICKSANDWICH  CDL_TAKURI  CDL_TASUKIGAP  CDL_THRUSTING  CDL_TRISTAR  \
0                0.0         0.0            0.0            0.0          0.0   

   CDL_UNIQUE3RIVER  CDL_UPSIDEGAP2CROWS  CDL_XSIDEGAP3METHODS    CMF_22  \
0               0.0                  0.0                   0.0  0.086506   

       CG_8  CKSPl_7_1.0_6  CKSPs_7_1.0_6    CMO_22  COPC_10_26_4  DEC_25  \
0 -4.469397     434.059703     428.230297  5.442778      8.301678       0   

      DEMA_16      DMP_28     DMN_28  DCL_27_22  DCM_27_22  DCU_27_22  \
0  436.888787  105.257594  76.885352     394.46     426.66     458.86   

           EFI_7      EMA_28   ENTP_27  EOM_26_100000000  BULLP_25  BEARP_25  \
0  257765.385925  424.522235  4.870059     -13946.585424  22.27288  -3.72712   

   FISHERT_27_5  FISHERTs_27_5     FWMA_24  

In [7]:

# Save final DataFrame to parquet file
final_df2.to_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_stocks_with_technical_indicators_cleaned_data.parquet")


In [3]:
final_df2 = pd.read_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\all_stocks_with_technical_indicators_cleaned_data.parquet")

In [3]:
final_df2 = final_df2.reset_index(drop=True)

In [4]:
final_df2['date'].min(),final_df2['date'].max()

(Timestamp('2022-10-20 00:00:00'), Timestamp('2024-11-07 00:00:00'))

In [5]:
final_df2.isna().sum()

level_0                                    0
date                                       0
index                                      0
open                                       0
high                                       0
low                                        0
close                                      0
volume                                     0
nse_symbol                                 0
adj_close                                  0
next_day_return_close                    480
target_close                               0
next_day_return_high                     480
target_high                                0
next_day_return_low                      480
target_low                                 0
ABER_ZG_29_15                              0
ABER_SG_29_15                              0
ABER_XG_29_15                              0
ABER_ATR_29_15                             0
ACCBL_5                                    0
ACCBM_5                                    0
ACCBU_5   

In [6]:
import pandas as pd
from pandas_market_calendars import get_calendar
from datetime import timedelta
import numpy as np

def cache_market_days(start_date: str, end_date: str):
    bse_calendar = get_calendar('XBOM')
    valid_days = bse_calendar.valid_days(start_date, end_date)
    return pd.to_datetime(valid_days).strftime('%Y-%m-%d').tolist()

# Map each date to the next market day using a precomputed list
def get_next_market_day_lookup(market_days):
    # Create a lookup dictionary for next market days
    market_days = pd.Series(market_days).sort_values()
    
    # Shift the series to get the next valid market day
    market_day_lookup = pd.DataFrame({
        'current_day': market_days,
        'next_market_day': market_days.shift(-1)
    }).dropna()
    
    return market_day_lookup.set_index('current_day').to_dict()['next_market_day']

def find_missing_dates(stock_df, market_day_lookup):
    stock_df['date'] = pd.to_datetime(stock_df['date'])
    stock_df['date_str'] = stock_df['date'].dt.strftime('%Y-%m-%d')

    stock_df['next_market_day'] = stock_df['date_str'].map(market_day_lookup)

    missing_dates = stock_df.loc[~stock_df['next_market_day'].isin(stock_df['date_str']), 'next_market_day']
    return missing_dates.dropna().tolist()

def get_missing_dates_for_all_stocks(df: pd.DataFrame):
    start_date = df['date'].min().date()
    end_date = df['date'].max().date()
    
    market_days = cache_market_days(str(start_date), str(end_date))
    
    market_day_lookup = get_next_market_day_lookup(market_days)
    
    missing_dates_dict = {}
    
    for stock_symbol, stock_df in df.groupby('nse_symbol'):
        print(f'Checking for {stock_symbol}')
        missing_dates = find_missing_dates(stock_df, market_day_lookup)
        print(f'Total dates missing for {stock_symbol}: {len(missing_dates)}')
        missing_dates_dict[stock_symbol] = missing_dates

    missing_dates_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in missing_dates_dict.items()]))
    
    return missing_dates_df


In [7]:
model_data = final_df2[final_df2['date']>='2022-10-20'].reset_index(drop=True)

In [8]:
model_data.date.min()

Timestamp('2022-10-20 00:00:00')

In [9]:
model_data.drop(columns=['index','level_0'],inplace=True)

In [10]:
model_data.head(2)

date    open    high     low   close    volume nse_symbol  adj_close  \
0 2022-10-20  448.25  448.25  422.25  426.67  304160.0     360ONE     426.67   
1 2022-10-21  428.81  435.19  408.14  412.65   85140.0     360ONE     412.65   

   next_day_return_close  target_close  next_day_return_high  target_high  \
0                  -3.77             0                  1.49            0   
1                  -0.63             0                  0.60            0   

   next_day_return_low  target_low  ABER_ZG_29_15  ABER_SG_29_15  \
0                -4.82           1     425.516207     441.410878   
1                -2.83           1     425.571149     442.209509   

   ABER_XG_29_15  ABER_ATR_29_15     ACCBL_5     ACCBM_5     ACCBU_5  \
0     409.621536       15.894671  393.001927  436.425503  488.055820   
1     408.932790       16.638359  371.839098  423.519110  491.804224   

             AD  ADOSC_29_28  AMATr_LR_24_25_28  AMATr_SR_24_25_28   AO_8_28  \
0 -2.488510e+06 -6381.464564                  1                  0  5.327411   
1 -2.545259e+06 -6467.023843                  1                  0  4.550089   

          OBV  OBV_min_22  OBV_max_22       OBVs_18       OBVs_29  AOBV_LR_2  \
0  13481036.0  12598512.0  13785196.0  1.296939e+07  1.275755e+07          1   
1  13395896.0  12598512.0  13785196.0  1.302401e+07  1.281819e+07          1   

   AOBV_SR_2  APO_10_18    ATRe_28  BBL_21_2.0  BBM_21_2.0  BBU_21_2.0  \
0          0   2.599889  15.519676  405.929639  428.685238  451.440837   
1          0   0.243444  15.931473  404.444438  427.242857  450.041277   

   BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27      AR_25  CDL_2CROWS  \
0   10.616460    0.455720     0.000350  93.911911         0.0   
1   10.672347    0.179959    -0.033321  89.221557         0.0   

   CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  CDL_3OUTSIDE  \
0              0.0          0.0            100.0           0.0   
1              0.0          0.0              0.0        -100.0   

   CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  CDL_ADVANCEBLOCK  \
0                0.0                 0.0                0.0               0.0   
1                0.0                 0.0                0.0               0.0   

   CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  CDL_CONCEALBABYSWALL  \
0        -100.0            0.0                  0.0                   0.0   
1           0.0            0.0                  0.0                   0.0   

   CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  CDL_DOJI_10_0.1  CDL_DOJISTAR  \
0                0.0                 0.0              0.0           0.0   
1                0.0                 0.0              0.0           0.0   

   CDL_DRAGONFLYDOJI  CDL_ENGULFING  CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  \
0                0.0         -100.0                  0.0              0.0   
1                0.0            0.0                  0.0              0.0   

   CDL_GAPSIDESIDEWHITE  CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  \
0                   0.0                 0.0         0.0             0.0   
1                   0.0                 0.0         0.0             0.0   

   CDL_HARAMI  CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0         0.0              0.0           0.0          0.0             0.0   
1         0.0              0.0           0.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   
1               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
0                 0.0          0.0                  0.0               0.0   
1                 0.0          0.0                  0.0               0.0   

   CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
0                 0.0           0.0           0.0              0.0   
1                 0.0           0.0         

In [11]:
model_data.isna().sum()

date                                       0
open                                       0
high                                       0
low                                        0
close                                      0
volume                                     0
nse_symbol                                 0
adj_close                                  0
next_day_return_close                    480
target_close                               0
next_day_return_high                     480
target_high                                0
next_day_return_low                      480
target_low                                 0
ABER_ZG_29_15                              0
ABER_SG_29_15                              0
ABER_XG_29_15                              0
ABER_ATR_29_15                             0
ACCBL_5                                    0
ACCBM_5                                    0
ACCBU_5                                    0
AD                                         0
ADOSC_29_2

In [12]:
missing_dates = get_missing_dates_for_all_stocks(model_data)

Checking for 360ONE
Total dates missing for 360ONE: 2
Checking for 3MINDIA
Total dates missing for 3MINDIA: 2
Checking for AARTIIND
Total dates missing for AARTIIND: 2
Checking for AAVAS
Total dates missing for AAVAS: 2
Checking for ABB
Total dates missing for ABB: 2
Checking for ABBOTINDIA
Total dates missing for ABBOTINDIA: 2
Checking for ABCAPITAL
Total dates missing for ABCAPITAL: 2
Checking for ABFRL
Total dates missing for ABFRL: 2
Checking for ACC
Total dates missing for ACC: 2
Checking for ACE
Total dates missing for ACE: 2
Checking for ACI
Total dates missing for ACI: 2
Checking for ADANIENT
Total dates missing for ADANIENT: 2
Checking for ADANIGREEN
Total dates missing for ADANIGREEN: 2
Checking for ADANIPORTS
Total dates missing for ADANIPORTS: 2
Checking for ADANIPOWER
Total dates missing for ADANIPOWER: 2
Checking for AETHER
Total dates missing for AETHER: 2
Checking for AFFLE
Total dates missing for AFFLE: 2
Checking for AIAENG
Total dates missing for AIAENG: 2
Checking f

Total dates missing for IIFL: 2
Checking for INDHOTEL
Total dates missing for INDHOTEL: 2
Checking for INDIACEM
Total dates missing for INDIACEM: 2
Checking for INDIAMART
Total dates missing for INDIAMART: 2
Checking for INDIANB
Total dates missing for INDIANB: 2
Checking for INDIGO
Total dates missing for INDIGO: 2
Checking for INDIGOPNTS
Total dates missing for INDIGOPNTS: 2
Checking for INDUSINDBK
Total dates missing for INDUSINDBK: 2
Checking for INDUSTOWER
Total dates missing for INDUSTOWER: 2
Checking for INFY
Total dates missing for INFY: 2
Checking for INOXWIND
Total dates missing for INOXWIND: 2
Checking for INTELLECT
Total dates missing for INTELLECT: 2
Checking for IOB
Total dates missing for IOB: 2
Checking for IOC
Total dates missing for IOC: 2
Checking for IPCALAB
Total dates missing for IPCALAB: 2
Checking for IRB
Total dates missing for IRB: 2
Checking for IRCON
Total dates missing for IRCON: 2
Checking for IRCTC
Total dates missing for IRCTC: 2
Checking for IRFC
Total 

Total dates missing for TECHM: 2
Checking for TEJASNET
Total dates missing for TEJASNET: 2
Checking for THERMAX
Total dates missing for THERMAX: 2
Checking for TIINDIA
Total dates missing for TIINDIA: 2
Checking for TIMKEN
Total dates missing for TIMKEN: 2
Checking for TITAGARH
Total dates missing for TITAGARH: 2
Checking for TITAN
Total dates missing for TITAN: 2
Checking for TMB
Total dates missing for TMB: 2
Checking for TORNTPHARM
Total dates missing for TORNTPHARM: 2
Checking for TORNTPOWER
Total dates missing for TORNTPOWER: 2
Checking for TRENT
Total dates missing for TRENT: 2
Checking for TRITURBINE
Total dates missing for TRITURBINE: 2
Checking for TRIVENI
Total dates missing for TRIVENI: 2
Checking for TTML
Total dates missing for TTML: 2
Checking for TVSMOTOR
Total dates missing for TVSMOTOR: 2
Checking for TVSSCS
Total dates missing for TVSSCS: 0
Checking for UBL
Total dates missing for UBL: 2
Checking for ULTRACEMCO
Total dates missing for ULTRACEMCO: 2
Checking for UNIONB

In [13]:
missing_dates

360ONE     3MINDIA    AARTIIND       AAVAS         ABB  ABBOTINDIA  \
0   2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22   
1   2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20   
2          NaN         NaN         NaN         NaN         NaN         NaN   
3          NaN         NaN         NaN         NaN         NaN         NaN   
4          NaN         NaN         NaN         NaN         NaN         NaN   
5          NaN         NaN         NaN         NaN         NaN         NaN   
6          NaN         NaN         NaN         NaN         NaN         NaN   
7          NaN         NaN         NaN         NaN         NaN         NaN   
8          NaN         NaN         NaN         NaN         NaN         NaN   
9          NaN         NaN         NaN         NaN         NaN         NaN   
10         NaN         NaN         NaN         NaN         NaN         NaN   
11         NaN         NaN         NaN         NaN         NaN         NaN   

     ABCAPITAL       ABFRL         ACC         ACE         ACI    ADANIENT  \
0   2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22   
1   2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20   
2          NaN         NaN         NaN         NaN         NaN         NaN   
3          NaN         NaN         NaN         NaN         NaN         NaN   
4          NaN         NaN         NaN         NaN         NaN         NaN   
5          NaN         NaN         NaN         NaN         NaN         NaN   
6          NaN         NaN         NaN         NaN         NaN         NaN   
7          NaN         NaN         NaN         NaN         NaN         NaN   
8          NaN         NaN         NaN         NaN         NaN         NaN   
9          NaN         NaN         NaN         NaN         NaN         NaN   
10         NaN         NaN         NaN         NaN         NaN         NaN   
11         NaN         NaN         NaN         NaN         NaN         NaN   

    ADANIGREEN  ADANIPORTS  ADANIPOWER      AETHER       AFFLE      AIAENG  \
0   2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22   
1   2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20   
2          NaN         NaN         NaN         NaN         NaN         NaN   
3          NaN         NaN         NaN         NaN         NaN         NaN   
4          NaN         NaN         NaN         NaN         NaN         NaN   
5          NaN         NaN         NaN         NaN         NaN         NaN   
6          NaN         NaN         NaN         NaN         NaN         NaN   
7          NaN         NaN         NaN         NaN         NaN         NaN   
8          NaN         NaN         NaN         NaN         NaN         NaN   
9          NaN         NaN         NaN         NaN         NaN         NaN   
10         NaN         NaN         NaN         NaN         NaN         NaN   
11         NaN         NaN         NaN         NaN         NaN         NaN   

    AJANTPHARM       ALKEM  ALKYLAMINE    ALLCARGO       AMBER   AMBUJACEM  \
0   2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22  2024-01-22   
1   2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20  2024-05-20   
2          NaN         NaN         NaN         NaN         NaN         NaN   
3          NaN         NaN         NaN         NaN         NaN         NaN   
4          NaN         NaN         NaN         NaN         NaN         NaN   
5          NaN         NaN         NaN         NaN         NaN         NaN   
6          NaN         NaN         NaN         NaN         NaN         NaN   
7          NaN         NaN         NaN         NaN         NaN         NaN   
8          NaN         NaN         NaN         NaN         NaN         NaN   
9          NaN         NaN         NaN         NaN         NaN         NaN   
10         NaN         NaN         NaN         NaN         NaN         NaN   
11         NaN         NaN 

In [14]:
# model_data.to_parquet(r"D:\alpha_generator\alpha_short_calls\data\model_raw_data_intraday2004_oct2024.parquet")

In [15]:
model_data['nse_symbol'].nunique()

480

In [16]:
today = model_data['date'].max()
today = today.strftime('%Y-%m-%d')

In [17]:
scoring_df = model_data[model_data['date']>='2023-01-01']

In [18]:
scoring_df.shape

(211197, 413)

In [19]:
scoring_df['date'].max()

Timestamp('2024-11-07 00:00:00')

In [20]:
scoring_df.to_parquet(r"D:\alpha_generator\alpha_intraday_short_model\scoring_data\daily_scoring_short_calls_data.parquet")

In [2]:
# scoring_df = pd.read_parquet(r"D:\alpha_generator\alpha_intraday_short_model\data\intraday_short_model_data_2005_20240ct.parquet")

In [3]:
scoring_df.head(2)

date    open    high     low   close   volume nse_symbol  adj_close  \
0 2020-03-04  361.25  369.75  357.01  362.94  77636.0     360ONE     362.94   
1 2020-03-05  365.61  386.74  362.50  368.23  74232.0     360ONE     368.23   

   next_day_return_close  target_close  next_day_return_high  target_high  \
0                   0.72             0                  5.78            1   
1                   1.91             0                  6.35            1   

   next_day_return_low  target_low  target_close_intraday  \
0                -0.85           0                      0   
1                -3.03           1                      0   

   target_high_intraday  target_low_intraday  5d_return_high  target_high_5d  \
0                     1                    0        5.779382             1.0   
1                     1                    1        6.350314             1.0   

   target_high_5d_multiclass  ABER_ATR_29_15  aber_band_width  aber_momentum  \
0                        1.0       18.519360        37.038720      -0.009990   
1                        1.0       18.900736        37.801472       0.139942   

   accband_band_width  norm_ad_position  alma_momentum  alma_slope  \
0           71.332465               0.0       6.258649    5.050155   
1           96.826783               0.0       7.204527    5.851126   

   AMATr_LR_24_25_28  AMATr_SR_24_25_28    AO_8_28    ao_roc  ao_zero_cross  \
0                  1                  0  11.797590 -6.284375              0   
1                  1                  0   8.087857 -3.709732              0   

   AOBV_LR_2  AOBV_SR_2  APO_10_18   apo_roc  apo_signal    ATRe_28  \
0          0          1  -2.246889 -4.005000           0  17.789556   
1          0          1  -6.923472 -4.676582           0  18.019929   

   atr_pct_close  atr_slope  BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27  \
0       4.901514  -0.187021   22.360926    0.366666    -0.002275   
1       4.893661   0.230373   20.292347    0.402255     0.008022   

   absolute_bias      AR_25  ar_br_ratio  CDL_2CROWS  CDL_3BLACKCROWS  \
0       0.002275  75.091965     0.613218         0.0              0.0   
1       0.008022  84.724496     0.609718         0.0              0.0   

   CDL_3INSIDE  CDL_3LINESTRIKE  CDL_3OUTSIDE  CDL_3STARSINSOUTH  \
0          0.0              0.0           0.0                0.0   
1          0.0              0.0           0.0                0.0   

   CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  CDL_ADVANCEBLOCK  CDL_BELTHOLD  \
0                 0.0                0.0               0.0           0.0   
1                 0.0                0.0               0.0           0.0   

   CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  CDL_CONCEALBABYSWALL  \
0            0.0                  0.0                   0.0   
1            0.0                  0.0                   0.0   

   CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  CDL_DOJI_10_0.1  CDL_DOJISTAR  \
0                0.0                 0.0            100.0           0.0   
1                0.0                 0.0              0.0           0.0   

   CDL_DRAGONFLYDOJI  CDL_ENGULFING  CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  \
0                0.0            0.0                  0.0              0.0   
1                0.0            0.0                  0.0              0.0   

   CDL_GAPSIDESIDEWHITE  CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  \
0                   0.0                 0.0         0.0             0.0   
1                   0.0                 0.0         0.0             0.0   

   CDL_HARAMI  CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0         0.0              0.0         100.0          0.0             0.0   
1         0.0              0.0           0.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   
1               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL

In [4]:
with open(r"D:\alpha_generator\alpha_intraday_short_model\model_object\intraday_sell_lgb_model_final.pkl", 'rb') as file:
    lgb = pickle.load(file)

In [5]:
feat = lgb.feature_names_in_.tolist()

In [6]:
len(feat)

196

In [4]:
# scoring_df = final_df2.copy()

In [7]:
scoring_df['lgb_prob'] = lgb.predict_proba(scoring_df[feat])[:, 1]

KeyError: "['PVOL', 'EFI_7', 'ADOSC_29_28', 'AD', 'days_since_convergence_KAMA_50_100', 'volume_oscillator', 'cv_DEMA_100', 'PVT', 'days_since_divergence_KAMA_50_100', 'ratio_DEMA_100', 'KSTs_6', 'PDIST', 'EOM_26_100000000', 'KAMA_difference_50_100', 'OBV_max_22', 'OBV_min_22', 'DMP_28', 'KST_11_17_9_17_7_6_6_26', 'ISA_11', 'DCL_27_22', 'DMN_28', '1w_ema_OBV', 'OBV', 'ACCBL_5', '1w_sma_OBV', 'ACCBU_5', 'OBVs_18', 'VWMA_22'] not in index"

In [26]:
# scoring_df['lgb_prob'] = scoring_df['lgb_prob'].round(2)

In [10]:
scoring_df.sort_values(by='lgb_prob',ascending=False,inplace=True)

In [11]:
scoring_df.head(2)

level_0       date  index    open    high     low   close      volume  \
6408        271 2023-02-06    271  492.56  542.71  471.49  539.49  38363919.0   
158626      300 2023-03-20    300   45.00   62.80   45.00   59.20   7507743.0   

        nse_symbol  adj_close  next_day_return_close  target_close  \
6408    ADANIPORTS     539.49                   0.57             0   
158626   NETWORK18      59.20                   3.56             1   

        next_day_return_high  target_high  next_day_return_low  target_low  \
6408                    8.81            1                -1.27           0   
158626                  6.34            1                -1.65           0   

        ABER_ZG_29_15  ABER_SG_29_15  ABER_XG_29_15  ABER_ATR_29_15  \
6408       727.506322     781.291304     673.721339       53.784982   
158626      58.494253      61.948803      55.039702        3.454551   

           ACCBL_5     ACCBM_5     ACCBU_5            AD    ADOSC_29_28  \
6408    244.781743  494.298361  730.456671 -2.413637e+06  610156.221446   
158626   26.215042   55.926031   87.246141 -7.254821e+07   30830.796691   

        AMATr_LR_24_25_28  AMATr_SR_24_25_28     AO_8_28         OBV  \
6408                    0                  1 -162.103304  48517551.0   
158626                  0                  1   -3.171875  39569724.0   

        OBV_min_22  OBV_max_22       OBVs_18       OBVs_29  AOBV_LR_2  \
6408   -79509234.0  89380578.0  5.918998e+07  7.468990e+07          0   
158626  27904383.0  39569724.0  3.089308e+07  3.121985e+07          0   

        AOBV_SR_2  APO_10_18    ATRe_28  BBL_21_2.0  BBM_21_2.0  BBU_21_2.0  \
6408            1 -79.748333  40.864129  461.019294  695.363810  929.708325   
158626          1  -1.100000   2.863718   52.455185   57.792857   63.130530   

        BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27       AR_25  CDL_2CROWS  \
6408     67.401988    0.167426    -0.251441   33.077583         0.0   
158626   18.471738    0.631812     0.013924  143.478261         0.0   

        CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  CDL_3OUTSIDE  \
6408                0.0          0.0              0.0           0.0   
158626              0.0          0.0              0.0           0.0   

        CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  \
6408                  0.0                 0.0                0.0   
158626                0.0                 0.0                0.0   

        CDL_ADVANCEBLOCK  CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  \
6408                 0.0           0.0            0.0                100.0   
158626               0.0         100.0            0.0                  0.0   

        CDL_CONCEALBABYSWALL  CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  \
6408                     0.0                0.0                 0.0   
158626                   0.0                0.0                 0.0   

        CDL_DOJI_10_0.1  CDL_DOJISTAR  CDL_DRAGONFLYDOJI  CDL_ENGULFING  \
6408                0.0           0.0                0.0            0.0   
158626              0.0           0.0                0.0          100.0   

        CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  CDL_GAPSIDESIDEWHITE  \
6408                    0.0              0.0                   0.0   
158626                  0.0              0.0                   0.0   

        CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  CDL_HARAMI  \
6408                   0.0         0.0             0.0         0.0   
158626                 0.0         0.0             0.0         0.0   

        CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
6408                0.0           0.0          0.0             0.0   
158626              0.0           0.0          0.0             0.0   

        CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
6408                 0.0                  0.0         0.0           0   
158626               0.0                  0.0         0.0           0   

        CDL_INVERTEDHAMMER  CDL_KICKING  C

In [12]:
cols = ['date','nse_symbol','lgb_prob']

In [13]:
scoring_df[scoring_df['lgb_prob']>=0.90].head()

level_0       date  index    open     high     low    close  \
6408        271 2023-02-06    271  492.56   542.71  471.49   539.49   
158626      300 2023-03-20    300   45.00    62.80   45.00    59.20   
188120      465 2023-11-21    465   65.60    77.85   64.80    76.00   
158630      304 2023-03-24    304   56.20    56.95   52.00    52.25   
129449      395 2024-08-09    395  913.85  1084.63  909.82  1069.26   

            volume  nse_symbol  adj_close  next_day_return_close  \
6408    38363919.0  ADANIPORTS     539.49                   0.57   
158626   7507743.0   NETWORK18      59.20                   3.56   
188120  87667467.0    RTNINDIA      76.00                  -3.09   
158630   2526118.0   NETWORK18      52.25                   0.19   
129449  12535899.0    KFINTECH    1069.26                  -2.96   

        target_close  next_day_return_high  target_high  next_day_return_low  \
6408               0                  8.81            1                -1.27   
158626             1                  6.34            1                -1.65   
188120             0                  1.78            0                -5.07   
158630             0                  7.10            1                -5.61   
129449             0                  0.59            0                -5.35   

        target_low  ABER_ZG_29_15  ABER_SG_29_15  ABER_XG_29_15  \
6408             0     727.506322     781.291304     673.721339   
158626           0      58.494253      61.948803      55.039702   
188120           1      57.756322      61.137275      54.375369   
158630           1      58.055172      61.534692      54.575653   
129449           1     787.571149     835.113881     740.028418   

        ABER_ATR_29_15     ACCBL_5     ACCBM_5      ACCBU_5            AD  \
6408         53.784982  244.781743  494.298361   730.456671 -2.413637e+06   
158626        3.454551   26.215042   55.926031    87.246141 -7.254821e+07   
188120        3.380953   47.952778   70.991007    91.652899 -6.538160e+07   
158630        3.479519   45.966155   54.595799    66.710374 -7.397043e+07   
129449       47.542732  659.127667  988.442719  1262.140495  2.866743e+07   

          ADOSC_29_28  AMATr_LR_24_25_28  AMATr_SR_24_25_28     AO_8_28  \
6408    610156.221446                  0                  1 -162.103304   
158626   30830.796691                  0                  1   -3.171875   
188120  101526.341686                  1                  0    5.667857   
158630  -11615.675228                  0                  1   -2.751786   
129449     169.830671                  1                  0   83.635089   

                OBV   OBV_min_22   OBV_max_22       OBVs_18       OBVs_29  \
6408     48517551.0  -79509234.0   89380578.0  5.918998e+07  7.468990e+07   
158626   39569724.0   27904383.0   39569724.0  3.089308e+07  3.121985e+07   
188120  255291637.0  100586893.0  255291637.0  1.325906e+08  1.189480e+08   
158630   41284451.0   27904383.0   48691745.0  3.320500e+07  3.207495e+07   
129449   46084999.0   18225567.0   46084999.0  2.476702e+07  2.150714e+07   

        AOBV_LR_2  AOBV_SR_2  APO_10_18    ATRe_28  BBL_21_2.0  BBM_21_2.0  \
6408            0          1 -79.748333  40.864129  461.019294  695.363810   
158626          0          1  -1.100000   2.863718   52.455185   57.792857   
188120          1          0   3.925000   3.083981   46.230201   58.171429   
158630          1          0  -1.585000   2.957109   51.792048   57.057143   
129449          1          0  54.086667  39.378891  644.695056  804.208571   

        BBU_21_2.0  BBB_21_2.0  BBP_21_2.0  BIAS_SMA_27       AR_25  \
6408    929.708325   67.401988    0.167426    -0.251441   33.077583   
158626   63.130530   18.471738    0.631812     0.013924  143.478261   
188120   70.112657   41.055302    1.246513     0.315216  162.062937   
158630   62.322238   18.455515    0.043489    -0.095934  140.312509   
129449  963.722087   39.669688    1.330812     0.349095  213.001830   

        CDL_2CR

In [15]:
scoring_df[scoring_df['date']=='2024-11-07']['target_low'].sum()

0

In [19]:
scoring_df[(scoring_df['date']=='2024-11-07') & (scoring_df['lgb_prob']>=0.90)][cols].sort_values(by='lgb_prob',ascending=False).head(10)

date  nse_symbol  lgb_prob
60995 2024-11-07  DCMSHRIRAM  0.931525

In [20]:
scoring_df[scoring_df['date']=='2024-11-07'].sort_values(by='lgb_prob',ascending=False).head(20)

level_0       date  index      open      high       low     close  \
60995       701 2024-11-07    701   1129.90   1275.00   1126.05   1255.00   
116192      701 2024-11-07    701    235.50    276.70    235.50    266.49   
207525      540 2024-11-07    540    538.05    580.25    538.05    568.35   
35728       701 2024-11-07    701   1916.05   1931.10   1732.90   1778.55   
81366       701 2024-11-07    701   1768.95   1774.90   1644.10   1657.35   
97005       701 2024-11-07    701    682.00    687.70    646.30    648.10   
118712      701 2024-11-07    701    328.40    337.80    317.70    318.60   
124429      701 2024-11-07    701   1264.00   1309.90   1242.30   1254.55   
154491      701 2024-11-07    701   1890.00   1890.00   1789.95   1823.85   
176462      701 2024-11-07    701  14895.15  14940.00  14200.00  14353.85   
206174      701 2024-11-07    701    493.95    569.60    492.60    535.90   
61499       701 2024-11-07    701   1397.00   1397.00   1337.00   1346.20   
41692       701 2024-11-07    701   2085.10   2176.75   2043.20   2121.70   
39676       617 2024-11-07    617    305.50    320.95    303.90    314.35   
37744       701 2024-11-07    701   1205.65   1224.95   1146.20   1155.45   
217968      701 2024-11-07    701   6999.00   7015.25   6306.85   6505.50   
131526      701 2024-11-07    701   1510.90   1520.00   1435.00   1441.55   
81870       701 2024-11-07    701   1098.05   1164.00   1098.05   1116.75   
67975       701 2024-11-07    701    610.75    610.75    586.30    593.70   
226635      701 2024-11-07    701    984.70   1038.45    965.10   1025.20   

            volume  nse_symbol  adj_close  next_day_return_close  \
60995    1764923.0  DCMSHRIRAM    1255.00                    NaN   
116192  34923768.0         ITI     266.49                    NaN   
207525   3132031.0       SYRMA     568.35                    NaN   
35728    1974593.0  BLUESTARCO    1778.55                    NaN   
81366    1390796.0    GLENMARK    1657.35                    NaN   
97005   26671445.0    HINDALCO     648.10                    NaN   
118712   1499391.0   JINDALSAW     318.60                    NaN   
124429    734660.0  JUBLPHARMA    1254.55                    NaN   
154491    680859.0  MUTHOOTFIN    1823.85                    NaN   
176462     57291.0  POWERINDIA   14353.85                    NaN   
206174  26645059.0  SWANENERGY     535.90                    NaN   
61499     587799.0  DEEPAKFERT    1346.20                    NaN   
41692     348917.0  CAPLIPOINT    2121.70                    NaN   
39676    1743918.0      CAMPUS     314.35                    NaN   
37744     433812.0     BRIGADE    1155.45                    NaN   
217968   4427899.0       TRENT    6505.50                    NaN   
131526   1185243.0    KPITTECH    1441.55                    NaN   
81870     221498.0         GLS    1116.75                    NaN   
67975     565746.0      ELECON     593.70                    NaN   
226635    562237.0      VIJAYA    1025.20                    NaN   

        target_close  next_day_return_high  target_high  next_day_return_low  \
60995              0                   NaN            0                  NaN   
116192             0                   NaN            0                  NaN   
207525             0                   NaN            0                  NaN   
35728              0                   NaN            0                  NaN   
81366              0                   NaN            0                  NaN   
97005              0                   NaN            0                  NaN   
118712             0                   NaN            0                  NaN   
124429             0                   NaN            0                  NaN   
154491             0                   NaN            0                  NaN   
176462             0                   NaN            0                  NaN   
206174             0                   NaN            0                  NaN   
6

In [48]:
import yfinance as yf
import time
def fetch_nifty_ohlcv(start_date, end_date):
        
        nifty_ticker = '^CRSLDX'
        
        # Attempt to download the data with a retry mechanism
        for attempt in range(4):
            try:
                # Download OHLCV data from Yahoo Finance
                nifty_data = yf.download(nifty_ticker, start=start_date, end=end_date).reset_index()
                
                # Rename columns to match the desired format
                nifty_data.rename(columns={
                    'Open': 'open',
                    'High': 'high',
                    'Low': 'low',
                    'Close': 'close',
                    'Adj Close': 'adj_close',
                    'Volume': 'volume'
                }, inplace=True)
                
                # Add the ticker symbol column
                nifty_data['Symbol'] = nifty_ticker
                
                # Rename 'Date' column to 'date'
                nifty_data.rename(columns={'Date': 'date'}, inplace=True)
                print("Nifty Data Fetched..")
                return nifty_data
            
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                time.sleep(2)  # Wait for 2 seconds before retrying
                
                if attempt == 1:  # If this is the last attempt, raise the error
                    raise

In [49]:
nifty_ohlcv = fetch_nifty_ohlcv('2023-01-01', '2024-11-30')

[*********************100%%**********************]  1 of 1 completed

Nifty Data Fetched..


In [50]:
nifty_ohlcv.tail()

date          open          high           low         close  \
446 2024-10-29  22670.800781  22808.800781  22450.000000  22795.050781   
447 2024-10-30  22731.150391  22896.550781  22685.400391  22749.800781   
448 2024-10-31  22758.599609  22779.500000  22621.099609  22689.349609   
449 2024-11-01  22806.949219  22881.500000  22799.699219  22823.550781   
450 2024-11-04  22831.800781  22832.650391  22362.650391  22518.300781   

        adj_close    volume   Symbol  
446  22795.050781  23194400  ^CRSLDX  
447  22749.800781  27854400  ^CRSLDX  
448  22689.349609  24362900  ^CRSLDX  
449  22823.550781   6042900  ^CRSLDX  
450  22518.300781         0  ^CRSLDX

In [51]:
nifty_ohlcv['day_return'] = (((nifty_ohlcv['close'] - nifty_ohlcv['open'])/(nifty_ohlcv['open']))*100).round(2)

In [52]:
nifty_ohlcv.tail()

date          open          high           low         close  \
446 2024-10-29  22670.800781  22808.800781  22450.000000  22795.050781   
447 2024-10-30  22731.150391  22896.550781  22685.400391  22749.800781   
448 2024-10-31  22758.599609  22779.500000  22621.099609  22689.349609   
449 2024-11-01  22806.949219  22881.500000  22799.699219  22823.550781   
450 2024-11-04  22831.800781  22832.650391  22362.650391  22518.300781   

        adj_close    volume   Symbol  day_return  
446  22795.050781  23194400  ^CRSLDX        0.55  
447  22749.800781  27854400  ^CRSLDX        0.08  
448  22689.349609  24362900  ^CRSLDX       -0.30  
449  22823.550781   6042900  ^CRSLDX        0.07  
450  22518.300781         0  ^CRSLDX       -1.37

In [53]:
nifty_ohlcv.rename(columns={'day_return':'nifty500_day_return'},inplace=True)

In [54]:
nifty_ohlcv['nifty500_day_return_next_day'] = nifty_ohlcv['nifty500_day_return'].shift(-1)

In [55]:
nifty_ohlcv.tail()

date          open          high           low         close  \
446 2024-10-29  22670.800781  22808.800781  22450.000000  22795.050781   
447 2024-10-30  22731.150391  22896.550781  22685.400391  22749.800781   
448 2024-10-31  22758.599609  22779.500000  22621.099609  22689.349609   
449 2024-11-01  22806.949219  22881.500000  22799.699219  22823.550781   
450 2024-11-04  22831.800781  22832.650391  22362.650391  22518.300781   

        adj_close    volume   Symbol  nifty500_day_return  \
446  22795.050781  23194400  ^CRSLDX                 0.55   
447  22749.800781  27854400  ^CRSLDX                 0.08   
448  22689.349609  24362900  ^CRSLDX                -0.30   
449  22823.550781   6042900  ^CRSLDX                 0.07   
450  22518.300781         0  ^CRSLDX                -1.37   

     nifty500_day_return_next_day  
446                          0.08  
447                         -0.30  
448                          0.07  
449                         -1.37  
450                           NaN

In [41]:
scoring_df.head(1)

date  open  high   low  close      volume nse_symbol  adj_close  \
43954 2024-02-12  68.0  68.2  58.3  59.95  49978373.0  CENTRALBK      59.95   

       next_day_return_close  target_close  next_day_return_high  target_high  \
43954                   7.29             1                  8.39            1   

       next_day_return_low  target_low  ABER_ZG_14_15  ABER_SG_14_15  \
43954                -1.02           0      61.461905      65.968685   

       ABER_XG_14_15  ABER_ATR_14_15   ACCBL_10  ACCBM_10   ACCBU_10  \
43954      56.955124        4.506781  49.745101    63.855  80.270101   

                 AD   ADOSC_12_26  AMATe_LR_10_20_2  AMATe_SR_10_20_2  \
43954 -5.430455e+07  1.129070e+07                 0                 1   

         AO_5_34           OBV    OBV_min_20    OBV_max_30       OBVs_12  \
43954  13.235147  2.669180e+09  2.035734e+09  2.867782e+09  2.530647e+09   

            OBVs_26  AOBV_LR_2  AOBV_SR_2  APO_12_26    ATRe_5  BBL_5_2.0  \
43954  2.286326e+09          1          0   5.548718  6.940652  58.417759   

       BBM_5_2.0  BBU_5_2.0  BBB_5_2.0  BBP_5_2.0  BIAS_SMA_14       AR_14  \
43954       67.7  76.982241  27.421687   0.082536     -0.02031  112.893983   

       CDL_2CROWS  CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  \
43954         0.0              0.0          0.0              0.0   

       CDL_3OUTSIDE  CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  \
43954           0.0                0.0                 0.0                0.0   

       CDL_ADVANCEBLOCK  CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  \
43954               0.0        -100.0            0.0                  0.0   

       CDL_CONCEALBABYSWALL  CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  \
43954                   0.0                0.0                 0.0   

       CDL_DOJI_10_0.1  CDL_DOJISTAR  CDL_DRAGONFLYDOJI  CDL_ENGULFING  \
43954              0.0           0.0                0.0            0.0   

       CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  CDL_GAPSIDESIDEWHITE  \
43954                  0.0              0.0                   0.0   

       CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  CDL_HARAMI  \
43954                 0.0         0.0             0.0         0.0   

       CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
43954              0.0           0.0          0.0             0.0   

       CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
43954               0.0                  0.0         0.0           0   

       CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
43954                 0.0          0.0                  0.0               0.0   

       CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
43954                 0.0           0.0           0.0              0.0   

       CDL_MATHOLD  CDL_MORNINGDOJISTAR  CDL_MORNINGSTAR  CDL_ONNECK  \
43954          0.0                  0.0              0.0         0.0   

       CDL_PIERCING  CDL_RICKSHAWMAN  CDL_RISEFALL3METHODS  \
43954           0.0              0.0                   0.0   

       CDL_SEPARATINGLINES  CDL_SHOOTINGSTAR  CDL_SHORTLINE  CDL_SPINNINGTOP  \
43954                  0.0               0.0            0.0              0.0   

       CDL_STALLEDPATTERN  CDL_STICKSANDWICH  CDL_TAKURI  CDL_TASUKIGAP  \
43954                 0.0                0.0         0.0            0.0   

       CDL_THRUSTING  CDL_TRISTAR  CDL_UNIQUE3RIVER  CDL_UPSIDEGAP2CROWS  \
43954            0.0          0.0               0.0                  0.0   

       CDL_XSIDEGAP3METHODS    CMF_20     CG_14  CKSPl_10_1.0_9  \
43954                   0.0  0.017723 -7.162075        72.62225   

       CKSPs_10_1.0_9    CMO_14  COPC_11_14_10  DEC_5    DEMA_10    DMP_14  \
43954       51.950771  5.040845      48.188162      1  67.251696  19.51816   

          DMN_14  DCL_20_20  DCM_20_20  DCU_20_20    EMA_20   ENTP_20  \
43954  12.793769      49.25     63.075       76.9  60.16317  4.601525

In [42]:
abv_thr = scoring_df[scoring_df['lgb_prob']>=0.84].sort_values(by=['date','lgb_prob'],ascending=[True,False])

In [43]:
top10_daily =  abv_thr.groupby('date').head(10)[['date','nse_symbol','next_day_return_high','target_high','lgb_prob']]

In [44]:
top10_daily_final = pd.merge(top10_daily,nifty_ohlcv[['date','nifty500_day_return_next_day']],how='left',on='date')

In [45]:
top10_daily_final.tail(20)

date  nse_symbol  next_day_return_high  target_high  lgb_prob  \
669 2024-10-22        NBCC                  3.04            1  0.869280   
670 2024-10-22        GRSE                  3.83            1  0.863053   
671 2024-10-22         BSE                  7.06            1  0.856089   
672 2024-10-22        PCBL                  6.29            1  0.855833   
673 2024-10-22    PNCINFRA                  1.02            0  0.852245   
674 2024-10-22  MANAPPURAM                  5.94            1  0.848912   
675 2024-10-22    NLCINDIA                  2.62            1  0.847647   
676 2024-10-23  COCHINSHIP                  3.12            1  0.842454   
677 2024-10-23  JMFINANCIL                  0.45            0  0.842290   
678 2024-10-24    PNCINFRA                  2.18            1  0.851468   
679 2024-10-24  TRITURBINE                  0.87            0  0.840202   
680 2024-10-25   JINDALSAW                  2.37            1  0.909084   
681 2024-10-25  HINDCOPPER                  6.63            1  0.901976   
682 2024-10-25  MOTILALOFS                  7.64            1  0.877728   
683 2024-10-25    PNCINFRA                  3.97            1  0.875336   
684 2024-10-25        GRSE                  3.60            1  0.871412   
685 2024-10-25  CHENNPETRO                  1.48            0  0.856310   
686 2024-10-25     FINPIPE                  4.72            1  0.853572   
687 2024-10-25         GLS                  4.58            1  0.848394   
688 2024-10-28  TRITURBINE                   NaN            0  0.840896   

     nifty500_day_return_next_day  
669                          0.39  
670                          0.39  
671                          0.39  
672                          0.39  
673                          0.39  
674                          0.39  
675                          0.39  
676                         -0.24  
677                         -0.24  
678                         -1.41  
679                         -1.41  
680                          0.45  
681                          0.45  
682                          0.45  
683                          0.45  
684                          0.45  
685                          0.45  
686                          0.45  
687                          0.45  
688                          0.58

In [46]:
top10_daily_final['target_high'].value_counts(normalize=True).round(2)

1    0.69
0    0.31
Name: target_high, dtype: float64

In [88]:
check_hypothesis = top10_daily_final.groupby('date').agg({'nse_symbol':'count'}).reset_index()

In [89]:
check_hypothesis = pd.merge(check_hypothesis,nifty_ohlcv[['date','nifty500_day_return_next_day']],how='left',on='date')

In [90]:
check_hypothesis.tail(60)

date  nse_symbol  nifty500_day_return_next_day
220 2024-06-21           2                          0.63
221 2024-06-24           3                          0.09
222 2024-06-25           3                          0.26
223 2024-06-26           3                          0.35
224 2024-06-27           1                         -0.14
225 2024-07-01           2                         -0.56
226 2024-07-03           3                         -0.10
227 2024-07-04           4                          0.48
228 2024-07-05           1                         -0.25
229 2024-07-08           1                          0.14
230 2024-07-09           1                         -0.63
231 2024-07-10           2                         -0.24
232 2024-07-12           1                          0.09
233 2024-07-15           1                         -0.23
234 2024-07-16           1                          0.18
235 2024-07-18           3                         -1.63
236 2024-07-19           5                          0.80
237 2024-07-22           1                         -0.68
238 2024-07-23           9                          0.28
239 2024-07-24           2                          0.72
240 2024-07-25           7                          1.40
241 2024-07-26           4                         -0.12
242 2024-07-29           2                          0.08
243 2024-07-30           1                          0.15
244 2024-08-01           1                         -0.16
245 2024-08-05          10                         -1.12
246 2024-08-06           4                          0.33
247 2024-08-12           1                         -1.08
248 2024-08-13           1                         -0.41
249 2024-08-14           1                          0.92
250 2024-08-19           3                          0.24
251 2024-08-20           1                          0.36
252 2024-08-23           1                          0.20
253 2024-08-30           1                         -0.31
254 2024-09-06           1                          0.21
255 2024-09-10           2                         -0.57
256 2024-09-11           1                          0.96
257 2024-09-12           1                         -0.16
258 2024-09-13           1                         -0.16
259 2024-09-16           2                         -0.16
260 2024-09-19           1                          0.99
261 2024-09-20           1                          0.35
262 2024-09-24           1                         -0.02
263 2024-09-25           1                          0.40
264 2024-09-26           1                         -0.16
265 2024-09-27           2                         -0.68
266 2024-10-03           1                         -0.72
267 2024-10-04           1                         -1.84
268 2024-10-07          10                          1.46
269 2024-10-08           2                         -0.00
270 2024-10-11           1                          0.24
271 2024-10-14           1                         -0.27
272 2024-10-17           1                          0.67
273 2024-10-18           2                         -1.12
274 2024-10-21           2                         -1.99
275 2024-10-22           9                          0.39
276 2024-10-23           2                         -0.24
277 2024-10-24           2                         -1.41
278 2024-10-25           8                          0.45
279 2024-10-28           1                          0.58

In [91]:
check_hypothesis.to_excel('hypothesis_check.xlsx')

In [84]:
check_hypothesis['hypothesis'] = np.where((check_hypothesis['nse_symbol']<4) & (check_hypothesis['nifty500_day_return_next_day']<=0),1,
                                         np.where((check_hypothesis['nse_symbol']>6) & (check_hypothesis['nifty500_day_return_next_day']>0),1,
                                                  np.where((check_hypothesis['nse_symbol']>=4) & (check_hypothesis['nse_symbol']<=6),2,0)))

In [85]:
check_hypothesis['hypothesis'].value_counts(normalize=True).round(2)

1    0.48
0    0.43
2    0.09
Name: hypothesis, dtype: float64